In [ ]:
# =============================================================================
# CELL 1: GOOGLE DRIVE CONNECTION AND BASIC SETUP
# =============================================================================
# This cell enables mounting Google Drive in the Google Colab environment.
# The dataset is located in the "Skin_Disease_Classification_Dataset" folder on Drive.
# =============================================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set working directory
import os

# Base directory path of the dataset
# NOTE: You may need to update this path according to your Drive structure
BASE_PATH = '/content/drive/MyDrive/Skin_Disease_Classification_Dataset/dataset'

# Verify directory existence
if os.path.exists(BASE_PATH):
    print("Dataset directory found successfully.")
    print(f"Dizin yolu: {BASE_PATH}")

    # List subdirectories (disease classes)
    classes = os.listdir(BASE_PATH)
    print(f"\nClasses found ({len(classes)}):")
    for cls in sorted(classes):
        cls_path = os.path.join(BASE_PATH, cls)
        if os.path.isdir(cls_path):
            num_images = len([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"   - {cls}: {num_images} images")
else:
    print("ERROR: Dataset directory not found!")
    print(f"   Checked path: {BASE_PATH}")
    print("   Please verify the directory path.")

In [ ]:
# =============================================================================
# CELL 2: IMPORTING REQUIRED LIBRARIES
# =============================================================================
# This cell imports all libraries used throughout the project.
# Libraries are organized by category.
# =============================================================================

# --- Core Python Libraries ---
import os                          # Operating system operations (file/directory management)
import random                      # Random number generation
import warnings                    # Warning message management
warnings.filterwarnings('ignore')  # Suppress unnecessary warnings

# --- Data Processing and Mathematical Operations ---
import numpy as np                 # Numerical computations and array operations
import pandas as pd                # DataFrame operations

# --- Visualization Libraries ---
import matplotlib.pyplot as plt    # Basic plotting
import seaborn as sns              # Statistical visualization
from PIL import Image              # Image processing (Python Imaging Library)

# --- Scikit-learn: Machine Learning Utilities ---
from sklearn.model_selection import train_test_split  # Data partitioning
from sklearn.metrics import (
    classification_report,         # Detailed classification report
    confusion_matrix,              # Confusion matrix
    accuracy_score,                # Accuracy score
    precision_score,               # Kesinlik skoru
    recall_score,                  # Recall score
    f1_score,                      # F1 skoru
    roc_curve,                     # ROC curve
    auc,                           # AUC (Area Under Curve)
    roc_auc_score,                 # ROC AUC skoru
    cohen_kappa_score              # Cohen's Kappa coefficient
)
from sklearn.preprocessing import label_binarize      # Convert labels to binary format
from sklearn.utils.class_weight import compute_class_weight  # Class weight computation

# --- TensorFlow and Keras: Deep Learning Framework ---
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.layers import (
    Conv2D,                        # 2D Convolution layer
    MaxPooling2D,                  # Max pooling
    AveragePooling2D,              # Ortalama havuzlama
    GlobalAveragePooling2D,        # Global ortalama havuzlama
    Dense,                         # Fully connected layer
    Dropout,                       # Dropout regularizasyon
    Flatten,                       # Flatten layer
    BatchNormalization,            # Batch normalizasyon
    Input,                         # Input layer
    Concatenate,                   # Concatenation layer
    Add,                           # Addition layer
    Multiply,                      # Multiplication layer
    Activation,                    # Activation layer
    Reshape,                       # Reshape layer
    Lambda                         # Custom operation layer
)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop  # Optimization algorithms
from tensorflow.keras.callbacks import (
    EarlyStopping,                 # Erken durdurma
    ModelCheckpoint,               # Model checkpointing
    ReduceLROnPlateau,             # Learning rate reduction
    TensorBoard,                   # TensorBoard visualization
    LearningRateScheduler          # Learning rate scheduler
)
from tensorflow.keras.regularizers import l2           # L2 regularizasyon
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,            # Data augmentation and streaming
    load_img,                      # Image loading
    img_to_array                   # Image to array conversion
)

# --- Pre-trained Models for Transfer Learning ---
from tensorflow.keras.applications import (
    VGG16,                         # VGG16 model
    ResNet50,                      # ResNet50 model
    InceptionV3,                   # InceptionV3 model
    EfficientNetB3,                # EfficientNetB3 model
    DenseNet121,                   # DenseNet121 model
    MobileNetV2,                   # MobileNetV2 model
    Xception                       # Xception model
)

# --- Additional Utility Libraries ---
from collections import Counter   # Element counting
import cv2                         # OpenCV - Advanced image processing
from tqdm.notebook import tqdm    # Progress bar (notebook compatible)
import gc                          # Garbage collector (bellek yönetimi)
from datetime import datetime      # Date and time operations

# --- GPU Detection and Configuration ---
print("=" * 60)
print("HARDWARE AND SOFTWARE INFORMATION")
print("=" * 60)

# TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")

# GPU detection
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"🖥️  GPU Bulundu: {len(gpus)} adet")
    for gpu in gpus:
        print(f"   • {gpu.name}")

    # Enable dynamic GPU memory growth (to prevent OOM errors)
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth set to dynamic.")
    except RuntimeError as e:
        print(f"[WARNING] GPU configuration failed: {e}")
else:
    print("[WARNING] No GPU detected; falling back to CPU.")

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print(f"\nRandom seed: {SEED}")

print("\nAll libraries imported successfully.")
print("=" * 60)

In [ ]:
# =============================================================================
# CELL 3: DEFINING CONSTANTS
# =============================================================================
# This cell centrally defines all constant values used throughout the project.
# Centralizing constants facilitates modification and ensures consistency.
# =============================================================================

# --- Dataset Paths ---
BASE_PATH = '/content/drive/MyDrive/Skin_Disease_Classification_Dataset/dataset'
MODEL_SAVE_PATH = '/content/drive/MyDrive/Skin_Disease_Classification_Dataset/models'
RESULTS_PATH = '/content/drive/MyDrive/Skin_Disease_Classification_Dataset/results'

# Create directories if they do not exist
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# --- Class Names ---
# 5 skin disease classes in the dataset
CLASS_NAMES = ['acne', 'hyperpigmentation', 'Nail_psoriasis', 'SJS-TEN', 'Vitiligo']
NUM_CLASSES = len(CLASS_NAMES)  # 5 classes

# Class descriptions (for documentation)
CLASS_NAMES_TR = {
    'acne': 'Akne (Sivilce)',
    'hyperpigmentation': 'Hiperpigmentasyon',
    'Nail_psoriasis': 'Nail Psoriasis',
    'SJS-TEN': 'Stevens-Johnson Sendromu / Toksik Epidermal Nekroliz',
    'Vitiligo': 'Vitiligo'
}

# --- Image Parameters ---
IMG_HEIGHT = 224          # Image height (pixels)
IMG_WIDTH = 224           # Image width (pixels)
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)  # Tuple format
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)  # 3 kanal (RGB)

# Different input size for InceptionV3 and Xception
IMG_SIZE_INCEPTION = (299, 299)
IMG_SHAPE_INCEPTION = (299, 299, 3)

# --- Training Parameters ---
BATCH_SIZE = 32           # Number of images processed per iteration
EPOCHS = 100              # Maximum epoch count (controlled by early stopping)
LEARNING_RATE = 1e-4      # Initial learning rate
VALIDATION_SPLIT = 0.15   # Validation set ratio (15%)
TEST_SPLIT = 0.15         # Test set ratio (15%)

# --- Regularization Parameters ---
DROPOUT_RATE = 0.5        # Dropout rate
L2_LAMBDA = 0.001         # L2 regularization coefficient

# --- Early Stopping and Callback Parameters ---
ES_PATIENCE = 15          # Early stopping patience (epoch count)
LR_PATIENCE = 5           # Learning rate reduction patience
LR_FACTOR = 0.5           # Learning rate reduction factor
MIN_LR = 1e-7             # Minimum learning rate

# --- Data Augmentation Parameters ---
ROTATION_RANGE = 30       # Rotation range (degrees)
WIDTH_SHIFT = 0.2         # Horizontal shift ratio
HEIGHT_SHIFT = 0.2        # Vertical shift ratio
SHEAR_RANGE = 0.2         # Shear range
ZOOM_RANGE = 0.2          # Zoom range
BRIGHTNESS_RANGE = [0.8, 1.2]  # Brightness range

# --- Random Seed Value ---
SEED = 42

# Print configuration
print("=" * 60)
print("PROJECT CONSTANTS")
print("=" * 60)
print(f"Dataset Path: {BASE_PATH}")
print(f"Model Save Path: {MODEL_SAVE_PATH}")
print(f"Results Save Path: {RESULTS_PATH}")
print(f"\nNumber of Classes: {NUM_CLASSES}")
print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Maksimum Epoch: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Validation Ratio: {VALIDATION_SPLIT*100:.0f}%")
print(f"Test Ratio: {TEST_SPLIT*100:.0f}%")
print(f"Dropout Rate: {DROPOUT_RATE}")
print(f"Early Stopping Patience: {ES_PATIENCE}")
print("=" * 60)

In [ ]:
# =============================================================================
# CELL 4: LOADING DATASET AND CREATING DATAFRAME
# =============================================================================
# This cell loads all image file paths and class labels into a pandas
# DataFrame, serving as the foundation for data analysis and model training.
# =============================================================================

def load_dataset_info(base_path, class_names):
    """
    Collects metadata for all images in the dataset and creates a DataFrame.

    Parametreler:
    -----------
    base_path : str
        Veri setinin ana dizin yolu
    class_names : list
        List of class names

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing image paths, class names, and labels
    """

    data = []  # Veri listesi

    print("Scanning dataset...")
    print("-" * 40)

    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(base_path, class_name)

        # Check if directory exists
        if not os.path.exists(class_path):
            print(f"[WARNING] {class_name} directory not found!")
            continue

        # List images in directory
        images = [f for f in os.listdir(class_path)
                  if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]

        # Record metadata for each image
        for img_name in images:
            img_path = os.path.join(class_path, img_name)
            data.append({
                'filepath': img_path,           # Tam dosya yolu
                'filename': img_name,           # Filename
                'class_name': class_name,       # Class name
                'class_idx': class_idx          # Class index (0-4)
            })

        print(f"   [OK] {class_name}: {len(images)} images loaded")

    # Create DataFrame
    df = pd.DataFrame(data)

    print("-" * 40)
    print(f"\nTotal number of images: {len(df)}")

    return df

# Load dataset
df = load_dataset_info(BASE_PATH, CLASS_NAMES)

# Display first rows of the DataFrame
print("\n" + "=" * 60)
print("DATASET PREVIEW (First 10 Rows)")
print("=" * 60)
print(df.head(10))

# DataFrame bilgileri
print("\n" + "=" * 60)
print("DATAFRAME INFORMATION")
print("=" * 60)
print(f"Shape: {df.shape[0]} rows x {df.shape[1]} columns")
print(f"Columns: {list(df.columns)}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Class distribution
print("\n" + "=" * 60)
print("CLASS DISTRIBUTION")
print("=" * 60)
class_distribution = df['class_name'].value_counts()
for class_name, count in class_distribution.items():
    percentage = (count / len(df)) * 100
    print(f"   {class_name}: {count} images ({percentage:.2f}%)")

# EDA

In [ ]:
# =============================================================================
# CELL 5: CLASS DISTRIBUTION VISUALIZATION
# =============================================================================
# This cell visualizes class distribution via bar chart and pie chart.
# The imbalanced dataset condition is analyzed.
# =============================================================================

# Visualization style settings
plt.style.use('seaborn-v0_8-whitegrid')  # Clean and professional appearance
sns.set_palette("husl")  # Renk paleti

# Create figure with 2 side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Left Plot: Bar Chart ---
# Compute class counts
class_counts = df['class_name'].value_counts()

# Create color palette (distinct color per class)
colors = sns.color_palette("husl", n_colors=NUM_CLASSES)

# Draw bar chart
bars = axes[0].bar(range(len(class_counts)), class_counts.values, color=colors, edgecolor='black', linewidth=1.2)

# Annotate each bar with its value
for i, (bar, count) in enumerate(zip(bars, class_counts.values)):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
                 f'{count}', ha='center', va='bottom', fontsize=12, fontweight='bold')
    # Add percentage annotation
    percentage = (count / len(df)) * 100
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height()/2,
                 f'%{percentage:.1f}', ha='center', va='center', fontsize=10,
                 color='white', fontweight='bold')

# X ekseni etiketleri
axes[0].set_xticks(range(len(class_counts)))
axes[0].set_xticklabels(class_counts.index, rotation=45, ha='right', fontsize=11)

# Axis labels and title
axes[0].set_xlabel('Skin Disease Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Image Count', fontsize=12, fontweight='bold')
axes[0].set_title('Image Distribution Per Class\n(Bar Chart)', fontsize=14, fontweight='bold')

# Adjust y-axis limit for label visibility
axes[0].set_ylim(0, max(class_counts.values) * 1.15)

# Grid ekle
axes[0].yaxis.grid(True, linestyle='--', alpha=0.7)

# --- Right Plot: Pie Chart ---
# Slightly explode pie slices
explode = [0.02] * NUM_CLASSES  # Slight separation for each slice

# Draw pie chart
wedges, texts, autotexts = axes[1].pie(
    class_counts.values,
    labels=class_counts.index,
    autopct='%1.1f%%',           # Percentage display
    explode=explode,             # Slice separation
    colors=colors,               # Renkler
    shadow=True,                 # Gölge efekti
    startangle=90,               # Starting angle
    wedgeprops={'edgecolor': 'black', 'linewidth': 1}  # Edge line
)

# Style percentage labels
for autotext in autotexts:
    autotext.set_fontsize(11)
    autotext.set_fontweight('bold')

# Style label text
for text in texts:
    text.set_fontsize(10)

axes[1].set_title('Class Distribution Ratios\n(Pie Chart)', fontsize=14, fontweight='bold')

# Main title
fig.suptitle('SKIN DISEASE DATASET - CLASS DISTRIBUTION ANALYSIS',
             fontsize=16, fontweight='bold', y=1.02)

# Adjust layout
plt.tight_layout()

# Save figure
plt.savefig(f'{RESULTS_PATH}/01_sinif_dagilimi.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
print(f"Figure saved: {RESULTS_PATH}/01_sinif_dagilimi.png")

plt.show()

# --- Imbalance Analysis ---
print("\n" + "=" * 60)
print("DATA IMBALANCE ANALYSIS")
print("=" * 60)

max_class = class_counts.idxmax()
min_class = class_counts.idxmin()
max_count = class_counts.max()
min_count = class_counts.min()
imbalance_ratio = max_count / min_count

print(f"Majority class: {max_class} ({max_count} images)")
print(f"Minority class: {min_class} ({min_count} images)")
print(f"Imbalance ratio (max/min): {imbalance_ratio:.2f}x")

if imbalance_ratio > 2:
    print(f"\n[WARNING] Dataset is imbalanced!")
    print("   Mitigation strategies:")
    print("   1. Class weights will be applied")
    print("   2. Additional data augmentation for minority classes")
    print("   3. Stratified sampling for data splitting")
else:
    print("\nDataset is relatively balanced.")

In [ ]:
# =============================================================================
# CELL 6: IMAGE DIMENSION ANALYSIS (FULL DATASET - MULTI-THREADED)
# =============================================================================
# This cell analyzes dimensions of ALL images in the dataset.
# Multi-threading is used for parallel processing to optimize execution time.
# Full data analysis is essential for academic rigor - no data points are skipped.
# =============================================================================

import concurrent.futures

def get_image_info_fast(args):
    """
    Quickly reads the dimensions of a single image.
    Uses PIL lazy loading - reads only header information,
    without loading the full image into memory.

    Parametreler:
    -----------
    args : tuple
        (filepath, class_name) tuple

    Returns:
    --------
    dict veya None
        Image dimension information or None on error
    """
    filepath, class_name = args
    try:
        with Image.open(filepath) as img:
            width, height = img.size
            return {
                'width': width,
                'height': height,
                'aspect_ratio': width / height if height > 0 else 0,
                'class_name': class_name
            }
    except Exception as e:
        return None

def analyze_all_images_parallel(df, max_workers=32):
    """
    Analyzes dimensions of all images in parallel.

    Parametreler:
    -----------
    df : pandas.DataFrame
        DataFrame containing image metadata
    max_workers : int
        Number of parallel threads (32 optimal for A100)

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing dimension statistics for all images
    """

    print(f"Analyzing all {len(df)} images (Multi-threading: {max_workers} workers)")
    print("This operation may take approximately 5-10 minutes...\n")

    # Prepare arguments
    args_list = [(row['filepath'], row['class_name']) for _, row in df.iterrows()]

    # Parallel processing via multi-threading
    dimensions = []
    failed_count = 0

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Progress bar via tqdm
        results = list(tqdm(
            executor.map(get_image_info_fast, args_list),
            total=len(args_list),
            desc="Boyut analizi",
            unit=" images"
        ))

    # Filter results (exclude None values)
    for result in results:
        if result is not None:
            dimensions.append(result)
        else:
            failed_count += 1

    if failed_count > 0:
        print(f"\n[WARNING] {failed_count} images could not be read (corrupt or inaccessible)")

    print(f"{len(dimensions)} images analyzed successfully.")

    return pd.DataFrame(dimensions)

# =============================================================================
# RUN ANALYSIS
# =============================================================================
print("=" * 60)
print("IMAGE DIMENSION ANALYSIS (FULL DATASET)")
print("=" * 60)

# Analyze all images
dim_df = analyze_all_images_parallel(df, max_workers=32)

# =============================================================================
# STATISTICAL SUMMARY
# =============================================================================
print("\n" + "=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)

print(f"\nWIDTH Statistics:")
print(f"   • Minimum  : {dim_df['width'].min()} piksel")
print(f"   • Maksimum : {dim_df['width'].max()} piksel")
print(f"   • Ortalama : {dim_df['width'].mean():.2f} piksel")
print(f"   • Medyan   : {dim_df['width'].median():.2f} piksel")
print(f"   • Std Sapma: {dim_df['width'].std():.2f} piksel")

print(f"\nHEIGHT Statistics:")
print(f"   • Minimum  : {dim_df['height'].min()} piksel")
print(f"   • Maksimum : {dim_df['height'].max()} piksel")
print(f"   • Ortalama : {dim_df['height'].mean():.2f} piksel")
print(f"   • Medyan   : {dim_df['height'].median():.2f} piksel")
print(f"   • Std Sapma: {dim_df['height'].std():.2f} piksel")

print(f"\nASPECT RATIO Statistics:")
print(f"   • Minimum  : {dim_df['aspect_ratio'].min():.3f}")
print(f"   • Maksimum : {dim_df['aspect_ratio'].max():.3f}")
print(f"   • Ortalama : {dim_df['aspect_ratio'].mean():.3f}")
print(f"   • Medyan   : {dim_df['aspect_ratio'].median():.3f}")

# =============================================================================
# VISUALIZATION
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Renk paleti
colors = sns.color_palette("husl", n_colors=NUM_CLASSES)

# --- 1. Width Distribution (Histogram) ---
axes[0, 0].hist(dim_df['width'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(dim_df['width'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Ortalama: {dim_df["width"].mean():.0f} px')
axes[0, 0].axvline(dim_df['width'].median(), color='orange', linestyle=':', linewidth=2,
                   label=f'Medyan: {dim_df["width"].median():.0f} px')
axes[0, 0].axvline(224, color='green', linestyle='-', linewidth=2, label='Model Hedef: 224 px')
axes[0, 0].set_xlabel('Width (pixels)', fontsize=11)
axes[0, 0].set_ylabel('Frekans', fontsize=11)
axes[0, 0].set_title('Image Width Distribution', fontsize=13, fontweight='bold')
axes[0, 0].legend(loc='upper right', fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# --- 2. Height Distribution (Histogram) ---
axes[0, 1].hist(dim_df['height'], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(dim_df['height'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Ortalama: {dim_df["height"].mean():.0f} px')
axes[0, 1].axvline(dim_df['height'].median(), color='orange', linestyle=':', linewidth=2,
                   label=f'Medyan: {dim_df["height"].median():.0f} px')
axes[0, 1].axvline(224, color='green', linestyle='-', linewidth=2, label='Model Hedef: 224 px')
axes[0, 1].set_xlabel('Height (pixels)', fontsize=11)
axes[0, 1].set_ylabel('Frekans', fontsize=11)
axes[0, 1].set_title('Image Height Distribution', fontsize=13, fontweight='bold')
axes[0, 1].legend(loc='upper right', fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# --- 3. Width vs Height (Scatter Plot - Per Class) ---
for i, class_name in enumerate(CLASS_NAMES):
    class_data = dim_df[dim_df['class_name'] == class_name]
    axes[1, 0].scatter(class_data['width'], class_data['height'],
                       alpha=0.5, s=15, label=class_name, color=colors[i])

# 1:1 aspect ratio line
max_dim = max(dim_df['width'].max(), dim_df['height'].max())
axes[1, 0].plot([0, max_dim], [0, max_dim], 'r--', linewidth=2, label='1:1 Ratio (Square)')

# Target size marker
axes[1, 0].scatter([224], [224], color='black', s=200, marker='*',
                   zorder=5, label='Hedef: 224×224')

axes[1, 0].set_xlabel('Width (pixels)', fontsize=11)
axes[1, 0].set_ylabel('Height (pixels)', fontsize=11)
axes[1, 0].set_title('Width vs Height (Per Class)', fontsize=13, fontweight='bold')
axes[1, 0].legend(loc='upper left', fontsize=8, ncol=2)
axes[1, 0].grid(True, alpha=0.3)

# --- 4. Per-Class Dimension Distribution (Box Plot) ---
dim_df['avg_size'] = (dim_df['width'] + dim_df['height']) / 2
box_plot = sns.boxplot(data=dim_df, x='class_name', y='avg_size', palette='husl', ax=axes[1, 1])
axes[1, 1].axhline(224, color='green', linestyle='--', linewidth=2, label='Hedef: 224 px')
axes[1, 1].set_xlabel('Skin Disease Class', fontsize=11)
axes[1, 1].set_ylabel('Ortalama Boyut (piksel)', fontsize=11)
axes[1, 1].set_title('Per-Class Image Dimension Distribution', fontsize=13, fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].legend(loc='upper right', fontsize=9)
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Main title
fig.suptitle('IMAGE DIMENSION ANALYSIS - FULL DATASET (n=3411)',
             fontsize=15, fontweight='bold', y=1.02)

plt.tight_layout()

# Save figure
save_path = f'{RESULTS_PATH}/02_goruntu_boyutlari.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
print(f"\nFigure saved: {save_path}")

plt.show()

# =============================================================================
# DETAILED STATISTICS
# =============================================================================
print("\n" + "=" * 60)
print("PER-CLASS DIMENSION STATISTICS")
print("=" * 60)

class_stats = dim_df.groupby('class_name').agg({
    'width': ['min', 'max', 'mean', 'std'],
    'height': ['min', 'max', 'mean', 'std'],
    'aspect_ratio': ['min', 'max', 'mean']
}).round(2)

print(class_stats)

# =============================================================================
# UNIQUE DIMENSIONS
# =============================================================================
print("\n" + "=" * 60)
print("UNIQUE IMAGE DIMENSIONS")
print("=" * 60)

unique_sizes = dim_df.groupby(['width', 'height']).size().reset_index(name='count')
unique_sizes = unique_sizes.sort_values('count', ascending=False)
print(f"Total unique dimension combinations: {len(unique_sizes)}")
print(f"\nTop 15 most common dimensions:")
print(unique_sizes.head(15).to_string(index=False))

# =============================================================================
# OUTLIER ANALYSIS
# =============================================================================
print("\n" + "=" * 60)
print("OUTLIER ANALYSIS")
print("=" * 60)

# Outlier detection via IQR method
Q1_w = dim_df['width'].quantile(0.25)
Q3_w = dim_df['width'].quantile(0.75)
IQR_w = Q3_w - Q1_w

Q1_h = dim_df['height'].quantile(0.25)
Q3_h = dim_df['height'].quantile(0.75)
IQR_h = Q3_h - Q1_h

outliers_width = dim_df[(dim_df['width'] < Q1_w - 1.5*IQR_w) | (dim_df['width'] > Q3_w + 1.5*IQR_w)]
outliers_height = dim_df[(dim_df['height'] < Q1_h - 1.5*IQR_h) | (dim_df['height'] > Q3_h + 1.5*IQR_h)]

print(f"Width outlier count: {len(outliers_width)} ({len(outliers_width)/len(dim_df)*100:.2f}%)")
print(f"Height outlier count: {len(outliers_height)} ({len(outliers_height)/len(dim_df)*100:.2f}%)")

# Very small and very large images
very_small = dim_df[(dim_df['width'] < 100) | (dim_df['height'] < 100)]
very_large = dim_df[(dim_df['width'] > 1000) | (dim_df['height'] > 1000)]

print(f"\nVery small images (<100px): {len(very_small)}")
print(f"Very large images (>1000px): {len(very_large)}")

# =============================================================================
# RECOMMENDATIONS FOR MODEL TRAINING
# =============================================================================
print("\n" + "=" * 60)
print("RECOMMENDATIONS FOR MODEL TRAINING")
print("=" * 60)
print(f"[OK] All images will be resized to 224x224")
print(f"Ortalama orijinal boyut: {dim_df['width'].mean():.0f}×{dim_df['height'].mean():.0f}")
print(f"[OK] High dimensional variability - resizing is necessary")
print(f"[OK] Bilinear/bicubic interpolation will be used to preserve image quality")
print(f"[OK] Data augmentation will increase size variation coverage")

In [ ]:
# =============================================================================
# CELL 7: SAMPLE IMAGE VISUALIZATION PER CLASS
# =============================================================================
# This cell displays randomly selected sample images from each skin disease class.
# This is an essential EDA step for understanding dataset content.
# =============================================================================

def display_sample_images(df, class_names, samples_per_class=5, figsize=(20, 15)):
    """
    Displays random sample images from each class.

    Parametreler:
    -----------
    df : pandas.DataFrame
        DataFrame containing image metadata
    class_names : list
        List of class names
    samples_per_class : int
        Number of samples to display per class
    figsize : tuple
        Figure boyutu
    """

    num_classes = len(class_names)

    # Create figure
    fig, axes = plt.subplots(num_classes, samples_per_class, figsize=figsize)

    # Select and display sample images for each class
    for row_idx, class_name in enumerate(class_names):
        # Filter images belonging to this class
        class_df = df[df['class_name'] == class_name]

        # Select random samples
        samples = class_df.sample(n=min(samples_per_class, len(class_df)), random_state=SEED)

        for col_idx, (_, sample) in enumerate(samples.iterrows()):
            ax = axes[row_idx, col_idx]

            try:
                # Load image
                img = Image.open(sample['filepath'])

                # Display image
                ax.imshow(img)

                # Write class name in first column
                if col_idx == 0:
                    ax.set_ylabel(f"{class_name}\n({CLASS_NAMES_TR[class_name]})",
                                  fontsize=10, fontweight='bold', rotation=0,
                                  ha='right', va='center', labelpad=60)

                # Add image dimensions as subtitle
                ax.set_title(f'{img.size[0]}×{img.size[1]}', fontsize=9)

            except Exception as e:
                ax.text(0.5, 0.5, f'Load failed\n{e}', ha='center', va='center',
                       transform=ax.transAxes, fontsize=8)

            # Eksenleri kapat
            ax.set_xticks([])
            ax.set_yticks([])

    # Main title
    fig.suptitle('SAMPLE IMAGES FROM SKIN DISEASE CLASSES',
                 fontsize=16, fontweight='bold', y=1.02)

    plt.tight_layout()
    return fig

# Display sample images
print("=" * 60)
print("SAMPLE IMAGES")
print("=" * 60)
print("Displaying 5 random sample images per class...\n")

fig = display_sample_images(df, CLASS_NAMES, samples_per_class=5, figsize=(18, 14))

# Save figure
plt.savefig(f'{RESULTS_PATH}/03_ornek_goruntuler.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
print(f"Figure saved: {RESULTS_PATH}/03_ornek_goruntuler.png")

plt.show()

# --- Disease Descriptions ---
print("\n" + "=" * 60)
print("SKIN DISEASE INFORMATION")
print("=" * 60)

disease_info = {
    'acne': """
    ACNE: A common skin condition resulting from hair follicles becoming clogged
    with oil and dead skin cells. Typically appears on the face, forehead, chest,
    back, and shoulders.""",

    'hyperpigmentation': """
    HYPERPIGMENTATION: Dark-colored patches caused by excess melanin production
    in specific areas of the skin. Can result from sun damage, hormonal changes,
    or skin injuries.""",

    'Nail_psoriasis': """
    NAIL PSORIASIS: A form of psoriasis affecting the nails.
    Symptoms include pitting, discoloration, thickening, and separation of
    belirtilere neden olur.""",

    'SJS-TEN': """
    STEVENS-JOHNSON SYNDROME / TOXIC EPIDERMAL NECROLYSIS: A severe and
    potentially fatal skin reaction, usually drug-induced.
    Characterized by rash, blistering, and skin peeling.""",

    'Vitiligo': """
    VITILIGO: A pigmentation disorder characterized by white patches
    resulting from the destruction of melanin-producing cells (melanocytes)
    in the skin."""
}

for disease, info in disease_info.items():
    print(f"\n🔬 {CLASS_NAMES_TR[disease].upper()}")
    print("-" * 40)
    print(info.strip())

In [ ]:
# =============================================================================
# CELL 8: COLOR DISTRIBUTION ANALYSIS (RGB HISTOGRAM)
# =============================================================================
# This cell analyzes the RGB channel distributions for images in each class.
# Understanding the color characteristics of different skin diseases provides
# valuable insights for model performance.
# =============================================================================

def analyze_color_distribution(df, class_names, sample_size=100):
    """
    Analyzes RGB color distribution for each class.

    Parametreler:
    -----------
    df : pandas.DataFrame
        DataFrame containing image metadata
    class_names : list
        Class names
    sample_size : int
        Number of samples to analyze per class

    Returns:
    --------
    dict
        Mean RGB values per class
    """

    color_stats = {cls: {'R': [], 'G': [], 'B': []} for cls in class_names}

    print("Performing color analysis...")

    for class_name in tqdm(class_names, desc="Classes"):
        # Get images for this class
        class_df = df[df['class_name'] == class_name]

        # Select random samples
        samples = class_df.sample(n=min(sample_size, len(class_df)), random_state=SEED)

        for _, row in samples.iterrows():
            try:
                # Load image and convert to numpy array
                img = Image.open(row['filepath']).convert('RGB')
                img_array = np.array(img)

                # Compute mean value for each channel
                color_stats[class_name]['R'].extend(img_array[:, :, 0].flatten())
                color_stats[class_name]['G'].extend(img_array[:, :, 1].flatten())
                color_stats[class_name]['B'].extend(img_array[:, :, 2].flatten())

            except Exception as e:
                continue

    return color_stats

# Run color analysis
print("=" * 60)
print("RGB COLOR DISTRIBUTION ANALYSIS")
print("=" * 60)

color_stats = analyze_color_distribution(df, CLASS_NAMES, sample_size=100)

# --- Visualization ---
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# RGB histogram per class
for idx, class_name in enumerate(CLASS_NAMES):
    ax = axes[idx]

    # Draw RGB histograms
    ax.hist(color_stats[class_name]['R'], bins=50, alpha=0.6, color='red',
            label='Red', density=True)
    ax.hist(color_stats[class_name]['G'], bins=50, alpha=0.6, color='green',
            label='Green', density=True)
    ax.hist(color_stats[class_name]['B'], bins=50, alpha=0.6, color='blue',
            label='Blue', density=True)

    # Compute and display mean values
    r_mean = np.mean(color_stats[class_name]['R'])
    g_mean = np.mean(color_stats[class_name]['G'])
    b_mean = np.mean(color_stats[class_name]['B'])

    ax.axvline(r_mean, color='darkred', linestyle='--', linewidth=2)
    ax.axvline(g_mean, color='darkgreen', linestyle='--', linewidth=2)
    ax.axvline(b_mean, color='darkblue', linestyle='--', linewidth=2)

    ax.set_xlabel('Pixel Value (0-255)', fontsize=10)
    ax.set_ylabel('Density', fontsize=10)
    ax.set_title(f'{class_name}\nR:{r_mean:.1f} | G:{g_mean:.1f} | B:{b_mean:.1f}',
                 fontsize=11, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xlim(0, 255)
    ax.grid(True, alpha=0.3)

# Use last subplot for cross-class comparison
ax = axes[5]

# Compute mean RGB values per class
class_means = []
for class_name in CLASS_NAMES:
    r_mean = np.mean(color_stats[class_name]['R'])
    g_mean = np.mean(color_stats[class_name]['G'])
    b_mean = np.mean(color_stats[class_name]['B'])
    class_means.append([r_mean, g_mean, b_mean])

class_means = np.array(class_means)

# Grouped bar chart
x = np.arange(len(CLASS_NAMES))
width = 0.25

bars1 = ax.bar(x - width, class_means[:, 0], width, label='Red', color='red', alpha=0.7)
bars2 = ax.bar(x, class_means[:, 1], width, label='Green', color='green', alpha=0.7)
bars3 = ax.bar(x + width, class_means[:, 2], width, label='Blue', color='blue', alpha=0.7)

ax.set_xlabel('Class', fontsize=10)
ax.set_ylabel('Mean Pixel Value', fontsize=10)
ax.set_title('Per-Class Mean RGB Values', fontsize=11, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=9)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

# Main title
fig.suptitle('SKIN DISEASE CLASS COLOR DISTRIBUTION ANALYSIS',
             fontsize=14, fontweight='bold', y=1.02)

plt.tight_layout()

# Save figure
plt.savefig(f'{RESULTS_PATH}/04_renk_dagilimi.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
print(f"\nFigure saved: {RESULTS_PATH}/04_renk_dagilimi.png")

plt.show()

# --- Color Statistics Table ---
print("\n" + "=" * 60)
print("PER-CLASS MEAN RGB VALUES")
print("=" * 60)

color_table = []
for idx, class_name in enumerate(CLASS_NAMES):
    r_mean = np.mean(color_stats[class_name]['R'])
    g_mean = np.mean(color_stats[class_name]['G'])
    b_mean = np.mean(color_stats[class_name]['B'])
    brightness = (r_mean + g_mean + b_mean) / 3
    color_table.append({
        'Class': class_name,
        'R (Red)': f'{r_mean:.2f}',
        'G (Green)': f'{g_mean:.2f}',
        'B (Mavi)': f'{b_mean:.2f}',
        'Brightness': f'{brightness:.2f}'
    })

color_df = pd.DataFrame(color_table)
print(color_df.to_string(index=False))

# Cell 9: Train/Validation/Test Split

In [ ]:
# =============================================================================
# CELL 9: TRAIN/VALIDATION/TEST SPLIT
# =============================================================================
# This cell splits the dataset into three partitions via stratified sampling:
# - Training Set (70%): For model training
# - Validation Set (15%): For hyperparameter tuning and early stopping
# - Test Set (15%): For final model evaluation
#
# Stratified sampling preserves class proportions across all partitions.
# This is critically important for imbalanced datasets.
# =============================================================================

from sklearn.model_selection import train_test_split

def create_stratified_split(df, test_size=0.15, val_size=0.15, random_state=42):
    """
    Splits the dataset into train/validation/test partitions using stratified sampling.

    Parametreler:
    -----------
    df : pandas.DataFrame
        DataFrame containing the full dataset
    test_size : float
        Test set ratio (default: 0.15 = 15%)
    val_size : float
        Validation set ratio (default: 0.15 = 15%)
    random_state : int
        Random seed for reproducibility

    Returns:
    --------
    tuple
        (train_df, val_df, test_df) - Three separate DataFrames
    """

    print("=" * 60)
    print("DATASET PARTITIONING (STRATIFIED SPLIT)")
    print("=" * 60)

    # First, separate the test set
    # The stratify parameter preserves class proportions
    train_val_df, test_df = train_test_split(
        df,
        test_size=test_size,
        stratify=df['class_name'],  # Preserve class distribution
        random_state=random_state
    )

    # Separate validation set from remaining data
    # Recalculate val_size (relative to train_val)
    val_ratio = val_size / (1 - test_size)

    train_df, val_df = train_test_split(
        train_val_df,
        test_size=val_ratio,
        stratify=train_val_df['class_name'],
        random_state=random_state
    )

    # Reset DataFrame indices
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    return train_df, val_df, test_df

# Split the dataset
train_df, val_df, test_df = create_stratified_split(
    df,
    test_size=TEST_SPLIT,
    val_size=VALIDATION_SPLIT,
    random_state=SEED
)

# =============================================================================
# DISPLAY PARTITIONING RESULTS
# =============================================================================

print(f"\nPARTITIONING RESULTS")
print("-" * 60)
print(f"{'Set':<15} {'Image Count':<20} {'Ratio':<15}")
print("-" * 60)
print(f"{'Training':<15} {len(train_df):<20} {len(train_df)/len(df)*100:.1f}%")
print(f"{'Validation':<15} {len(val_df):<20} {len(val_df)/len(df)*100:.1f}%")
print(f"{'Test':<15} {len(test_df):<20} {len(test_df)/len(df)*100:.1f}%")
print("-" * 60)
print(f"{'TOPLAM':<15} {len(train_df)+len(val_df)+len(test_df):<20} 100.0%")

# =============================================================================
# VERIFY CLASS DISTRIBUTIONS
# =============================================================================

print("\n" + "=" * 60)
print("CLASS DISTRIBUTION VERIFICATION (Stratified Sampling Validation)")
print("=" * 60)

# Compute class distribution for each set
def get_class_distribution(dataframe, set_name):
    """Computes class distribution within a DataFrame."""
    dist = dataframe['class_name'].value_counts()
    percentages = (dist / len(dataframe) * 100).round(2)
    return percentages

train_dist = get_class_distribution(train_df, "Training")
val_dist = get_class_distribution(val_df, "Validation")
test_dist = get_class_distribution(test_df, "Test")
original_dist = get_class_distribution(df, "Original")

# Build comparison table
comparison_df = pd.DataFrame({
    'Original (%)': original_dist,
    'Train (%)': train_dist,
    'Validation (%)': val_dist,
    'Test (%)': test_dist
})

print("\nClass Distribution Comparison:")
print(comparison_df.to_string())

# Verify distribution consistency
max_deviation = 0
for class_name in CLASS_NAMES:
    orig = original_dist[class_name]
    train_pct = train_dist[class_name]
    val_pct = val_dist[class_name]
    test_pct = test_dist[class_name]

    deviation = max(abs(orig - train_pct), abs(orig - val_pct), abs(orig - test_pct))
    max_deviation = max(max_deviation, deviation)

print(f"\nMaksimum sapma: {max_deviation:.2f}%")
if max_deviation < 1.0:
    print("   Stratified sampling successful - distributions are consistent.")
else:
    print("   [WARNING] Minor deviations detected (expected behavior).")

# =============================================================================
# VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sets = [('Training Set', train_df), ('Validation Set', val_df), ('Test Set', test_df)]
colors = sns.color_palette("husl", n_colors=NUM_CLASSES)

for idx, (set_name, set_df) in enumerate(sets):
    class_counts = set_df['class_name'].value_counts()

    # Draw bar chart preserving class order
    counts = [class_counts.get(cls, 0) for cls in CLASS_NAMES]

    bars = axes[idx].bar(range(len(CLASS_NAMES)), counts, color=colors, edgecolor='black')

    # Annotate values above bars
    for bar, count in zip(bars, counts):
        axes[idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                       str(count), ha='center', va='bottom', fontsize=10, fontweight='bold')

    axes[idx].set_xticks(range(len(CLASS_NAMES)))
    axes[idx].set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=9)
    axes[idx].set_ylabel('Image Count', fontsize=11)
    axes[idx].set_title(f'{set_name}\n(n={len(set_df)})', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='y')

fig.suptitle('DATASET PARTITIONING - CLASS DISTRIBUTIONS', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

# Save figure
plt.savefig(f'{RESULTS_PATH}/05_veri_bolumlemesi.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
print(f"\nFigure saved: {RESULTS_PATH}/05_veri_bolumlemesi.png")

plt.show()

# =============================================================================
# DETAILED STATISTICS PER SET
# =============================================================================

print("\n" + "=" * 60)
print("DETAILED SET STATISTICS")
print("=" * 60)

for set_name, set_df in [('Training', train_df), ('Validation', val_df), ('Test', test_df)]:
    print(f"\n{set_name} Set:")
    for class_name in CLASS_NAMES:
        count = len(set_df[set_df['class_name'] == class_name])
        pct = count / len(set_df) * 100
        print(f"   - {class_name}: {count} images ({pct:.2f}%)")

# Cell 10: Class Weights Computation (Imbalanced Data)

In [ ]:
# =============================================================================
# CELL 10: CLASS WEIGHTS COMPUTATION (FOR IMBALANCED DATA)
# =============================================================================
# In imbalanced datasets, class weights ensure adequate learning of minority classes.
# These weights adjust each class's contribution to the loss function.
# Formula: weight_i = n_samples / (n_classes * n_samples_i)
#
# This formula assigns higher weights to underrepresented classes.

# =============================================================================

def calculate_class_weights(train_df, class_names):
    """
    Computes class weights for the training set.

    Parametreler:
    -----------
    train_df : pandas.DataFrame
        Training veri seti
    class_names : list
        List of class names

    Returns:
    --------
    dict
        Class index -> weight mapping
    """

    print("=" * 60)
    print("CLASS WEIGHTS HESAPLAMA")
    print("=" * 60)

    # Extract class labels
    y_train = train_df['class_idx'].values

    # Compute class weights via sklearn
    # 'balanced' modu: n_samples / (n_classes * np.bincount(y))
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )

    # Convert to dictionary format
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights_array)}

    # Print results
    print("\nComputed Class Weights:")
    print("-" * 50)
    print(f"{'Class Index':<15} {'Class Name':<20} {'Weight':<15}")
    print("-" * 50)

    for idx, class_name in enumerate(class_names):
        weight = class_weights_dict[idx]
        sample_count = len(train_df[train_df['class_idx'] == idx])
        print(f"{idx:<15} {class_name:<20} {weight:.4f}")

    return class_weights_dict

# Compute class weights
class_weights = calculate_class_weights(train_df, CLASS_NAMES)

# =============================================================================
# VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Sample count
sample_counts = [len(train_df[train_df['class_name'] == cls]) for cls in CLASS_NAMES]
colors = sns.color_palette("husl", n_colors=NUM_CLASSES)

bars1 = axes[0].bar(range(len(CLASS_NAMES)), sample_counts, color=colors, edgecolor='black')
for bar, count in zip(bars1, sample_counts):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
                 str(count), ha='center', va='bottom', fontsize=10, fontweight='bold')
axes[0].set_xticks(range(len(CLASS_NAMES)))
axes[0].set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=10)
axes[0].set_ylabel('Sample Count', fontsize=11)
axes[0].set_title('Training Set - Class Distribution', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Right: Class weights
weights = [class_weights[i] for i in range(len(CLASS_NAMES))]
bars2 = axes[1].bar(range(len(CLASS_NAMES)), weights, color=colors, edgecolor='black')
for bar, weight in zip(bars2, weights):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                 f'{weight:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
axes[1].set_xticks(range(len(CLASS_NAMES)))
axes[1].set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=10)
axes[1].set_ylabel('Class Weight', fontsize=11)
axes[1].set_title('Hesaplanan Class Weights', fontsize=12, fontweight='bold')
axes[1].axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Baseline (1.0)')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3, axis='y')

fig.suptitle('IMBALANCED DATA MANAGEMENT - CLASS WEIGHTS', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

# Save figure
plt.savefig(f'{RESULTS_PATH}/06_class_weights.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
print(f"\nFigure saved: {RESULTS_PATH}/06_class_weights.png")

plt.show()

# =============================================================================
# EXPLANATION
# =============================================================================

print("\n" + "=" * 60)
print("CLASS WEIGHTS EXPLANATION")
print("=" * 60)

print("""
How Do Class Weights Work?

1. OBJECTIVE: Ensure adequate learning of minority classes in imbalanced datasets

2. FORMULA: weight = n_samples / (n_classes x n_samples_in_class)
   - Fewer samples -> Higher weight
   - More samples -> Lower weight

3. EFFECT:
   - Each sample's loss contribution is multiplied by its weight
   - Errors on minority classes are penalized more heavily
   - The model learns all classes in a balanced manner

4. UYGULAMA:
   - Used via model.fit(..., class_weight=class_weights)
   - Applied automatically at each epoch
""")

# Weight interpretation
max_weight_idx = np.argmax(weights)
min_weight_idx = np.argmin(weights)

print(f"\nYORUM:")
print(f"   - Highest weight: {CLASS_NAMES[max_weight_idx]} ({weights[max_weight_idx]:.3f})")
print(f"     -> The least represented class receives the highest weight")
print(f"   - Lowest weight: {CLASS_NAMES[min_weight_idx]} ({weights[min_weight_idx]:.3f})")
print(f"     -> The most represented class receives the lowest weight")
print(f"   - Weight ratio: {weights[max_weight_idx]/weights[min_weight_idx]:.2f}x")

# Cell 11: Data Generator Construction (Data Augmentation)

In [ ]:
# =============================================================================
# CELL 11: TF.DATA PIPELINE (PREPROCESSING HANDLED INSIDE MODEL)
# =============================================================================
# IMPORTANT CHANGE:
# - Normalizasyon (rescaling) KALDIRILDI
# - Each model will use its own preprocessing
# - This ensures EfficientNet, ResNet, VGG all receive correctly preprocessed inputs
# =============================================================================

print("=" * 70)
print("TF.DATA PIPELINE (PREPROCESSING INSIDE MODEL)")
print("=" * 70)

BATCH_SIZE = 64  # Reduced from 256 to 64 (more stable training)
AUTOTUNE = tf.data.AUTOTUNE
SHUFFLE_BUFFER = 1000

print(f"\n⚡ Ayarlar:")
print(f"   • Batch Size: {BATCH_SIZE}")
print(f"   • Shuffle Buffer: {SHUFFLE_BUFFER}")
print(f"   - Preprocessing: INSIDE MODEL (model-specific)")

# =============================================================================
# CLASS MAPPING
# =============================================================================

class_names_sorted = sorted(df['class_name'].unique().tolist())
class_indices = {name: idx for idx, name in enumerate(class_names_sorted)}
idx_to_class = {idx: name for name, idx in class_indices.items()}
class_names_ordered = class_names_sorted

print(f"\nClass Mapping:")
for name, idx in class_indices.items():
    print(f"   {name} → {idx}")

# =============================================================================
# DATA AUGMENTATION (GPU TABANLI)
# =============================================================================

print("\n" + "=" * 70)
print("DATA AUGMENTATION")
print("=" * 70)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
], name="data_augmentation")

print("Augmentation: RandomFlip, RandomRotation(±10%), RandomZoom(±10%)")

# =============================================================================
# IMAGE LOADING (NO PREPROCESSING)
# =============================================================================

def load_image_no_preprocess(filepath, label):
    """
    Loads and resizes the image.
    NO PREPROCESSING - handled inside the model.
    """
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    # NOTE: No normalization applied. Images remain in [0, 255] range.
    return image, label

# =============================================================================
# DATASET CONSTRUCTION
# =============================================================================

def create_dataset_no_preprocess(filepaths, labels, is_training=False):
    """
    Creates dataset WITHOUT preprocessing.
    Her model kendi preprocessing'ini yapacak.
    """
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    if is_training:
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER, seed=SEED)

    dataset = dataset.map(load_image_no_preprocess, num_parallel_calls=AUTOTUNE)

    # Augmentation applied to training set only
    if is_training:
        dataset = dataset.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE
        )

    dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)

    return dataset

# =============================================================================
# DATASETLER
# =============================================================================

print("\n" + "=" * 70)
print("CONSTRUCTING DATASETS")
print("=" * 70)

def prepare_labels(dataframe):
    labels = dataframe['class_name'].map(class_indices).values
    return tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

print("\nTraining Dataset...")
train_filepaths = train_df['filepath'].values
train_labels = prepare_labels(train_df)
train_dataset = create_dataset_no_preprocess(train_filepaths, train_labels, is_training=True)
print("   [OK] Preprocessing: None (handled inside model)")
print("   [OK] Augmentation: ENABLED")

print("\nValidation Dataset...")
val_filepaths = val_df['filepath'].values
val_labels = prepare_labels(val_df)
val_dataset = create_dataset_no_preprocess(val_filepaths, val_labels, is_training=False)
print("   [OK] Preprocessing: None (handled inside model)")
print("   Augmentation: KAPALI")

print("\nTest Dataset...")
test_filepaths = test_df['filepath'].values
test_labels = prepare_labels(test_df)
test_dataset = create_dataset_no_preprocess(test_filepaths, test_labels, is_training=False)
print("   [OK] Preprocessing: None (handled inside model)")
print("   Augmentation: KAPALI")

# =============================================================================
# INFORMATION
# =============================================================================

steps_per_epoch = int(np.ceil(len(train_df) / BATCH_SIZE))
validation_steps = int(np.ceil(len(val_df) / BATCH_SIZE))
test_steps = int(np.ceil(len(test_df) / BATCH_SIZE))

print(f"\nDataset Bilgileri:")
print(f"   - Training: {len(train_df)} images, {steps_per_epoch} steps")
print(f"   - Validation: {len(val_df)} images, {validation_steps} steps")
print(f"   - Test: {len(test_df)} images, {test_steps} steps")
print(f"   • Batch Size: {BATCH_SIZE}")

# Visual inspection
print("\n" + "=" * 70)
print("IMAGE INSPECTION")
print("=" * 70)

for images, labels in train_dataset.take(1):
    print(f"   • Image shape: {images.shape}")
    print(f"   • Image dtype: {images.dtype}")
    print(f"   • Image min: {tf.reduce_min(images).numpy():.2f}")
    print(f"   • Image max: {tf.reduce_max(images).numpy():.2f}")
    print(f"   • Label shape: {labels.shape}")

print("\ntf.data pipeline ready.")
print("\n[IMPORTANT] Each model must include its own preprocessing layer.")

# Cell 12: Class Weights Correction and Callback Definition


In [ ]:
# =============================================================================
# CELL 12: CLASS WEIGHTS AND CALLBACK FUNCTIONS
# =============================================================================

print("=" * 70)
print("CLASS WEIGHTS HESAPLAMA")
print("=" * 70)

# Class distribution in training set
class_counts = train_df['class_name'].value_counts()
total_samples = len(train_df)
n_classes = NUM_CLASSES

# Compute class weights
class_weights = {}

print("\nClass Weights:")
print("-" * 60)
print(f"{'Index':<8} {'Class':<20} {'Samples':<10} {'Weight':<10}")
print("-" * 60)

for class_name, idx in class_indices.items():
    count = class_counts[class_name]
    weight = total_samples / (n_classes * count)
    class_weights[idx] = weight
    print(f"{idx:<8} {class_name:<20} {count:<10} {weight:.4f}")

print("-" * 60)

max_weight_idx = max(class_weights, key=class_weights.get)
min_weight_idx = min(class_weights, key=class_weights.get)

print(f"\nAnalysis:")
print(f"   - Highest weight: {idx_to_class[max_weight_idx]} ({class_weights[max_weight_idx]:.4f})")
print(f"   - Lowest weight: {idx_to_class[min_weight_idx]} ({class_weights[min_weight_idx]:.4f})")
print(f"   - Imbalance ratio: {class_weights[max_weight_idx]/class_weights[min_weight_idx]:.2f}x")

# =============================================================================
# CALLBACK FUNCTION
# =============================================================================

print("\n" + "=" * 70)
print("CALLBACK FUNCTIONS")
print("=" * 70)

def create_callbacks(model_name):
    """Creates callbacks for model training."""
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            mode='max',
            verbose=1
        ),
        ModelCheckpoint(
            filepath=f'{MODEL_SAVE_PATH}/{model_name}_best.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    return callbacks

print("""
Callback'ler:
   • EarlyStopping: val_accuracy, patience=15, mode=max
   • ModelCheckpoint: val_accuracy, en iyi modeli kaydet
   • ReduceLROnPlateau: val_loss, factor=0.5, patience=5
""")

# =============================================================================
# RESULTS LIST
# =============================================================================

all_results = []

print("=" * 70)
print("Preparation complete - ready for model training.")
print("=" * 70)

# DATA LEAKAGE VERIFICATION

In [ ]:
# =============================================================================
# DATA LEAKAGE VERIFICATION
# =============================================================================
# Run this cell once to verify results.
# If "NO DATA LEAKAGE" is reported, proceed.
# =============================================================================

print("=" * 70)
print("DATA LEAKAGE VERIFICATION")
print("=" * 70)

# Convert file paths to sets
train_files = set(train_df['filepath'].values)
val_files = set(val_df['filepath'].values)
test_files = set(test_df['filepath'].values)

# Check intersections
train_val_overlap = train_files.intersection(val_files)
train_test_overlap = train_files.intersection(test_files)
val_test_overlap = val_files.intersection(test_files)

print(f"\nIntersection Check:")
print(f"   • Train ∩ Validation : {len(train_val_overlap)} dosya")
print(f"   • Train ∩ Test       : {len(train_test_overlap)} dosya")
print(f"   • Validation ∩ Test  : {len(val_test_overlap)} dosya")

# Result
if len(train_val_overlap) == 0 and len(train_test_overlap) == 0 and len(val_test_overlap) == 0:
    print("\n" + "=" * 70)
    print("DATA LEAKAGE YOK!")
    print("=" * 70)
    print("   All datasets are completely disjoint. Results are reliable.")
else:
    print("\n" + "=" * 70)
    print("[ERROR] DATA LEAKAGE DETECTED!")
    print("=" * 70)
    if len(train_val_overlap) > 0:
        print(f"   Train-Validation ortak dosyalar: {len(train_val_overlap)}")
    if len(train_test_overlap) > 0:
        print(f"   Train-Test ortak dosyalar: {len(train_test_overlap)}")
    if len(val_test_overlap) > 0:
        print(f"   Validation-Test ortak dosyalar: {len(val_test_overlap)}")

# Summary table
print(f"\nDataset Summary:")
print(f"   • Training   : {len(train_files):,} dosya")
print(f"   • Validation : {len(val_files):,} dosya")
print(f"   • Test       : {len(test_files):,} dosya")
print(f"   - Total     : {len(train_files) + len(val_files) + len(test_files):,} files")
print(f"   • Orijinal   : {len(df):,} dosya")

# Validation
if len(train_files) + len(val_files) + len(test_files) == len(df):
    print("\nTotal file count is consistent.")
else:
    print("\n[WARNING] Total file count mismatch!")

# MODELS

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import gc

# Cell 13: MODEL 1 - CUSTOM CNN

In [ ]:
# =============================================================================
# CELL 13: MODEL 1 - CUSTOM CNN (DESIGNED FROM SCRATCH)
# =============================================================================
# This model is a custom CNN architecture designed from scratch without transfer learning.
# It fulfills the project requirement for an originally developed model.
#
# Architectural Features:
# - 4 Convolutional Block (32 → 64 → 128 → 256 filtre)
# - Batch Normalization (training stability)
# - He Normal initialization (optimized for ReLU)
# - L2 Regularization (overfitting prevention)
# - Global Average Pooling (parameter reduction)
# - Dropout (generalization)
#
# A100 GPU Optimizasyonu:
# - Batch Size: 256
# - Fast data feeding via tf.data pipeline
# - Mixed precision support
# =============================================================================

import time
import numpy as np
from tensorflow.keras.regularizers import l2

MODEL_NAME = "Model_01_Custom_CNN"

print("=" * 70)
print("MODEL 1: CUSTOM CNN (SIFIRDAN TASARIM)")
print("=" * 70)
print(f"Model Name: {MODEL_NAME}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")

# =============================================================================
# MODEL ARCHITECTURE
# =============================================================================

def build_custom_cnn(input_shape, num_classes):
    """
    Custom CNN architecture for skin disease classification.

    Architecture:
    - 4 Convolutional Block
    - Her blokta: Conv → BN → ReLU → Conv → BN → ReLU → MaxPool → Dropout
    - Global Average Pooling
    - Classification via Dense layers

    Args:
        input_shape: Input dimensions (224, 224, 3)
        num_classes: Number of classes (5)

    Returns:
        tf.keras.Model
    """

    model = Sequential(name='Custom_CNN')

    # -------------------------------------------------------------------------
    # BLOCK 1: 32 Filters - Low-level feature extraction
    # -------------------------------------------------------------------------
    model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # -------------------------------------------------------------------------
    # BLOCK 2: 64 Filters - Mid-level features
    # -------------------------------------------------------------------------
    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # -------------------------------------------------------------------------
    # BLOCK 3: 128 Filters - High-level features
    # -------------------------------------------------------------------------
    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # -------------------------------------------------------------------------
    # BLOCK 4: 256 Filters - Deep features
    # -------------------------------------------------------------------------
    model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # -------------------------------------------------------------------------
    # CLASSIFICATION HEAD
    # -------------------------------------------------------------------------
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build model
print("\nBuilding model...")
model_1 = build_custom_cnn(IMG_SHAPE, NUM_CLASSES)

# Model summary
print("\n" + "=" * 70)
print("MODEL ARCHITECTURE")
print("=" * 70)
model_1.summary()

total_params = model_1.count_params()
trainable_params = np.sum([np.prod(v.shape) for v in model_1.trainable_weights])
print(f"\nParameter Information:")
print(f"   - Total: {total_params:,}")
print(f"   - Trainable: {trainable_params:,}")

# =============================================================================
# MODEL COMPILATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL DERLEME")
print("=" * 70)

LEARNING_RATE_M1 = 1e-3  # Slightly higher LR for Custom CNN

optimizer = Adam(learning_rate=LEARNING_RATE_M1)

model_1.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Optimizer: Adam")
print(f"Learning Rate: {LEARNING_RATE_M1}")
print(f"Loss: Categorical Crossentropy")
print(f"Metrics: Accuracy")

# =============================================================================
# CALLBACKS
# =============================================================================

callbacks_m1 = create_callbacks(MODEL_NAME)
print(f"\n[OK] Callbacks created: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau")

# =============================================================================
# MODEL TRAINING
# =============================================================================

print("\n" + "=" * 70)
print("MODEL TRAINING STARTING")
print("=" * 70)

print(f"""
Training Parameters:
   • Model: {MODEL_NAME}
   • Epochs: {EPOCHS}
   • Batch Size: {BATCH_SIZE}
   • Learning Rate: {LEARNING_RATE_M1}
   • Training Samples: {len(train_df)}
   • Validation Samples: {len(val_df)}
   • Steps per Epoch: {steps_per_epoch}
   • Class Weights: Aktif
""")

# Measure training time
start_time = time.time()

history_1 = model_1.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks_m1,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_time

print(f"\n{'=' * 70}")
print(f"Training complete.")
print(f"   - Duration: {training_time/60:.2f} minutes")
print(f"   • Tamamlanan Epoch: {len(history_1.history['loss'])}")
print(f"{'=' * 70}")

# =============================================================================
# TRAINING CURVES
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING CURVES")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history_1.history['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
axes[0].plot(history_1.history['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].legend(loc='lower right', fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss plot
axes[1].plot(history_1.history['loss'], 'b-', label='Training Loss', linewidth=2)
axes[1].plot(history_1.history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Loss', fontsize=11)
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
history_path = f"{RESULTS_PATH}/{MODEL_NAME}_history.png"
plt.savefig(history_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {history_path}")

# =============================================================================
# MODEL EVALUATION (TEST SET)
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

# Generate predictions on test set
print("\nGenerating predictions on test set...")
y_pred_proba = model_1.predict(test_dataset, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)

# Extract ground-truth labels
y_true = []
for _, labels in test_dataset:
    y_true.extend(np.argmax(labels.numpy(), axis=1))
y_true = np.array(y_true)

# =============================================================================
# PERFORMANCE METRICS
# =============================================================================

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
kappa = cohen_kappa_score(y_true, y_pred)

print(f"\nPERFORMANCE METRICS:")
print("-" * 50)
print(f"   Accuracy  : {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Precision : {precision:.4f}")
print(f"   Recall    : {recall:.4f}")
print(f"   F1-Score  : {f1:.4f}")
print(f"   Kappa     : {kappa:.4f}")
print("-" * 50)

# Classification Report
print(f"\nPER-CLASS PERFORMANCE:")
print("=" * 70)
print(classification_report(y_true, y_pred, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered,
            yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Class', fontsize=12)
plt.ylabel('True Class', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()

cm_path = f"{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png"
plt.savefig(cm_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {cm_path}")

# =============================================================================
# ROC CURVES (BUG-FIXED)
# =============================================================================

print("\n" + "=" * 70)
print("ROC CURVES")
print("=" * 70)

# y_true is already a numpy array (converted above)
n_classes = NUM_CLASSES

# One-hot encoding for ROC
y_true_bin = label_binarize(y_true, classes=range(n_classes))

# Compute ROC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Micro-average ROC
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Macro-average ROC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# ROC Plot
plt.figure(figsize=(10, 8))

# Renk paleti
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']

# ROC curve per class
for i, (class_name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'{class_name} (AUC = {roc_auc[i]:.3f})')

# Micro ve Macro average
plt.plot(fpr["micro"], tpr["micro"], color='deeppink', linestyle=':', lw=3,
         label=f'Micro-average (AUC = {roc_auc["micro"]:.3f})')
plt.plot(fpr["macro"], tpr["macro"], color='navy', linestyle=':', lw=3,
         label=f'Macro-average (AUC = {roc_auc["macro"]:.3f})')

# Reference diagonal
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12)
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()

roc_path = f"{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png"
plt.savefig(roc_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {roc_path}")

# Print ROC AUC values
print(f"\nROC AUC Values:")
print("-" * 40)
for i, class_name in enumerate(class_names_ordered):
    print(f"   {class_name}: {roc_auc[i]:.4f}")
print("-" * 40)
print(f"   Macro-average: {roc_auc['macro']:.4f}")
print(f"   Micro-average: {roc_auc['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_1 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'kappa': kappa,
    'roc_auc_macro': roc_auc['macro'],
    'roc_auc_micro': roc_auc['micro'],
    'training_time_min': training_time / 60,
    'total_epochs': len(history_1.history['loss']),
    'total_params': total_params
}

# Append AUC per class
for i, class_name in enumerate(class_names_ordered):
    results_1[f'auc_{class_name}'] = roc_auc[i]

all_results.append(results_1)

# =============================================================================
# SAVE MODEL
# =============================================================================

print("\n" + "=" * 70)
print("MODEL KAYDETME")
print("=" * 70)

# Final model
final_model_path = f"{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras"
model_1.save(final_model_path)
print(f"Model kaydedildi: {final_model_path}")

# Training history
history_df = pd.DataFrame(history_1.history)
history_csv_path = f"{RESULTS_PATH}/{MODEL_NAME}_training_history.csv"
history_df.to_csv(history_csv_path, index=False)
print(f"[OK] Training history saved: {history_csv_path}")

# =============================================================================
# MODEL 1 SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 1 - SUMMARY REPORT")
print("=" * 70)

print(f"""
MODEL INFORMATION
{'─' * 50}
   Model Name       : {MODEL_NAME}
   Model Type       : Custom CNN (Designed from Scratch)
   Toplam Parametre : {total_params:,}

ARCHITECTURE
{'─' * 50}
   • 4 Convolutional Block (32→64→128→256)
   • Batch Normalization + ReLU
   • MaxPooling2D + Dropout
   • Global Average Pooling
   • Dense(256) + Softmax(5)

TRAINING CONFIGURATION
{'─' * 50}
   • Optimizer       : Adam (lr={LEARNING_RATE_M1})
   • Loss            : Categorical Crossentropy
   • Batch Size      : {BATCH_SIZE}
   • Tamamlanan Epoch: {results_1['total_epochs']}
   • Training Duration: {results_1['training_time_min']:.2f} minutes

TEST RESULTS
{'─' * 50}
   • Accuracy        : {results_1['accuracy']*100:.2f}%
   • Precision       : {results_1['precision']:.4f}
   • Recall          : {results_1['recall']:.4f}
   • F1-Score        : {results_1['f1_score']:.4f}
   • Cohen's Kappa   : {results_1['kappa']:.4f}
   • ROC AUC (Macro) : {results_1['roc_auc_macro']:.4f}
   • ROC AUC (Micro) : {results_1['roc_auc_micro']:.4f}

SAVED FILES
{'─' * 50}
   • {final_model_path}
   • {history_csv_path}
   • {history_path}
   • {cm_path}
   • {roc_path}
""")

# =============================================================================
# MEMORY CLEANUP
# =============================================================================

# Keep Model 1 in memory (for comparison)
# Only clean unnecessary variables
gc.collect()

print("=" * 70)
print("MODEL 1 (CUSTOM CNN) TAMAMLANDI!")
print("=" * 70)

# Cell 14: MODEL 2 - EfficientNetV2-S (TRANSFER LEARNING)

In [ ]:
# =============================================================================
# CELL 14: MODEL 2 - EfficientNetV2-S (CORRECTED)
# =============================================================================
# CRITICAL FIX:
# - EfficientNetV2 includes its own preprocessing (include_preprocessing=True)
# - Images enter as [0, 255] and are transformed inside the model
# =============================================================================

import time
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, BatchNormalization, Activation)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

MODEL_NAME = "Model_02_EfficientNetV2S"

print("=" * 70)
print("MODEL 2: EfficientNetV2-S (CORRECTED)")
print("=" * 70)

# =============================================================================
# MODEL ARCHITECTURE
# =============================================================================

def build_efficientnet_v2(input_shape, num_classes):
    """
    EfficientNetV2-S modeli.

    IMPORTANT: With include_preprocessing=True, the model handles its own preprocessing.
    Input: [0, 255] -> Inside model: [-1, 1]
    """

    inputs = Input(shape=input_shape, name='input')

    # Base model - PREPROCESSING INCLUDED
    base_model = EfficientNetV2S(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs,
        include_preprocessing=True  # CRITICAL: Handles its own preprocessing!
    )

    # Freeze base model
    base_model.trainable = False

    # Classification head
    x = base_model.output
    x = GlobalAveragePooling2D(name='gap')(x)
    x = BatchNormalization(name='bn')(x)
    x = Dense(256, kernel_regularizer=l2(1e-4), name='dense')(x)
    x = Activation('relu', name='relu')(x)
    x = Dropout(0.5, name='dropout')(x)
    outputs = Dense(num_classes, activation='softmax', name='predictions')(x)

    model = Model(inputs=inputs, outputs=outputs, name='EfficientNetV2S')

    return model, base_model

# Build model
print("\nBuilding model...")
model_2, base_model_2 = build_efficientnet_v2(IMG_SHAPE, NUM_CLASSES)

# Parameter information
total_params = model_2.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model_2.trainable_weights])
non_trainable_params = total_params - trainable_params

print(f"\nParameter Information:")
print(f"   - Total: {total_params:,}")
print(f"   - Trainable: {trainable_params:,}")
print(f"   - Frozen: {non_trainable_params:,}")
print(f"\n[OK] include_preprocessing=True -> Model handles its own preprocessing")

# =============================================================================
# PHASE 1: FEATURE EXTRACTION
# =============================================================================

print("\n" + "=" * 70)
print("PHASE 1: FEATURE EXTRACTION (BASE FROZEN)")
print("=" * 70)

model_2.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_p1 = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='max', verbose=1),
    ModelCheckpoint(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_p1_best.keras', monitor='val_accuracy',
                    save_best_only=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
]

print("Phase 1 starting (20 epochs)...")
start_p1 = time.time()

history_p1 = model_2.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    callbacks=callbacks_p1,
    class_weight=class_weights,
    verbose=1
)

time_p1 = time.time() - start_p1
print(f"\nPhase 1: {time_p1/60:.2f} minutes")

# =============================================================================
# PHASE 2: FINE-TUNING
# =============================================================================

print("\n" + "=" * 70)
print("PHASE 2: FINE-TUNING")
print("=" * 70)

# Unfreeze last 50 layers
base_model_2.trainable = True
fine_tune_at = len(base_model_2.layers) - 50

for layer in base_model_2.layers[:fine_tune_at]:
    layer.trainable = False

trainable_count = sum([1 for l in model_2.layers if l.trainable])
print(f"Last {len(base_model_2.layers) - fine_tune_at} layers unfrozen")

# Very low LR
model_2.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_p2 = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='max', verbose=1),
    ModelCheckpoint(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras', monitor='val_accuracy',
                    save_best_only=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8, verbose=1)
]

print("Phase 2 starting (30 epochs, LR=1e-5)...")
start_p2 = time.time()

history_p2 = model_2.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks_p2,
    class_weight=class_weights,
    verbose=1
)

time_p2 = time.time() - start_p2
total_time = time_p1 + time_p2
print(f"\nPhase 2: {time_p2/60:.2f} minutes")
print(f"Total: {total_time/60:.2f} minutes")

# =============================================================================
# TRAINING CURVES
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING CURVES")
print("=" * 70)

# Merge histories
history_combined = {
    'accuracy': history_p1.history['accuracy'] + history_p2.history['accuracy'],
    'val_accuracy': history_p1.history['val_accuracy'] + history_p2.history['val_accuracy'],
    'loss': history_p1.history['loss'] + history_p2.history['loss'],
    'val_loss': history_p1.history['val_loss'] + history_p2.history['val_loss']
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
epochs_range = range(1, len(history_combined['accuracy']) + 1)
phase1_end = len(history_p1.history['accuracy'])

# Accuracy
axes[0].plot(epochs_range, history_combined['accuracy'], 'b-', label='Training', lw=2)
axes[0].plot(epochs_range, history_combined['val_accuracy'], 'r-', label='Validation', lw=2)
axes[0].axvline(x=phase1_end, color='g', linestyle='--', label=f'Fine-tune Start')
axes[0].set_title(f'{MODEL_NAME} - Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss
axes[1].plot(epochs_range, history_combined['loss'], 'b-', label='Training', lw=2)
axes[1].plot(epochs_range, history_combined['val_loss'], 'r-', label='Validation', lw=2)
axes[1].axvline(x=phase1_end, color='g', linestyle='--', label=f'Fine-tune Start')
axes[1].set_title(f'{MODEL_NAME} - Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

# Predictions
print("Generating predictions...")
y_pred_proba = model_2.predict(test_dataset, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)

# Ground-truth labels
y_true = []
for _, labels in test_dataset:
    y_true.extend(np.argmax(labels.numpy(), axis=1))
y_true = np.array(y_true)

# Metrikler
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
kappa = cohen_kappa_score(y_true, y_pred)

print(f"\nPERFORMANS:")
print(f"   Accuracy : {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Precision: {precision:.4f}")
print(f"   Recall   : {recall:.4f}")
print(f"   F1-Score : {f1:.4f}")
print(f"   Kappa    : {kappa:.4f}")

# Classification Report
print(f"\nPER-CLASS RESULTS:")
print(classification_report(y_true, y_pred, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME} - Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Tahmin')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

print("\nROC Curves...")

n_classes = NUM_CLASSES
y_true_bin = label_binarize(y_true, classes=range(n_classes))

fpr, tpr, roc_auc = {}, {}, {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Micro/Macro
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= n_classes
fpr["macro"], tpr["macro"] = all_fpr, mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot
plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']

for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'{name} (AUC={roc_auc[i]:.3f})')

plt.plot(fpr["micro"], tpr["micro"], 'deeppink', ls=':', lw=3, label=f'Micro (AUC={roc_auc["micro"]:.3f})')
plt.plot(fpr["macro"], tpr["macro"], 'navy', ls=':', lw=3, label=f'Macro (AUC={roc_auc["macro"]:.3f})')
plt.plot([0,1], [0,1], 'k--', lw=2)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME} - ROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nROC AUC: Macro={roc_auc['macro']:.4f}, Micro={roc_auc['micro']:.4f}")

# =============================================================================
# KAYDET
# =============================================================================

results_2 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'kappa': kappa,
    'roc_auc_macro': roc_auc['macro'],
    'roc_auc_micro': roc_auc['micro'],
    'training_time_min': total_time / 60,
    'total_epochs': len(history_combined['accuracy']),
    'total_params': total_params
}

for i, name in enumerate(class_names_ordered):
    results_2[f'auc_{name}'] = roc_auc[i]

# Update all_results list (replace old Model 2 if exists)
if len(all_results) > 1:
    all_results[1] = results_2
else:
    all_results.append(results_2)

model_2.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_combined).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

print(f"\nModel kaydedildi!")

# =============================================================================
# SUMMARY
# =============================================================================

print("\n" + "=" * 70)
print(f"MODEL 2 SUMMARY")
print("=" * 70)
print(f"""
   Model: {MODEL_NAME}
   Accuracy: {accuracy*100:.2f}%
   F1-Score: {f1:.4f}
   ROC AUC: {roc_auc['macro']:.4f}
   Duration: {total_time/60:.2f} minutes
""")

gc.collect()
print("=" * 70)
print("MODEL 2 TAMAMLANDI!")
print("=" * 70)

# Cell 15: MODEL 3 - ConvNeXt Tiny

In [ ]:
# =============================================================================
# CELL 15: MODEL 3 - ConvNeXt Tiny (Transformer-Style CNN)
# =============================================================================
# ConvNeXt applies Swin Transformer design principles to CNN:
# - Patchify stem (4x4 non-overlapping conv)
# - Inverted bottleneck
# - Larger kernels (7x7 depthwise conv)
# - LayerNorm instead of BatchNorm
# - GELU activation
# - Fewer activation functions
#
# This model bridges the gap between CNN and Transformer architectures.
# It fulfills the project requirement for an alternative architectural approach.
# =============================================================================

import time
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, BatchNormalization, Activation)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

MODEL_NAME = "Model_03_ConvNeXtTiny"

print("=" * 70)
print("MODEL 3: ConvNeXt Tiny (Transformer-Style CNN)")
print("=" * 70)

print("""
MODEL HAKKINDA:
   ConvNeXt was developed by Meta AI in 2022.
   It modernized CNN architecture inspired by Swin Transformer's success.

   Özellikler:
   - 7x7 depthwise convolution (large receptive field)
   • LayerNorm (BatchNorm yerine)
   • GELU activation (ReLU yerine)
   • Inverted bottleneck (MobileNet'ten)
   • Patchify stem (ViT'ten)
""")

# =============================================================================
# MODEL ARCHITECTURE
# =============================================================================

def build_convnext_model(input_shape, num_classes):
    """
    ConvNeXt Tiny modeli.
    With include_preprocessing=True, the model accepts [0,255] input.
    """
    inputs = Input(shape=input_shape, name='input_layer')

    # Base model - ImageNet pretrained
    base_model = ConvNeXtTiny(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs,
        include_preprocessing=True  # [0,255] → normalize
    )

    # Freeze initially
    base_model.trainable = False

    # Classification head
    x = base_model.output
    x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = BatchNormalization(name='batch_norm')(x)
    x = Dense(256, kernel_regularizer=l2(1e-4), name='dense_256')(x)
    x = Activation('relu', name='relu')(x)
    x = Dropout(0.5, name='dropout')(x)
    outputs = Dense(num_classes, activation='softmax', name='predictions')(x)

    model = Model(inputs=inputs, outputs=outputs, name='ConvNeXtTiny_Transfer')

    return model, base_model

# Build model
print("\nBuilding model...")
model_3, base_model_3 = build_convnext_model(IMG_SHAPE, NUM_CLASSES)

# Parameter information
total_params_3 = model_3.count_params()
trainable_params_3 = sum([tf.keras.backend.count_params(w) for w in model_3.trainable_weights])
non_trainable_3 = total_params_3 - trainable_params_3

print(f"\nParameter Information:")
print(f"   - Total parameters  : {total_params_3:,}")
print(f"   - Trainable          : {trainable_params_3:,}")
print(f"   - Frozen             : {non_trainable_3:,}")
print(f"\n[OK] include_preprocessing=True (Model handles its own normalization)")

# =============================================================================
# PHASE 1: FEATURE EXTRACTION (BASE FROZEN)
# =============================================================================

print("\n" + "=" * 70)
print("PHASE 1: FEATURE EXTRACTION (BASE MODEL FROZEN)")
print("=" * 70)

model_3.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Optimizer: Adam (lr=1e-3)")
print(f"Loss: Categorical Crossentropy")
print(f"Base Model: FROZEN")

callbacks_p1_m3 = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_p1_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

EPOCHS_P1 = 20

print(f"\nPhase 1 Training Starting...")
print(f"   • Epochs: {EPOCHS_P1}")
print(f"   • Batch Size: {BATCH_SIZE}")

start_p1_m3 = time.time()

history_p1_m3 = model_3.fit(
    train_dataset,
    epochs=EPOCHS_P1,
    validation_data=val_dataset,
    callbacks=callbacks_p1_m3,
    class_weight=class_weights,
    verbose=1
)

time_p1_m3 = time.time() - start_p1_m3
epochs_completed_p1 = len(history_p1_m3.history['accuracy'])

print(f"\nPhase 1 Complete: {time_p1_m3/60:.2f} minutes ({epochs_completed_p1} epochs)")

# =============================================================================
# PHASE 2: FINE-TUNING
# =============================================================================

print("\n" + "=" * 70)
print("PHASE 2: FINE-TUNING (FINAL LAYERS UNFROZEN)")
print("=" * 70)

# Unfreeze final layers of base model
base_model_3.trainable = True

# Train only last 30 layers
fine_tune_at = len(base_model_3.layers) - 30
for layer in base_model_3.layers[:fine_tune_at]:
    layer.trainable = False

trainable_layers = sum([1 for l in base_model_3.layers if l.trainable])
print(f"Fine-tuning configuration:")
print(f"   - Total layers: {len(base_model_3.layers)}")
print(f"   - Unfrozen layers: {len(base_model_3.layers) - fine_tune_at}")
print(f"   - Frozen: {fine_tune_at}")

# Recompile with low learning rate
model_3.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n[OK] Optimizer: Adam (lr=1e-5) - Very low")
print(f"[OK] Pre-trained weights will be preserved")

callbacks_p2_m3 = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-8,
        verbose=1
    )
]

EPOCHS_P2 = 30

print(f"\nPhase 2 Training Starting...")
print(f"   • Epochs: {EPOCHS_P2}")
print(f"   • Learning Rate: 1e-5")

start_p2_m3 = time.time()

history_p2_m3 = model_3.fit(
    train_dataset,
    epochs=EPOCHS_P2,
    validation_data=val_dataset,
    callbacks=callbacks_p2_m3,
    class_weight=class_weights,
    verbose=1
)

time_p2_m3 = time.time() - start_p2_m3
epochs_completed_p2 = len(history_p2_m3.history['accuracy'])

# Total time and epochs
total_time_m3 = time_p1_m3 + time_p2_m3
total_epochs_m3 = epochs_completed_p1 + epochs_completed_p2
avg_epoch_time_m3 = total_time_m3 / total_epochs_m3

print(f"\nPhase 2 Complete: {time_p2_m3/60:.2f} minutes ({epochs_completed_p2} epochs)")
print(f"Total Duration: {total_time_m3/60:.2f} minutes ({total_epochs_m3} epochs)")
print(f"Average Epoch Duration: {avg_epoch_time_m3:.2f} seconds")

# =============================================================================
# TRAINING CURVES
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING CURVES")
print("=" * 70)

# Merge histories
history_combined_m3 = {
    'accuracy': history_p1_m3.history['accuracy'] + history_p2_m3.history['accuracy'],
    'val_accuracy': history_p1_m3.history['val_accuracy'] + history_p2_m3.history['val_accuracy'],
    'loss': history_p1_m3.history['loss'] + history_p2_m3.history['loss'],
    'val_loss': history_p1_m3.history['val_loss'] + history_p2_m3.history['val_loss']
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_combined_m3['accuracy']) + 1)

# Accuracy
axes[0].plot(epochs_range, history_combined_m3['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
axes[0].plot(epochs_range, history_combined_m3['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
axes[0].axvline(x=epochs_completed_p1, color='green', linestyle='--', label=f'Fine-tuning Start (Epoch {epochs_completed_p1})')
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss
axes[1].plot(epochs_range, history_combined_m3['loss'], 'b-', label='Training Loss', linewidth=2)
axes[1].plot(epochs_range, history_combined_m3['val_loss'], 'r-', label='Validation Loss', linewidth=2)
axes[1].axvline(x=epochs_completed_p1, color='green', linestyle='--', label=f'Fine-tuning Start (Epoch {epochs_completed_p1})')
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
history_path_m3 = f'{RESULTS_PATH}/{MODEL_NAME}_history.png'
plt.savefig(history_path_m3, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {history_path_m3}")

# =============================================================================
# MODEL EVALUATION (TEST SET)
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

print("\nGenerating predictions on test set...")
y_pred_proba_3 = model_3.predict(test_dataset, verbose=1)
y_pred_3 = np.argmax(y_pred_proba_3, axis=1)

# Ground-truth labels
y_true_3 = []
for _, labels in test_dataset:
    y_true_3.extend(np.argmax(labels.numpy(), axis=1))
y_true_3 = np.array(y_true_3)

# Metrikler
accuracy_3 = accuracy_score(y_true_3, y_pred_3)
precision_3 = precision_score(y_true_3, y_pred_3, average='weighted', zero_division=0)
recall_3 = recall_score(y_true_3, y_pred_3, average='weighted', zero_division=0)
f1_3 = f1_score(y_true_3, y_pred_3, average='weighted', zero_division=0)
kappa_3 = cohen_kappa_score(y_true_3, y_pred_3)

print(f"\nPERFORMANCE METRICS:")
print("-" * 50)
print(f"   Accuracy  : {accuracy_3:.4f} ({accuracy_3*100:.2f}%)")
print(f"   Precision : {precision_3:.4f}")
print(f"   Recall    : {recall_3:.4f}")
print(f"   F1-Score  : {f1_3:.4f}")
print(f"   Kappa     : {kappa_3:.4f}")
print("-" * 50)

# Classification Report
print(f"\nPER-CLASS PERFORMANCE:")
print("=" * 70)
print(classification_report(y_true_3, y_pred_3, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm_3 = confusion_matrix(y_true_3, y_pred_3)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_3, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered,
            yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Class', fontsize=12)
plt.ylabel('True Class', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

cm_path_3 = f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png'
plt.savefig(cm_path_3, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {cm_path_3}")

# =============================================================================
# ROC CURVES
# =============================================================================

print("\n" + "=" * 70)
print("ROC CURVES")
print("=" * 70)

n_classes = NUM_CLASSES
y_true_bin_3 = label_binarize(y_true_3, classes=range(n_classes))

# ROC per class
fpr_3, tpr_3, roc_auc_3 = {}, {}, {}

for i in range(n_classes):
    fpr_3[i], tpr_3[i], _ = roc_curve(y_true_bin_3[:, i], y_pred_proba_3[:, i])
    roc_auc_3[i] = auc(fpr_3[i], tpr_3[i])

# Micro average
fpr_3["micro"], tpr_3["micro"], _ = roc_curve(y_true_bin_3.ravel(), y_pred_proba_3.ravel())
roc_auc_3["micro"] = auc(fpr_3["micro"], tpr_3["micro"])

# Macro average
all_fpr_3 = np.unique(np.concatenate([fpr_3[i] for i in range(n_classes)]))
mean_tpr_3 = np.zeros_like(all_fpr_3)
for i in range(n_classes):
    mean_tpr_3 += np.interp(all_fpr_3, fpr_3[i], tpr_3[i])
mean_tpr_3 /= n_classes
fpr_3["macro"] = all_fpr_3
tpr_3["macro"] = mean_tpr_3
roc_auc_3["macro"] = auc(fpr_3["macro"], tpr_3["macro"])

# Plot
plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']

for i, (class_name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_3[i], tpr_3[i], color=color, lw=2,
             label=f'{class_name} (AUC = {roc_auc_3[i]:.4f})')

plt.plot(fpr_3["micro"], tpr_3["micro"], color='deeppink', linestyle=':', lw=3,
         label=f'Micro-average (AUC = {roc_auc_3["micro"]:.4f})')
plt.plot(fpr_3["macro"], tpr_3["macro"], color='navy', linestyle=':', lw=3,
         label=f'Macro-average (AUC = {roc_auc_3["macro"]:.4f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()

roc_path_3 = f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png'
plt.savefig(roc_path_3, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {roc_path_3}")

# ROC AUC values
print(f"\nROC AUC Values:")
print("-" * 45)
for i, class_name in enumerate(class_names_ordered):
    print(f"   {class_name:<20}: {roc_auc_3[i]:.4f}")
print("-" * 45)
print(f"   {'Macro-average':<20}: {roc_auc_3['macro']:.4f}")
print(f"   {'Micro-average':<20}: {roc_auc_3['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_3 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_3,
    'precision': precision_3,
    'recall': recall_3,
    'f1_score': f1_3,
    'kappa': kappa_3,
    'roc_auc_macro': roc_auc_3['macro'],
    'roc_auc_micro': roc_auc_3['micro'],
    'training_time_min': total_time_m3 / 60,
    'avg_epoch_time_sec': avg_epoch_time_m3,
    'total_epochs': total_epochs_m3,
    'total_params': total_params_3
}

for i, class_name in enumerate(class_names_ordered):
    results_3[f'auc_{class_name}'] = roc_auc_3[i]

all_results.append(results_3)

# =============================================================================
# SAVE MODEL
# =============================================================================

print("\n" + "=" * 70)
print("MODEL KAYDETME")
print("=" * 70)

model_save_path_3 = f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras'
model_3.save(model_save_path_3)
print(f"Model kaydedildi: {model_save_path_3}")

history_csv_path_3 = f'{RESULTS_PATH}/{MODEL_NAME}_history.csv'
pd.DataFrame(history_combined_m3).to_csv(history_csv_path_3, index=False)
print(f"[OK] Training history saved: {history_csv_path_3}")

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 70)
print("MODEL COMPARISON TABLE")
print("=" * 70)

print(f"\n{'Method':<40} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 95)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', (r['training_time_min'] * 60) / r['total_epochs'])
    print(f"{r['model_name']:<40} {r['precision']:>10.4f} {r['recall']:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 95)

# Best model
best_model = max(all_results, key=lambda x: x['accuracy'])
print(f"\nHighest accuracy: {best_model['model_name']} ({best_model['accuracy']*100:.2f}%)")

# =============================================================================
# MODEL 3 DETAILED SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 3 - DETAILED SUMMARY REPORT")
print("=" * 70)

print(f"""
MODEL INFORMATION
{'─' * 60}
   Model Name        : {MODEL_NAME}
   Model Tipi        : Transfer Learning (Transformer-Style CNN)
   Base Model        : ConvNeXt Tiny (ImageNet pretrained)
   Toplam Parametre  : {total_params_3:,}
   Preprocessing     : include_preprocessing=True (Inside model)

ARCHITECTURE DETAILS
{'─' * 60}
   ConvNeXt Tiny Özellikleri:
   • Patchify stem: 4x4 non-overlapping convolution
   • Inverted bottleneck blocks
   - 7x7 depthwise convolution (large receptive field)
   • LayerNorm (BatchNorm yerine)
   • GELU activation (ReLU yerine)

   Classification Head:
   • GlobalAveragePooling2D
   • BatchNormalization
   • Dense(256) + ReLU + Dropout(0.5)
   • Dense(5) + Softmax

TRAINING STRATEGY
{'─' * 60}
   Phase 1 - Feature Extraction:
   • Base model: FROZEN
   • Learning rate: 1e-3
   • Epochs: {epochs_completed_p1}
   • Duration: {time_p1_m3/60:.2f} minutes

   Phase 2 - Fine-tuning:
   • Unfrozen layers: Last {len(base_model_3.layers) - fine_tune_at}
   • Learning rate: 1e-5
   • Epochs: {epochs_completed_p2}
   • Duration: {time_p2_m3/60:.2f} minutes

PERFORMANCE METRICS
{'─' * 60}
   • Accuracy        : {accuracy_3*100:.2f}%
   • Precision       : {precision_3:.4f}
   • Recall          : {recall_3:.4f}
   • F1-Score        : {f1_3:.4f}
   • Cohen's Kappa   : {kappa_3:.4f}

ROC AUC VALUES (Per Class)
{'─' * 60}""")

for i, class_name in enumerate(class_names_ordered):
    print(f"   • {class_name:<20}: {roc_auc_3[i]:.4f}")

print(f"""   {'─' * 40}
   • Macro-average       : {roc_auc_3['macro']:.4f}
   • Micro-average       : {roc_auc_3['micro']:.4f}

TRAINING DURATION
{'─' * 60}
   • Phase 1            : {time_p1_m3/60:.2f} minutes
   • Phase 2            : {time_p2_m3/60:.2f} minutes
   • Toplam             : {total_time_m3/60:.2f} minutes
   • Toplam Epoch       : {total_epochs_m3}
   • Ortalama Epoch     : {avg_epoch_time_m3:.2f} saniye

SAVED FILES
{'─' * 60}
   • {model_save_path_3}
   • {history_csv_path_3}
   • {history_path_m3}
   • {cm_path_3}
   • {roc_path_3}
""")

# Bellek temizle
gc.collect()

print("=" * 70)
print("MODEL 3 (ConvNeXt Tiny) TAMAMLANDI!")
print("=" * 70)

# Cell 16: MODEL 4 - DenseNet121 + InceptionV3 (HYBRID)

In [ ]:
!pip install keras-tuner -q

In [ ]:
# =============================================================================
# CELL 16: MODEL 4 - DenseNet121 + InceptionV3 (HYBRID MODEL)
# =============================================================================
# This model is designed following the reference architecture schematic:
# - Two backbones: DenseNet121 and InceptionV3
# - Separate classification head per backbone
# - Feature concatenation
# - Final classification head
#
# KEY MODIFICATIONS:
# 1. No fine-tuning - Base models fully FROZEN
# 2. Hyperparameter optimization via Keras Tuner
# 3. Architecture (Dense 256 -> 128 -> output)
# =============================================================================

import time
import keras_tuner as kt
from tensorflow.keras.applications import DenseNet121, InceptionV3
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, BatchNormalization, Concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

MODEL_NAME = "Model_04_DenseNet121_InceptionV3_Hybrid"

print("=" * 70)
print("MODEL 4: DenseNet121 + InceptionV3 (HYBRID)")
print("=" * 70)

print("""
MODEL ARCHITECTURE:

    Input Image (224x224x3)
           |
    ┌──────┴──────┐
    ▼             ▼
DenseNet121   InceptionV3
(FROZEN)      (FROZEN)
    |             |
   GAP           GAP
    |             |
Dense(256)    Dense(256)
Dropout       Dropout
Dense(128)    Dense(128)
Dropout       Dropout
    |             |
    └──────┬──────┘
           |
      Concatenate
           |
      Dense(256)
      Dropout
      Dense(128)
      Dropout
           |
      Dense(5) + Softmax
           |
        Output
""")

# =============================================================================
# ENHANCED DATA AUGMENTATION
# =============================================================================

print("\n" + "=" * 70)
print("ENHANCED DATA AUGMENTATION")
print("=" * 70)

# Update current augmentation
enhanced_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),  # +/-15% (slightly increased)
    tf.keras.layers.RandomZoom(0.15),       # ±15%
    tf.keras.layers.RandomContrast(0.1),    # Contrast variation
    tf.keras.layers.RandomBrightness(0.1),  # Brightness variation
], name="enhanced_augmentation")

print("[OK] Augmentation Layers:")
print("   • RandomFlip: horizontal")
print("   • RandomRotation: ±15%")
print("   • RandomZoom: ±15%")
print("   • RandomContrast: ±10%")
print("   • RandomBrightness: ±10%")

# Create augmented dataset
def create_augmented_dataset(filepaths, labels, is_training=False):
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    if is_training:
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER, seed=SEED)

    dataset = dataset.map(load_image_no_preprocess, num_parallel_calls=AUTOTUNE)

    if is_training:
        dataset = dataset.map(
            lambda x, y: (enhanced_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE
        )

    dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)

    return dataset

# Yeni datasetler
train_dataset_aug = create_augmented_dataset(train_filepaths, train_labels, is_training=True)
val_dataset_aug = create_augmented_dataset(val_filepaths, val_labels, is_training=False)
test_dataset_aug = create_augmented_dataset(test_filepaths, test_labels, is_training=False)

print("\nDatasets prepared with enhanced augmentation.")

# =============================================================================
# HYBRID MODEL ARCHITECTURE (FOR HYPERPARAMETER TUNING)
# =============================================================================

def build_hybrid_model(hp):
    """
    Hyperparameter-tunable model builder for Keras Tuner.
    Dual-backbone hybrid architecture per reference schematic.
    """

    # Hyperparameters
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    dropout_1 = hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])
    dropout_2 = hp.Choice('dropout_2', values=[0.4, 0.5, 0.6])
    dense_units_1 = hp.Choice('dense_units_1', values=[256, 512])
    dense_units_2 = hp.Choice('dense_units_2', values=[128, 256])
    optimizer_name = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    # Input
    inputs = Input(shape=IMG_SHAPE, name='input')

    # =========================================================================
    # BACKBONE 1: DenseNet121
    # =========================================================================
    densenet = DenseNet121(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs
    )
    densenet.trainable = False  # FROZEN - Fine-tuning YOK!

    # DenseNet preprocessing (scale to [0,1] then normalize)
    x1 = tf.keras.applications.densenet.preprocess_input(inputs)
    x1 = densenet(x1)
    x1 = GlobalAveragePooling2D(name='gap_densenet')(x1)

    # DenseNet classification head (per reference schematic)
    x1 = Dense(dense_units_1, activation='relu', name='dense1_dn')(x1)
    x1 = Dropout(dropout_1, name='drop1_dn')(x1)
    x1 = Dense(dense_units_2, activation='relu', name='dense2_dn')(x1)
    x1 = Dropout(dropout_2, name='drop2_dn')(x1)

    # =========================================================================
    # BACKBONE 2: InceptionV3
    # =========================================================================
    inception = InceptionV3(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs
    )
    inception.trainable = False  # FROZEN - Fine-tuning YOK!

    # InceptionV3 preprocessing (scale to [-1,1])
    x2 = tf.keras.applications.inception_v3.preprocess_input(inputs)
    x2 = inception(x2)
    x2 = GlobalAveragePooling2D(name='gap_inception')(x2)

    # Inception classification head (per reference schematic)
    x2 = Dense(dense_units_1, activation='relu', name='dense1_inc')(x2)
    x2 = Dropout(dropout_1, name='drop1_inc')(x2)
    x2 = Dense(dense_units_2, activation='relu', name='dense2_inc')(x2)
    x2 = Dropout(dropout_2, name='drop2_inc')(x2)

    # =========================================================================
    # CONCATENATE & FINAL CLASSIFICATION
    # =========================================================================
    merged = Concatenate(name='concat')([x1, x2])

    # Final classification head
    x = Dense(dense_units_1, activation='relu', name='dense1_final')(merged)
    x = Dropout(dropout_1, name='drop1_final')(x)
    x = Dense(dense_units_2, activation='relu', name='dense2_final')(x)
    x = Dropout(dropout_2, name='drop2_final')(x)

    # Output
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

    # Model
    model = Model(inputs=inputs, outputs=outputs, name='DenseNet121_InceptionV3_Hybrid')

    # Optimizer selection
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =============================================================================
# HYPERPARAMETER OPTIMIZATION VIA KERAS TUNER
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER OPTIMIZATION (KERAS TUNER)")
print("=" * 70)

# Tuner dizini
tuner_dir = f'{MODEL_SAVE_PATH}/tuner_{MODEL_NAME}'

# Bayesian Optimization tuner
tuner = kt.BayesianOptimization(
    build_hybrid_model,
    objective='val_accuracy',
    max_trials=15,  # Try 15 different combinations
    num_initial_points=5,
    directory=tuner_dir,
    project_name='hybrid_tuning',
    overwrite=True
)

print(f"""
Keras Tuner Settings:
   • Algoritma: Bayesian Optimization
   • Max Trials: 15
   • Objective: val_accuracy (maximize)

Aranacak Hyperparameter'lar:
   • Learning Rate: [0.01, 0.001, 0.0001]
   • Dropout 1: [0.2, 0.3, 0.4]
   • Dropout 2: [0.4, 0.5, 0.6]
   • Dense Units 1: [256, 512]
   • Dense Units 2: [128, 256]
   • Optimizer: [Adam, SGD, RMSprop]
""")

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

# Tuner search
print("Hyperparameter search starting...")
print("   (This may take several minutes)\n")

start_tuning = time.time()

tuner.search(
    train_dataset_aug,
    epochs=20,
    validation_data=val_dataset_aug,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)

tuning_time = time.time() - start_tuning
print(f"\nTuning duration: {tuning_time/60:.2f} minutes")

# =============================================================================
# BEST HYPERPARAMETERS
# =============================================================================

print("\n" + "=" * 70)
print("BEST HYPERPARAMETERS")
print("=" * 70)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Optimal Hyperparameter'lar:
   • Learning Rate: {best_hps.get('learning_rate')}
   • Dropout 1: {best_hps.get('dropout_1')}
   • Dropout 2: {best_hps.get('dropout_2')}
   • Dense Units 1: {best_hps.get('dense_units_1')}
   • Dense Units 2: {best_hps.get('dense_units_2')}
   • Optimizer: {best_hps.get('optimizer')}
""")

# =============================================================================
# FINAL TRAINING WITH BEST MODEL
# =============================================================================

print("\n" + "=" * 70)
print("FINAL MODEL TRAINING")
print("=" * 70)

# Best model
model_4 = tuner.hypermodel.build(best_hps)

# Model summary
total_params_4 = model_4.count_params()
trainable_params_4 = sum([tf.keras.backend.count_params(w) for w in model_4.trainable_weights])

print(f"\nModel Bilgisi:")
print(f"   - Total parameters: {total_params_4:,}")
print(f"   - Trainable: {trainable_params_4:,}")
print(f"   • Frozen (backbone): {total_params_4 - trainable_params_4:,}")

# Callbacks
callbacks_final = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Final training
print("\nFinal training starting (50 epochs)...")
start_training = time.time()

history_4 = model_4.fit(
    train_dataset_aug,
    epochs=50,
    validation_data=val_dataset_aug,
    callbacks=callbacks_final,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_training
total_time_4 = tuning_time + training_time
total_epochs_4 = len(history_4.history['accuracy'])
avg_epoch_time_4 = training_time / total_epochs_4

print(f"\nFinal training: {training_time/60:.2f} minutes ({total_epochs_4} epochs)")
print(f"Total duration (tuning + training): {total_time_4/60:.2f} minutes")

# =============================================================================
# TRAINING CURVES
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING CURVES")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_4.history['accuracy']) + 1)

# Accuracy
axes[0].plot(epochs_range, history_4.history['accuracy'], 'b-', label='Training', lw=2)
axes[0].plot(epochs_range, history_4.history['val_accuracy'], 'r-', label='Validation', lw=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss
axes[1].plot(epochs_range, history_4.history['loss'], 'b-', label='Training', lw=2)
axes[1].plot(epochs_range, history_4.history['val_loss'], 'r-', label='Validation', lw=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

y_pred_proba_4 = model_4.predict(test_dataset_aug, verbose=1)
y_pred_4 = np.argmax(y_pred_proba_4, axis=1)

y_true_4 = []
for _, labels in test_dataset_aug:
    y_true_4.extend(np.argmax(labels.numpy(), axis=1))
y_true_4 = np.array(y_true_4)

# Metrikler
accuracy_4 = accuracy_score(y_true_4, y_pred_4)
precision_4 = precision_score(y_true_4, y_pred_4, average='weighted', zero_division=0)
recall_4 = recall_score(y_true_4, y_pred_4, average='weighted', zero_division=0)
f1_4 = f1_score(y_true_4, y_pred_4, average='weighted', zero_division=0)
kappa_4 = cohen_kappa_score(y_true_4, y_pred_4)

print(f"\nPERFORMANCE METRICS:")
print("-" * 50)
print(f"   Accuracy  : {accuracy_4:.4f} ({accuracy_4*100:.2f}%)")
print(f"   Precision : {precision_4:.4f}")
print(f"   Recall    : {recall_4:.4f}")
print(f"   F1-Score  : {f1_4:.4f}")
print(f"   Kappa     : {kappa_4:.4f}")

print(f"\nClassification Report:")
print(classification_report(y_true_4, y_pred_4, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

cm_4 = confusion_matrix(y_true_4, y_pred_4)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_4, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Tahmin')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

y_true_bin_4 = label_binarize(y_true_4, classes=range(NUM_CLASSES))
fpr_4, tpr_4, roc_auc_4 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_4[i], tpr_4[i], _ = roc_curve(y_true_bin_4[:, i], y_pred_proba_4[:, i])
    roc_auc_4[i] = auc(fpr_4[i], tpr_4[i])

fpr_4["micro"], tpr_4["micro"], _ = roc_curve(y_true_bin_4.ravel(), y_pred_proba_4.ravel())
roc_auc_4["micro"] = auc(fpr_4["micro"], tpr_4["micro"])

all_fpr_4 = np.unique(np.concatenate([fpr_4[i] for i in range(NUM_CLASSES)]))
mean_tpr_4 = np.zeros_like(all_fpr_4)
for i in range(NUM_CLASSES):
    mean_tpr_4 += np.interp(all_fpr_4, fpr_4[i], tpr_4[i])
mean_tpr_4 /= NUM_CLASSES
fpr_4["macro"], tpr_4["macro"] = all_fpr_4, mean_tpr_4
roc_auc_4["macro"] = auc(fpr_4["macro"], tpr_4["macro"])

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_4[i], tpr_4[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_4[i]:.4f})')
plt.plot(fpr_4["micro"], tpr_4["micro"], 'deeppink', ls=':', lw=3, label=f'Micro (AUC={roc_auc_4["micro"]:.4f})')
plt.plot(fpr_4["macro"], tpr_4["macro"], 'navy', ls=':', lw=3, label=f'Macro (AUC={roc_auc_4["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nROC AUC Values:")
for i, name in enumerate(class_names_ordered):
    print(f"   {name}: {roc_auc_4[i]:.4f}")
print(f"   Macro: {roc_auc_4['macro']:.4f}, Micro: {roc_auc_4['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_4 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_4,
    'precision': precision_4,
    'recall': recall_4,
    'f1_score': f1_4,
    'kappa': kappa_4,
    'roc_auc_macro': roc_auc_4['macro'],
    'roc_auc_micro': roc_auc_4['micro'],
    'training_time_min': total_time_4 / 60,
    'avg_epoch_time_sec': avg_epoch_time_4,
    'total_epochs': total_epochs_4,
    'total_params': total_params_4,
    'best_lr': best_hps.get('learning_rate'),
    'best_dropout1': best_hps.get('dropout_1'),
    'best_dropout2': best_hps.get('dropout_2'),
    'best_optimizer': best_hps.get('optimizer')
}

for i, name in enumerate(class_names_ordered):
    results_4[f'auc_{name}'] = roc_auc_4[i]

all_results.append(results_4)

# Save model
model_4.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_4.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 70)
print("MODEL COMPARISON TABLE")
print("=" * 70)

print(f"\n{'Method':<45} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 100)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', (r['training_time_min'] * 60) / max(r['total_epochs'], 1))
    print(f"{r['model_name']:<45} {r['precision']:>10.4f} {r['recall']:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 100)
best = max(all_results, key=lambda x: x['accuracy'])
print(f"\nEn iyi: {best['model_name']} ({best['accuracy']*100:.2f}%)")

# =============================================================================
# DETAILED SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 4 - DETAILED SUMMARY REPORT")
print("=" * 70)

print(f"""
MODEL INFORMATION
{'─' * 60}
   Model Name        : {MODEL_NAME}
   Model Tipi        : Hybrid (Dual-Backbone)
   Backbone 1        : DenseNet121 (FROZEN)
   Backbone 2        : InceptionV3 (FROZEN)
   Toplam Parametre  : {total_params_4:,}
   Trainable          : {trainable_params_4:,}

ARCHITECTURE (Per Reference Schematic)
{'─' * 60}
   • DenseNet121 → GAP → Dense({best_hps.get('dense_units_1')}) → Dropout({best_hps.get('dropout_1')}) → Dense({best_hps.get('dense_units_2')}) → Dropout({best_hps.get('dropout_2')})
   • InceptionV3 → GAP → Dense({best_hps.get('dense_units_1')}) → Dropout({best_hps.get('dropout_1')}) → Dense({best_hps.get('dense_units_2')}) → Dropout({best_hps.get('dropout_2')})
   • Concatenate → Dense({best_hps.get('dense_units_1')}) → Dense({best_hps.get('dense_units_2')}) → Softmax(5)

OPTIMAL HYPERPARAMETERS (Keras Tuner)
{'─' * 60}
   • Learning Rate   : {best_hps.get('learning_rate')}
   • Dropout 1       : {best_hps.get('dropout_1')}
   • Dropout 2       : {best_hps.get('dropout_2')}
   • Dense Units 1   : {best_hps.get('dense_units_1')}
   • Dense Units 2   : {best_hps.get('dense_units_2')}
   • Optimizer       : {best_hps.get('optimizer')}

PERFORMANCE METRICS
{'─' * 60}
   • Accuracy        : {accuracy_4*100:.2f}%
   • Precision       : {precision_4:.4f}
   • Recall          : {recall_4:.4f}
   • F1-Score        : {f1_4:.4f}
   • Cohen's Kappa   : {kappa_4:.4f}

ROC AUC VALUES
{'─' * 60}""")

for i, name in enumerate(class_names_ordered):
    print(f"   • {name:<20}: {roc_auc_4[i]:.4f}")

print(f"""   {'─' * 40}
   • Macro-average       : {roc_auc_4['macro']:.4f}
   • Micro-average       : {roc_auc_4['micro']:.4f}

TRAINING DURATION
{'─' * 60}
   • Hyperparameter Tuning : {tuning_time/60:.2f} minutes
   • Final Training        : {training_time/60:.2f} minutes
   • Toplam                : {total_time_4/60:.2f} minutes
   • Epoch Count           : {total_epochs_4}
   • Ortalama Epoch        : {avg_epoch_time_4:.2f} saniye

SAVED FILES
{'─' * 60}
   • {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras
   • {RESULTS_PATH}/{MODEL_NAME}_history.csv
   • {RESULTS_PATH}/{MODEL_NAME}_history.png
   • {RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png
   • {RESULTS_PATH}/{MODEL_NAME}_roc_curves.png
""")

gc.collect()
print("=" * 70)
print("MODEL 4 (DenseNet121 + InceptionV3 Hybrid) TAMAMLANDI!")
print("=" * 70)

# Cell 17: MODEL 5 - ResNet50 + VGG16 (HYBRID)

In [ ]:
# =============================================================================
# CELL 17: MODEL 5 - ResNet50 + VGG16 (HYBRID MODEL)
# =============================================================================
# This model combines two classical and powerful CNN architectures:
# - ResNet50: Deep network with residual connections
# - VGG16: Simple yet effective architecture
#
# Features:
# - No fine-tuning (base models FROZEN)
# - Hyperparameter optimization via Keras Tuner
# =============================================================================

import time
import keras_tuner as kt
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, Concatenate, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

MODEL_NAME = "Model_05_ResNet50_VGG16_Hybrid"

print("=" * 70)
print("MODEL 5: ResNet50 + VGG16 (HYBRID)")
print("=" * 70)

print("""
Model Mimarisi:

    Input Image (224x224x3)
           |
    ┌──────┴──────┐
    ▼             ▼
  ResNet50      VGG16
  (FROZEN)     (FROZEN)
    |             |
   GAP           GAP
    |             |
Dense(256)    Dense(256)
Dropout       Dropout
Dense(128)    Dense(128)
Dropout       Dropout
    |             |
    └──────┬──────┘
           |
      Concatenate
           |
      Dense(256)
      Dropout
      Dense(128)
      Dropout
           |
      Dense(5) + Softmax
""")

# =============================================================================
# HYBRID MODEL BUILDER (FOR KERAS TUNER)
# =============================================================================

def build_resnet_vgg_hybrid(hp):
    """
    ResNet50 + VGG16 Hybrid model builder.
    Hyperparameter-tunable for Keras Tuner.
    """

    # Hyperparameters
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    dropout_1 = hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])
    dropout_2 = hp.Choice('dropout_2', values=[0.4, 0.5, 0.6])
    dense_units_1 = hp.Choice('dense_units_1', values=[256, 512])
    dense_units_2 = hp.Choice('dense_units_2', values=[128, 256])
    optimizer_name = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    # Input
    inputs = Input(shape=IMG_SHAPE, name='input_layer')

    # =========================================================================
    # BACKBONE 1: ResNet50
    # =========================================================================
    resnet = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE
    )
    resnet.trainable = False  # FROZEN

    # ResNet preprocessing ve feature extraction
    x1 = tf.keras.applications.resnet50.preprocess_input(inputs)
    x1 = resnet(x1, training=False)
    x1 = GlobalAveragePooling2D(name='gap_resnet')(x1)

    # ResNet classification head
    x1 = Dense(dense_units_1, activation='relu', name='dense1_resnet')(x1)
    x1 = Dropout(dropout_1, name='drop1_resnet')(x1)
    x1 = Dense(dense_units_2, activation='relu', name='dense2_resnet')(x1)
    x1 = Dropout(dropout_2, name='drop2_resnet')(x1)

    # =========================================================================
    # BACKBONE 2: VGG16
    # =========================================================================
    vgg = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE
    )
    vgg.trainable = False  # FROZEN

    # VGG preprocessing ve feature extraction
    x2 = tf.keras.applications.vgg16.preprocess_input(inputs)
    x2 = vgg(x2, training=False)
    x2 = GlobalAveragePooling2D(name='gap_vgg')(x2)

    # VGG classification head
    x2 = Dense(dense_units_1, activation='relu', name='dense1_vgg')(x2)
    x2 = Dropout(dropout_1, name='drop1_vgg')(x2)
    x2 = Dense(dense_units_2, activation='relu', name='dense2_vgg')(x2)
    x2 = Dropout(dropout_2, name='drop2_vgg')(x2)

    # =========================================================================
    # CONCATENATE & FINAL CLASSIFICATION
    # =========================================================================
    merged = Concatenate(name='concat')([x1, x2])

    # Final classification head
    x = Dense(dense_units_1, activation='relu', name='dense1_final')(merged)
    x = Dropout(dropout_1, name='drop1_final')(x)
    x = Dense(dense_units_2, activation='relu', name='dense2_final')(x)
    x = Dropout(dropout_2, name='drop2_final')(x)

    # Output
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

    # Model
    model = Model(inputs=inputs, outputs=outputs, name='ResNet50_VGG16_Hybrid')

    # Optimizer
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =============================================================================
# KERAS TUNER - HYPERPARAMETER SEARCH
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER OPTIMIZATION (KERAS TUNER)")
print("=" * 70)

tuner_dir = f'{MODEL_SAVE_PATH}/tuner_{MODEL_NAME}'

tuner = kt.BayesianOptimization(
    build_resnet_vgg_hybrid,
    objective='val_accuracy',
    max_trials=15,
    num_initial_points=5,
    directory=tuner_dir,
    project_name='resnet_vgg_tuning',
    overwrite=True
)

print("""
Keras Tuner Ayarlari:
   - Algoritma: Bayesian Optimization
   - Max Trials: 15
   - Objective: val_accuracy

Aranan Hyperparameter'lar:
   - Learning Rate: [0.01, 0.001, 0.0001]
   - Dropout 1: [0.2, 0.3, 0.4]
   - Dropout 2: [0.4, 0.5, 0.6]
   - Dense Units 1: [256, 512]
   - Dense Units 2: [128, 256]
   - Optimizer: [Adam, SGD, RMSprop]
""")

early_stop_tuner = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

print("Hyperparameter aramasi basliyor...")
print("(This process may take 30-40 minutes)\n")

start_tuning = time.time()

tuner.search(
    train_dataset_aug,
    epochs=20,
    validation_data=val_dataset_aug,
    callbacks=[early_stop_tuner],
    class_weight=class_weights,
    verbose=1
)

tuning_time = time.time() - start_tuning
print(f"\nTuning duration: {tuning_time/60:.2f} minutes")

# =============================================================================
# EN IYI HYPERPARAMETER'LAR
# =============================================================================

print("\n" + "=" * 70)
print("EN IYI HYPERPARAMETER'LAR")
print("=" * 70)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Optimal Hyperparameter'lar:
   - Learning Rate: {best_hps.get('learning_rate')}
   - Dropout 1: {best_hps.get('dropout_1')}
   - Dropout 2: {best_hps.get('dropout_2')}
   - Dense Units 1: {best_hps.get('dense_units_1')}
   - Dense Units 2: {best_hps.get('dense_units_2')}
   - Optimizer: {best_hps.get('optimizer')}
""")

# =============================================================================
# FINAL MODEL TRAINING
# =============================================================================

print("\n" + "=" * 70)
print("FINAL MODEL EGITIMI")
print("=" * 70)

model_5 = tuner.hypermodel.build(best_hps)

total_params_5 = model_5.count_params()
trainable_params_5 = sum([tf.keras.backend.count_params(w) for w in model_5.trainable_weights])

print(f"\nModel Bilgisi:")
print(f"   - Total parameters: {total_params_5:,}")
print(f"   - Egitilebilir: {trainable_params_5:,}")
print(f"   - Frozen (backbone): {total_params_5 - trainable_params_5:,}")

callbacks_final = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("\nFinal egitim basliyor (50 epoch)...")
start_training = time.time()

history_5 = model_5.fit(
    train_dataset_aug,
    epochs=50,
    validation_data=val_dataset_aug,
    callbacks=callbacks_final,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_training
total_time_5 = tuning_time + training_time
total_epochs_5 = len(history_5.history['accuracy'])
avg_epoch_time_5 = training_time / total_epochs_5

print(f"\nFinal training: {training_time/60:.2f} minutes ({total_epochs_5} epochs)")
print(f"Total duration: {total_time_5/60:.2f} minutes")

# =============================================================================
# EGITIM GRAFIKLERI
# =============================================================================

print("\n" + "=" * 70)
print("EGITIM GRAFIKLERI")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_5.history['accuracy']) + 1)

# Accuracy - Train/Test formatinda
axes[0].plot(epochs_range, history_5.history['accuracy'], 'b-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history_5.history['val_accuracy'], 'r-', label='Test', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss - Train/Test formatinda
axes[1].plot(epochs_range, history_5.history['loss'], 'b-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history_5.history['val_loss'], 'r-', label='Test', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL DEGERLENDIRME (TEST)")
print("=" * 70)

y_pred_proba_5 = model_5.predict(test_dataset_aug, verbose=1)
y_pred_5 = np.argmax(y_pred_proba_5, axis=1)

y_true_5 = []
for _, labels in test_dataset_aug:
    y_true_5.extend(np.argmax(labels.numpy(), axis=1))
y_true_5 = np.array(y_true_5)

# Metrikler
accuracy_5 = accuracy_score(y_true_5, y_pred_5)
precision_5 = precision_score(y_true_5, y_pred_5, average='weighted', zero_division=0)
recall_5 = recall_score(y_true_5, y_pred_5, average='weighted', zero_division=0)
f1_5 = f1_score(y_true_5, y_pred_5, average='weighted', zero_division=0)
kappa_5 = cohen_kappa_score(y_true_5, y_pred_5)

print(f"\nPerformance Metrics:")
print("-" * 50)
print(f"   Accuracy  : {accuracy_5:.4f} ({accuracy_5*100:.2f}%)")
print(f"   Precision : {precision_5:.4f}")
print(f"   Recall    : {recall_5:.4f}")
print(f"   F1-Score  : {f1_5:.4f}")
print(f"   Kappa     : {kappa_5:.4f}")

print(f"\nClassification Report:")
print(classification_report(y_true_5, y_pred_5, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm_5 = confusion_matrix(y_true_5, y_pred_5)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_5, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Tahmin')
plt.ylabel('Gercek')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

print("\n" + "=" * 70)
print("ROC EGRILERI")
print("=" * 70)

y_true_bin_5 = label_binarize(y_true_5, classes=range(NUM_CLASSES))
fpr_5, tpr_5, roc_auc_5 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_5[i], tpr_5[i], _ = roc_curve(y_true_bin_5[:, i], y_pred_proba_5[:, i])
    roc_auc_5[i] = auc(fpr_5[i], tpr_5[i])

fpr_5["micro"], tpr_5["micro"], _ = roc_curve(y_true_bin_5.ravel(), y_pred_proba_5.ravel())
roc_auc_5["micro"] = auc(fpr_5["micro"], tpr_5["micro"])

all_fpr_5 = np.unique(np.concatenate([fpr_5[i] for i in range(NUM_CLASSES)]))
mean_tpr_5 = np.zeros_like(all_fpr_5)
for i in range(NUM_CLASSES):
    mean_tpr_5 += np.interp(all_fpr_5, fpr_5[i], tpr_5[i])
mean_tpr_5 /= NUM_CLASSES
fpr_5["macro"], tpr_5["macro"] = all_fpr_5, mean_tpr_5
roc_auc_5["macro"] = auc(fpr_5["macro"], tpr_5["macro"])

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_5[i], tpr_5[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_5[i]:.4f})')
plt.plot(fpr_5["micro"], tpr_5["micro"], 'deeppink', ls=':', lw=3, label=f'Micro-avg (AUC={roc_auc_5["micro"]:.4f})')
plt.plot(fpr_5["macro"], tpr_5["macro"], 'navy', ls=':', lw=3, label=f'Macro-avg (AUC={roc_auc_5["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"\nROC AUC Degerleri:")
for i, name in enumerate(class_names_ordered):
    print(f"   {name}: {roc_auc_5[i]:.4f}")
print(f"   Macro-avg: {roc_auc_5['macro']:.4f}")
print(f"   Micro-avg: {roc_auc_5['micro']:.4f}")

# =============================================================================
# SONUCLARI KAYDET
# =============================================================================

results_5 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_5,
    'precision': precision_5,
    'recall': recall_5,
    'f1_score': f1_5,
    'kappa': kappa_5,
    'roc_auc_macro': roc_auc_5['macro'],
    'roc_auc_micro': roc_auc_5['micro'],
    'training_time_min': total_time_5 / 60,
    'avg_epoch_time_sec': avg_epoch_time_5,
    'total_epochs': total_epochs_5,
    'total_params': total_params_5,
    'best_lr': best_hps.get('learning_rate'),
    'best_dropout1': best_hps.get('dropout_1'),
    'best_dropout2': best_hps.get('dropout_2'),
    'best_optimizer': best_hps.get('optimizer')
}

for i, name in enumerate(class_names_ordered):
    results_5[f'auc_{name}'] = roc_auc_5[i]

all_results.append(results_5)

# Save model
model_5.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_5.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

print(f"\nModel kaydedildi: {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras")

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 70)
print("MODEL COMPARISON TABLE")
print("=" * 70)

print(f"\n{'Method':<45} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 100)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', (r['training_time_min'] * 60) / max(r['total_epochs'], 1))
    print(f"{r['model_name']:<45} {r['precision']:>10.4f} {r['recall']:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 100)
best = max(all_results, key=lambda x: x['accuracy'])
print(f"\nEn iyi model: {best['model_name']} (Accuracy: {best['accuracy']*100:.2f}%)")

# =============================================================================
# OZET RAPOR
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 5 - OZET RAPOR")
print("=" * 70)

print(f"""
Model Bilgileri:
   Model Adi         : {MODEL_NAME}
   Model Tipi        : Hybrid (Dual-Backbone)
   Backbone 1        : ResNet50 (FROZEN)
   Backbone 2        : VGG16 (FROZEN)
   Toplam Parametre  : {total_params_5:,}
   Egitilebilir      : {trainable_params_5:,}

Optimal Hyperparameter'lar (Keras Tuner):
   Learning Rate   : {best_hps.get('learning_rate')}
   Dropout 1       : {best_hps.get('dropout_1')}
   Dropout 2       : {best_hps.get('dropout_2')}
   Dense Units 1   : {best_hps.get('dense_units_1')}
   Dense Units 2   : {best_hps.get('dense_units_2')}
   Optimizer       : {best_hps.get('optimizer')}

Performans Metrikleri:
   Accuracy        : {accuracy_5*100:.2f}%
   Precision       : {precision_5:.4f}
   Recall          : {recall_5:.4f}
   F1-Score        : {f1_5:.4f}
   Cohen's Kappa   : {kappa_5:.4f}
   ROC AUC (Macro) : {roc_auc_5['macro']:.4f}

Egitim Suresi:
   Tuning          : {tuning_time/60:.2f} minutes
   Final Egitim    : {training_time/60:.2f} minutes
   Toplam          : {total_time_5/60:.2f} minutes
""")

gc.collect()
print("=" * 70)
print("MODEL 5 (ResNet50 + VGG16 Hybrid) TAMAMLANDI!")
print("=" * 70)

# Cell 18: MODEL 6 - EfficientNetB0 + MobileNetV2 (HYBRID)

In [ ]:
# =============================================================================
# CELL 18: MODEL 6 - EfficientNetB0 + MobileNetV2 (HYBRID MODEL)
# =============================================================================
# This model combines two lightweight and efficient CNN architectures:
# - EfficientNetB0: Architecture optimized via compound scaling
# - MobileNetV2: Lightweight architecture with depthwise separable convolution
#
# Ozellikler:
# - Base models FROZEN (transfer learning)
# - Hyperparameter optimization via Keras Tuner
# - Dual-backbone hybrid architecture
# =============================================================================

import time
import keras_tuner as kt
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, Concatenate, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

MODEL_NAME = "Model_06_EfficientNetB0_MobileNetV2_Hybrid"

print("=" * 70)
print("MODEL 6: EfficientNetB0 + MobileNetV2 (HYBRID)")
print("=" * 70)

print("""
Model Mimarisi:

    Input Image (224x224x3)
           |
    +------+------+
    |             |
EfficientNetB0  MobileNetV2
  (FROZEN)       (FROZEN)
    |             |
   GAP           GAP
    |             |
Dense(256)    Dense(256)
Dropout       Dropout
Dense(128)    Dense(128)
Dropout       Dropout
    |             |
    +------+------+
           |
      Concatenate
           |
      Dense(256)
      Dropout
      Dense(128)
      Dropout
           |
      Dense(5) + Softmax
           |
        Output

Backbone Ozellikleri:
- EfficientNetB0: 5.3M parametre, ImageNet top-1 accuracy 77.1%
- MobileNetV2: 3.4M parametre, ImageNet top-1 accuracy 71.8%
- Toplam: Hafif ve hizli hybrid model
""")

# =============================================================================
# HYBRID MODEL BUILDER (FOR KERAS TUNER)
# =============================================================================

def build_efficientnet_mobilenet_hybrid(hp):
    """
    EfficientNetB0 + MobileNetV2 Hybrid model builder.
    Keras Tuner ile hyperparameter optimization.
    """

    # Hyperparameters
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    dropout_1 = hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])
    dropout_2 = hp.Choice('dropout_2', values=[0.4, 0.5, 0.6])
    dense_units_1 = hp.Choice('dense_units_1', values=[256, 512])
    dense_units_2 = hp.Choice('dense_units_2', values=[128, 256])
    optimizer_name = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    # Input
    inputs = Input(shape=IMG_SHAPE, name='input_layer')

    # =========================================================================
    # BACKBONE 1: EfficientNetB0
    # =========================================================================
    efficientnet = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE
    )
    efficientnet.trainable = False  # FROZEN

    # EfficientNet branch
    x1 = efficientnet(inputs, training=False)
    x1 = GlobalAveragePooling2D(name='gap_efficientnet')(x1)
    x1 = Dense(dense_units_1, activation='relu', name='dense1_eff')(x1)
    x1 = Dropout(dropout_1, name='drop1_eff')(x1)
    x1 = Dense(dense_units_2, activation='relu', name='dense2_eff')(x1)
    x1 = Dropout(dropout_2, name='drop2_eff')(x1)

    # =========================================================================
    # BACKBONE 2: MobileNetV2
    # =========================================================================
    mobilenet = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE
    )
    mobilenet.trainable = False  # FROZEN

    # MobileNet branch
    x2 = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x2 = mobilenet(x2, training=False)
    x2 = GlobalAveragePooling2D(name='gap_mobilenet')(x2)
    x2 = Dense(dense_units_1, activation='relu', name='dense1_mob')(x2)
    x2 = Dropout(dropout_1, name='drop1_mob')(x2)
    x2 = Dense(dense_units_2, activation='relu', name='dense2_mob')(x2)
    x2 = Dropout(dropout_2, name='drop2_mob')(x2)

    # =========================================================================
    # CONCATENATE & FINAL CLASSIFICATION
    # =========================================================================
    merged = Concatenate(name='concat')([x1, x2])

    # Final classification head
    x = Dense(dense_units_1, activation='relu', name='dense1_final')(merged)
    x = Dropout(dropout_1, name='drop1_final')(x)
    x = Dense(dense_units_2, activation='relu', name='dense2_final')(x)
    x = Dropout(dropout_2, name='drop2_final')(x)

    # Output
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

    # Model
    model = Model(inputs=inputs, outputs=outputs, name='EfficientNetB0_MobileNetV2_Hybrid')

    # Optimizer
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =============================================================================
# KERAS TUNER - HYPERPARAMETER OPTIMIZATION
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER OPTIMIZATION (KERAS TUNER)")
print("=" * 70)

tuner_dir = f'{MODEL_SAVE_PATH}/tuner_{MODEL_NAME}'

tuner = kt.BayesianOptimization(
    build_efficientnet_mobilenet_hybrid,
    objective='val_accuracy',
    max_trials=15,
    num_initial_points=5,
    directory=tuner_dir,
    project_name='effnet_mobilenet_tuning',
    overwrite=True
)

print("""
Tuner Konfigurasyonu:
   - Algoritma      : Bayesian Optimization
   - Max Trials     : 15
   - Objective      : val_accuracy (maximize)

Hyperparameter Arama Uzayi:
   - Learning Rate  : [0.01, 0.001, 0.0001]
   - Dropout 1      : [0.2, 0.3, 0.4]
   - Dropout 2      : [0.4, 0.5, 0.6]
   - Dense Units 1  : [256, 512]
   - Dense Units 2  : [128, 256]
   - Optimizer      : [Adam, SGD, RMSprop]
""")

early_stop_tuner = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

print("Hyperparameter aramasi baslatiliyor...")
start_tuning = time.time()

tuner.search(
    train_dataset_aug,
    epochs=20,
    validation_data=val_dataset_aug,
    callbacks=[early_stop_tuner],
    class_weight=class_weights,
    verbose=1
)

tuning_time = time.time() - start_tuning
print(f"\nTuning complete: {tuning_time/60:.2f} minutes")

# =============================================================================
# OPTIMAL HYPERPARAMETERS
# =============================================================================

print("\n" + "=" * 70)
print("OPTIMAL HYPERPARAMETERS")
print("=" * 70)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Bulunan Optimal Degerler:
   - Learning Rate  : {best_hps.get('learning_rate')}
   - Dropout 1      : {best_hps.get('dropout_1')}
   - Dropout 2      : {best_hps.get('dropout_2')}
   - Dense Units 1  : {best_hps.get('dense_units_1')}
   - Dense Units 2  : {best_hps.get('dense_units_2')}
   - Optimizer      : {best_hps.get('optimizer')}
""")

# =============================================================================
# FINAL MODEL TRAINING
# =============================================================================

print("\n" + "=" * 70)
print("FINAL MODEL TRAINING")
print("=" * 70)

model_6 = tuner.hypermodel.build(best_hps)

total_params_6 = model_6.count_params()
trainable_params_6 = sum([tf.keras.backend.count_params(w) for w in model_6.trainable_weights])

print(f"""
Model Parametreleri:
   - Toplam         : {total_params_6:,}
   - Egitilebilir   : {trainable_params_6:,}
   - Frozen         : {total_params_6 - trainable_params_6:,}
""")

callbacks_final = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Final model egitimi baslatiliyor (50 epoch)...")
start_training = time.time()

history_6 = model_6.fit(
    train_dataset_aug,
    epochs=50,
    validation_data=val_dataset_aug,
    callbacks=callbacks_final,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_training
total_time_6 = tuning_time + training_time
total_epochs_6 = len(history_6.history['accuracy'])
avg_epoch_time_6 = training_time / total_epochs_6

print(f"\nFinal training complete: {training_time/60:.2f} minutes ({total_epochs_6} epochs)")
print(f"Total duration: {total_time_6/60:.2f} minutes")

# =============================================================================
# TRAINING HISTORY VISUALIZATION
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING HISTORY")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_6.history['accuracy']) + 1)

# Accuracy Plot
axes[0].plot(epochs_range, history_6.history['accuracy'], 'b-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history_6.history['val_accuracy'], 'r-', label='Test', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss Plot
axes[1].plot(epochs_range, history_6.history['loss'], 'b-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history_6.history['val_loss'], 'r-', label='Test', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

y_pred_proba_6 = model_6.predict(test_dataset_aug, verbose=1)
y_pred_6 = np.argmax(y_pred_proba_6, axis=1)

y_true_6 = []
for _, labels in test_dataset_aug:
    y_true_6.extend(np.argmax(labels.numpy(), axis=1))
y_true_6 = np.array(y_true_6)

# Metrics
accuracy_6 = accuracy_score(y_true_6, y_pred_6)
precision_6 = precision_score(y_true_6, y_pred_6, average='weighted', zero_division=0)
recall_6 = recall_score(y_true_6, y_pred_6, average='weighted', zero_division=0)
f1_6 = f1_score(y_true_6, y_pred_6, average='weighted', zero_division=0)
kappa_6 = cohen_kappa_score(y_true_6, y_pred_6)

print(f"""
Performance Metrics:
{'-' * 50}
   Accuracy     : {accuracy_6:.4f} ({accuracy_6*100:.2f}%)
   Precision    : {precision_6:.4f}
   Recall       : {recall_6:.4f}
   F1-Score     : {f1_6:.4f}
   Cohen Kappa  : {kappa_6:.4f}
{'-' * 50}
""")

print("Classification Report:")
print(classification_report(y_true_6, y_pred_6, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm_6 = confusion_matrix(y_true_6, y_pred_6)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_6, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

print("\n" + "=" * 70)
print("ROC CURVES")
print("=" * 70)

y_true_bin_6 = label_binarize(y_true_6, classes=range(NUM_CLASSES))
fpr_6, tpr_6, roc_auc_6 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_6[i], tpr_6[i], _ = roc_curve(y_true_bin_6[:, i], y_pred_proba_6[:, i])
    roc_auc_6[i] = auc(fpr_6[i], tpr_6[i])

fpr_6["micro"], tpr_6["micro"], _ = roc_curve(y_true_bin_6.ravel(), y_pred_proba_6.ravel())
roc_auc_6["micro"] = auc(fpr_6["micro"], tpr_6["micro"])

all_fpr_6 = np.unique(np.concatenate([fpr_6[i] for i in range(NUM_CLASSES)]))
mean_tpr_6 = np.zeros_like(all_fpr_6)
for i in range(NUM_CLASSES):
    mean_tpr_6 += np.interp(all_fpr_6, fpr_6[i], tpr_6[i])
mean_tpr_6 /= NUM_CLASSES
fpr_6["macro"], tpr_6["macro"] = all_fpr_6, mean_tpr_6
roc_auc_6["macro"] = auc(fpr_6["macro"], tpr_6["macro"])

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_6[i], tpr_6[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_6[i]:.4f})')
plt.plot(fpr_6["micro"], tpr_6["micro"], 'deeppink', ls=':', lw=3, label=f'Micro-avg (AUC={roc_auc_6["micro"]:.4f})')
plt.plot(fpr_6["macro"], tpr_6["macro"], 'navy', ls=':', lw=3, label=f'Macro-avg (AUC={roc_auc_6["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("ROC AUC Scores:")
for i, name in enumerate(class_names_ordered):
    print(f"   {name}: {roc_auc_6[i]:.4f}")
print(f"   Macro-average: {roc_auc_6['macro']:.4f}")
print(f"   Micro-average: {roc_auc_6['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_6 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_6,
    'precision': precision_6,
    'recall': recall_6,
    'f1_score': f1_6,
    'kappa': kappa_6,
    'roc_auc_macro': roc_auc_6['macro'],
    'roc_auc_micro': roc_auc_6['micro'],
    'training_time_min': total_time_6 / 60,
    'avg_epoch_time_sec': avg_epoch_time_6,
    'total_epochs': total_epochs_6,
    'total_params': total_params_6,
    'best_lr': best_hps.get('learning_rate'),
    'best_dropout1': best_hps.get('dropout_1'),
    'best_dropout2': best_hps.get('dropout_2'),
    'best_optimizer': best_hps.get('optimizer')
}

for i, name in enumerate(class_names_ordered):
    results_6[f'auc_{name}'] = roc_auc_6[i]

all_results.append(results_6)

# Save model
model_6.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_6.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

print(f"\nModel saved: {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras")

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 100)
print("MODEL COMPARISON TABLE")
print("=" * 100)

print(f"\n{'Method':<50} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 105)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', 0)
    print(f"{r['model_name']:<50} {r['precision']:>10.4f} {r['recall']:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 105)

best = max(all_results, key=lambda x: x['accuracy'])
print(f"\nBest Model: {best['model_name']} (Accuracy: {best['accuracy']*100:.2f}%)")

# =============================================================================
# MODEL SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 6 - SUMMARY REPORT")
print("=" * 70)

print(f"""
Model Information:
   Name            : {MODEL_NAME}
   Type            : Hybrid (Dual-Backbone)
   Backbone 1      : EfficientNetB0 (FROZEN)
   Backbone 2      : MobileNetV2 (FROZEN)
   Total Params    : {total_params_6:,}
   Trainable       : {trainable_params_6:,}

Optimal Hyperparameters:
   Learning Rate   : {best_hps.get('learning_rate')}
   Dropout 1       : {best_hps.get('dropout_1')}
   Dropout 2       : {best_hps.get('dropout_2')}
   Dense Units 1   : {best_hps.get('dense_units_1')}
   Dense Units 2   : {best_hps.get('dense_units_2')}
   Optimizer       : {best_hps.get('optimizer')}

Performance Metrics:
   Accuracy        : {accuracy_6*100:.2f}%
   Precision       : {precision_6:.4f}
   Recall          : {recall_6:.4f}
   F1-Score        : {f1_6:.4f}
   Cohen Kappa     : {kappa_6:.4f}
   ROC AUC (Macro) : {roc_auc_6['macro']:.4f}

Training Time:
   Tuning          : {tuning_time/60:.2f} minutes
   Final Training  : {training_time/60:.2f} minutes
   Total           : {total_time_6/60:.2f} minutes

Saved Files:
   - {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras
   - {MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras
   - {RESULTS_PATH}/{MODEL_NAME}_history.csv
   - {RESULTS_PATH}/{MODEL_NAME}_history.png
   - {RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png
   - {RESULTS_PATH}/{MODEL_NAME}_roc_curves.png
""")

gc.collect()
print("=" * 70)
print("MODEL 6 COMPLETED SUCCESSFULLY")
print("=" * 70)

# Cell 19: MODEL 7 - Xception + InceptionResNetV2 (HYBRID)

In [ ]:
# =============================================================================
# CELL 19: MODEL 7 - Xception + InceptionResNetV2 (HYBRID MODEL)
# =============================================================================
# This model combines two powerful and modern CNN architectures:
# - Xception: Depthwise separable convolutions, efficient architecture
# - InceptionResNetV2: Inception modules + Residual connections
#
# NOTE: These models require 299x299 input, so the dataset is re-created
# olusturulacak. Mevcut train/val/test split'leri korunacak.
# =============================================================================

import time
import keras_tuner as kt
from tensorflow.keras.applications import Xception, InceptionResNetV2
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, Concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

MODEL_NAME = "Model_07_Xception_InceptionResNetV2_Hybrid"

print("=" * 70)
print("MODEL 7: Xception + InceptionResNetV2 (HYBRID)")
print("=" * 70)

print("""
Model Mimarisi:

    Input Image (299x299x3)
           |
    +------+------+
    |             |
  Xception   InceptionResNetV2
  (FROZEN)      (FROZEN)
    |             |
   GAP           GAP
    |             |
Dense(256)    Dense(256)
Dropout       Dropout
Dense(128)    Dense(128)
Dropout       Dropout
    |             |
    +------+------+
           |
      Concatenate
           |
      Dense(256)
      Dropout
      Dense(128)
      Dropout
           |
      Dense(5) + Softmax
           |
        Output

Backbone Ozellikleri:
- Xception: 22.9M parametre, depthwise separable convolutions
- InceptionResNetV2: 55.9M parametre, inception + residual
- Input Size: 299x299 (native resolution for both models)
""")

# =============================================================================
# 299x299 DATASET OLUSTURMA
# =============================================================================

print("\n299x299 Dataset olusturuluyor...")

IMG_SIZE_299 = 299
IMG_SHAPE_299 = (IMG_SIZE_299, IMG_SIZE_299, 3)

# Augmentation layer for 299x299
augmentation_layer_299 = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.RandomBrightness(0.1),
], name='augmentation_299')

# Resize function
def resize_to_299(image, label):
    image = tf.image.resize(image, [IMG_SIZE_299, IMG_SIZE_299])
    return image, label

# Train dataset 299x299 (augmented)
train_dataset_299 = train_dataset.map(resize_to_299, num_parallel_calls=AUTOTUNE)
train_dataset_299 = train_dataset_299.map(
    lambda x, y: (augmentation_layer_299(x, training=True), y),
    num_parallel_calls=AUTOTUNE
)
train_dataset_299 = train_dataset_299.prefetch(AUTOTUNE)

# Validation dataset 299x299
val_dataset_299 = val_dataset.map(resize_to_299, num_parallel_calls=AUTOTUNE)
val_dataset_299 = val_dataset_299.prefetch(AUTOTUNE)

# Test dataset 299x299
test_dataset_299 = test_dataset.map(resize_to_299, num_parallel_calls=AUTOTUNE)
test_dataset_299 = test_dataset_299.prefetch(AUTOTUNE)

print("299x299 Dataset hazir!")

# =============================================================================
# HYBRID MODEL BUILDER (FOR KERAS TUNER)
# =============================================================================

def build_xception_inceptionresnet_hybrid(hp):
    """
    Xception + InceptionResNetV2 Hybrid model builder.
    Keras Tuner ile hyperparameter optimization.
    """

    # Hyperparameters
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    dropout_1 = hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])
    dropout_2 = hp.Choice('dropout_2', values=[0.4, 0.5, 0.6])
    dense_units_1 = hp.Choice('dense_units_1', values=[256, 512])
    dense_units_2 = hp.Choice('dense_units_2', values=[128, 256])
    optimizer_name = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    # Input
    inputs = Input(shape=IMG_SHAPE_299, name='input_layer')

    # =========================================================================
    # BACKBONE 1: Xception
    # =========================================================================
    xception = Xception(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE_299
    )
    xception.trainable = False  # FROZEN

    # Xception branch
    x1 = tf.keras.applications.xception.preprocess_input(inputs)
    x1 = xception(x1, training=False)
    x1 = GlobalAveragePooling2D(name='gap_xception')(x1)
    x1 = Dense(dense_units_1, activation='relu', name='dense1_xception')(x1)
    x1 = Dropout(dropout_1, name='drop1_xception')(x1)
    x1 = Dense(dense_units_2, activation='relu', name='dense2_xception')(x1)
    x1 = Dropout(dropout_2, name='drop2_xception')(x1)

    # =========================================================================
    # BACKBONE 2: InceptionResNetV2
    # =========================================================================
    inception_resnet = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE_299
    )
    inception_resnet.trainable = False  # FROZEN

    # InceptionResNetV2 branch
    x2 = tf.keras.applications.inception_resnet_v2.preprocess_input(inputs)
    x2 = inception_resnet(x2, training=False)
    x2 = GlobalAveragePooling2D(name='gap_inception_resnet')(x2)
    x2 = Dense(dense_units_1, activation='relu', name='dense1_incres')(x2)
    x2 = Dropout(dropout_1, name='drop1_incres')(x2)
    x2 = Dense(dense_units_2, activation='relu', name='dense2_incres')(x2)
    x2 = Dropout(dropout_2, name='drop2_incres')(x2)

    # =========================================================================
    # CONCATENATE & FINAL CLASSIFICATION
    # =========================================================================
    merged = Concatenate(name='concat')([x1, x2])

    # Final classification head
    x = Dense(dense_units_1, activation='relu', name='dense1_final')(merged)
    x = Dropout(dropout_1, name='drop1_final')(x)
    x = Dense(dense_units_2, activation='relu', name='dense2_final')(x)
    x = Dropout(dropout_2, name='drop2_final')(x)

    # Output
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

    # Model
    model = Model(inputs=inputs, outputs=outputs, name='Xception_InceptionResNetV2_Hybrid')

    # Optimizer
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =============================================================================
# KERAS TUNER - HYPERPARAMETER OPTIMIZATION
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER OPTIMIZATION (KERAS TUNER)")
print("=" * 70)

tuner_dir = f'{MODEL_SAVE_PATH}/tuner_{MODEL_NAME}'

tuner = kt.BayesianOptimization(
    build_xception_inceptionresnet_hybrid,
    objective='val_accuracy',
    max_trials=15,
    num_initial_points=5,
    directory=tuner_dir,
    project_name='xception_incresnet_tuning',
    overwrite=True
)

print("""
Tuner Configuration:
   - Algorithm      : Bayesian Optimization
   - Max Trials     : 15
   - Objective      : val_accuracy (maximize)

Hyperparameter Search Space:
   - Learning Rate  : [0.01, 0.001, 0.0001]
   - Dropout 1      : [0.2, 0.3, 0.4]
   - Dropout 2      : [0.4, 0.5, 0.6]
   - Dense Units 1  : [256, 512]
   - Dense Units 2  : [128, 256]
   - Optimizer      : [Adam, SGD, RMSprop]
""")

early_stop_tuner = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

print("Hyperparameter search starting...")
start_tuning = time.time()

tuner.search(
    train_dataset_299,
    epochs=20,
    validation_data=val_dataset_299,
    callbacks=[early_stop_tuner],
    class_weight=class_weights,
    verbose=1
)

tuning_time = time.time() - start_tuning
print(f"\nTuning completed: {tuning_time/60:.2f} minutes")

# =============================================================================
# OPTIMAL HYPERPARAMETERS
# =============================================================================

print("\n" + "=" * 70)
print("OPTIMAL HYPERPARAMETERS")
print("=" * 70)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Optimal Values Found:
   - Learning Rate  : {best_hps.get('learning_rate')}
   - Dropout 1      : {best_hps.get('dropout_1')}
   - Dropout 2      : {best_hps.get('dropout_2')}
   - Dense Units 1  : {best_hps.get('dense_units_1')}
   - Dense Units 2  : {best_hps.get('dense_units_2')}
   - Optimizer      : {best_hps.get('optimizer')}
""")

# =============================================================================
# FINAL MODEL TRAINING
# =============================================================================

print("\n" + "=" * 70)
print("FINAL MODEL TRAINING")
print("=" * 70)

model_7 = tuner.hypermodel.build(best_hps)

total_params_7 = model_7.count_params()
trainable_params_7 = sum([tf.keras.backend.count_params(w) for w in model_7.trainable_weights])

print(f"""
Model Parameters:
   - Total          : {total_params_7:,}
   - Trainable      : {trainable_params_7:,}
   - Frozen         : {total_params_7 - trainable_params_7:,}
""")

callbacks_final = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Final model training starting (50 epochs)...")
start_training = time.time()

history_7 = model_7.fit(
    train_dataset_299,
    epochs=50,
    validation_data=val_dataset_299,
    callbacks=callbacks_final,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_training
total_time_7 = tuning_time + training_time
total_epochs_7 = len(history_7.history['accuracy'])
avg_epoch_time_7 = training_time / total_epochs_7

print(f"\nFinal training completed: {training_time/60:.2f} minutes ({total_epochs_7} epochs)")
print(f"Total time: {total_time_7/60:.2f} minutes")

# =============================================================================
# TRAINING HISTORY VISUALIZATION
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING HISTORY")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_7.history['accuracy']) + 1)

# Accuracy Plot
axes[0].plot(epochs_range, history_7.history['accuracy'], 'b-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history_7.history['val_accuracy'], 'r-', label='Test', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss Plot
axes[1].plot(epochs_range, history_7.history['loss'], 'b-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history_7.history['val_loss'], 'r-', label='Test', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

y_pred_proba_7 = model_7.predict(test_dataset_299, verbose=1)
y_pred_7 = np.argmax(y_pred_proba_7, axis=1)

y_true_7 = []
for _, labels in test_dataset_299:
    y_true_7.extend(np.argmax(labels.numpy(), axis=1))
y_true_7 = np.array(y_true_7)

# Metrics
accuracy_7 = accuracy_score(y_true_7, y_pred_7)
precision_7 = precision_score(y_true_7, y_pred_7, average='weighted', zero_division=0)
recall_7 = recall_score(y_true_7, y_pred_7, average='weighted', zero_division=0)
f1_7 = f1_score(y_true_7, y_pred_7, average='weighted', zero_division=0)
kappa_7 = cohen_kappa_score(y_true_7, y_pred_7)

print(f"""
Performance Metrics:
{'-' * 50}
   Accuracy     : {accuracy_7:.4f} ({accuracy_7*100:.2f}%)
   Precision    : {precision_7:.4f}
   Recall       : {recall_7:.4f}
   F1-Score     : {f1_7:.4f}
   Cohen Kappa  : {kappa_7:.4f}
{'-' * 50}
""")

print("Classification Report:")
print(classification_report(y_true_7, y_pred_7, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm_7 = confusion_matrix(y_true_7, y_pred_7)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_7, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

print("\n" + "=" * 70)
print("ROC CURVES")
print("=" * 70)

y_true_bin_7 = label_binarize(y_true_7, classes=range(NUM_CLASSES))
fpr_7, tpr_7, roc_auc_7 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_7[i], tpr_7[i], _ = roc_curve(y_true_bin_7[:, i], y_pred_proba_7[:, i])
    roc_auc_7[i] = auc(fpr_7[i], tpr_7[i])

fpr_7["micro"], tpr_7["micro"], _ = roc_curve(y_true_bin_7.ravel(), y_pred_proba_7.ravel())
roc_auc_7["micro"] = auc(fpr_7["micro"], tpr_7["micro"])

all_fpr_7 = np.unique(np.concatenate([fpr_7[i] for i in range(NUM_CLASSES)]))
mean_tpr_7 = np.zeros_like(all_fpr_7)
for i in range(NUM_CLASSES):
    mean_tpr_7 += np.interp(all_fpr_7, fpr_7[i], tpr_7[i])
mean_tpr_7 /= NUM_CLASSES
fpr_7["macro"], tpr_7["macro"] = all_fpr_7, mean_tpr_7
roc_auc_7["macro"] = auc(fpr_7["macro"], tpr_7["macro"])

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_7[i], tpr_7[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_7[i]:.4f})')
plt.plot(fpr_7["micro"], tpr_7["micro"], 'deeppink', ls=':', lw=3, label=f'Micro-avg (AUC={roc_auc_7["micro"]:.4f})')
plt.plot(fpr_7["macro"], tpr_7["macro"], 'navy', ls=':', lw=3, label=f'Macro-avg (AUC={roc_auc_7["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("ROC AUC Scores:")
for i, name in enumerate(class_names_ordered):
    print(f"   {name}: {roc_auc_7[i]:.4f}")
print(f"   Macro-average: {roc_auc_7['macro']:.4f}")
print(f"   Micro-average: {roc_auc_7['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_7 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_7,
    'precision': precision_7,
    'recall': recall_7,
    'f1_score': f1_7,
    'kappa': kappa_7,
    'roc_auc_macro': roc_auc_7['macro'],
    'roc_auc_micro': roc_auc_7['micro'],
    'training_time_min': total_time_7 / 60,
    'avg_epoch_time_sec': avg_epoch_time_7,
    'total_epochs': total_epochs_7,
    'total_params': total_params_7,
    'best_lr': best_hps.get('learning_rate'),
    'best_dropout1': best_hps.get('dropout_1'),
    'best_dropout2': best_hps.get('dropout_2'),
    'best_optimizer': best_hps.get('optimizer')
}

for i, name in enumerate(class_names_ordered):
    results_7[f'auc_{name}'] = roc_auc_7[i]

all_results.append(results_7)

# Save model
model_7.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_7.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

print(f"\nModel saved: {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras")

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 100)
print("MODEL COMPARISON TABLE")
print("=" * 100)

print(f"\n{'Method':<50} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 105)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', 0)
    print(f"{r['model_name']:<50} {r['precision']:>10.4f} {r['recall']:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 105)

best = max(all_results, key=lambda x: x['accuracy'])
print(f"\nBest Model: {best['model_name']} (Accuracy: {best['accuracy']*100:.2f}%)")

# =============================================================================
# MODEL SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 7 - SUMMARY REPORT")
print("=" * 70)

print(f"""
Model Information:
   Name            : {MODEL_NAME}
   Type            : Hybrid (Dual-Backbone)
   Backbone 1      : Xception (FROZEN)
   Backbone 2      : InceptionResNetV2 (FROZEN)
   Input Size      : 299x299x3
   Total Params    : {total_params_7:,}
   Trainable       : {trainable_params_7:,}

Optimal Hyperparameters:
   Learning Rate   : {best_hps.get('learning_rate')}
   Dropout 1       : {best_hps.get('dropout_1')}
   Dropout 2       : {best_hps.get('dropout_2')}
   Dense Units 1   : {best_hps.get('dense_units_1')}
   Dense Units 2   : {best_hps.get('dense_units_2')}
   Optimizer       : {best_hps.get('optimizer')}

Performance Metrics:
   Accuracy        : {accuracy_7*100:.2f}%
   Precision       : {precision_7:.4f}
   Recall          : {recall_7:.4f}
   F1-Score        : {f1_7:.4f}
   Cohen Kappa     : {kappa_7:.4f}
   ROC AUC (Macro) : {roc_auc_7['macro']:.4f}

Training Time:
   Tuning          : {tuning_time/60:.2f} minutes
   Final Training  : {training_time/60:.2f} minutes
   Total           : {total_time_7/60:.2f} minutes
""")

gc.collect()
print("=" * 70)
print("MODEL 7 COMPLETED SUCCESSFULLY")
print("=" * 70)

# Cell 20: MODEL 8 - InceptionResNetV2 + CBAM (ATTENTION)

In [ ]:
# =============================================================================
# CELL 20: MODEL 8 - InceptionResNetV2 + CBAM (ATTENTION MODEL)
# =============================================================================
# This model combines transfer learning with attention mechanisms:
# - InceptionResNetV2: Guclu feature extraction
# - CBAM: Convolutional Block Attention Module (Channel + Spatial Attention)
#
# CBAM enables the model to focus on salient regions.
# Augmentation oranlari 0.10'a dusurulmustur (daha stabil egitim icin).
# =============================================================================

import time
import keras_tuner as kt
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                      Dropout, Multiply, Add, Activation,
                                      Reshape, Concatenate, Conv2D,
                                      GlobalMaxPooling2D, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow.keras.backend as K

MODEL_NAME = "Model_08_InceptionResNetV2_CBAM"

print("=" * 70)
print("MODEL 8: InceptionResNetV2 + CBAM (ATTENTION)")
print("=" * 70)

print("""
Model Mimarisi:

    Input Image (299x299x3)
           |
    InceptionResNetV2
        (FROZEN)
           |
        CBAM
    (Attention Module)
           |
    +------+------+
    |             |
 Channel      Spatial
 Attention    Attention
    |             |
    +------+------+
           |
          GAP
           |
      Dense(512)
      Dropout
      Dense(256)
      Dropout
           |
      Dense(5) + Softmax
           |
        Output

CBAM (Convolutional Block Attention Module):
- Channel Attention: "Ne" onemli (feature maps)
- Spatial Attention: "Nerede" onemli (lokasyon)
- Sirasiyla uygulanir: Channel -> Spatial
""")

# =============================================================================
# 299x299 DATASET (DUSUK AUGMENTATION)
# =============================================================================

print("\n299x299 Dataset olusturuluyor (dusuk augmentation)...")

IMG_SIZE_299 = 299
IMG_SHAPE_299 = (IMG_SIZE_299, IMG_SIZE_299, 3)

# Dusurulmus augmentation layer (0.15 -> 0.10)
augmentation_layer_low = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.10),  # 0.15'ten 0.10'a dusuruldu
    tf.keras.layers.RandomZoom(0.10),       # 0.15'ten 0.10'a dusuruldu
    tf.keras.layers.RandomContrast(0.05),   # 0.10'dan 0.05'e dusuruldu
    tf.keras.layers.RandomBrightness(0.05), # 0.10'dan 0.05'e dusuruldu
], name='augmentation_low')

# Resize function
def resize_to_299(image, label):
    image = tf.image.resize(image, [IMG_SIZE_299, IMG_SIZE_299])
    return image, label

# Train dataset 299x299 (dusuk augmentation)
train_dataset_299_low = train_dataset.map(resize_to_299, num_parallel_calls=AUTOTUNE)
train_dataset_299_low = train_dataset_299_low.map(
    lambda x, y: (augmentation_layer_low(x, training=True), y),
    num_parallel_calls=AUTOTUNE
)
train_dataset_299_low = train_dataset_299_low.prefetch(AUTOTUNE)

# Validation dataset 299x299 (augmentation yok)
val_dataset_299 = val_dataset.map(resize_to_299, num_parallel_calls=AUTOTUNE)
val_dataset_299 = val_dataset_299.prefetch(AUTOTUNE)

# Test dataset 299x299 (augmentation yok)
test_dataset_299 = test_dataset.map(resize_to_299, num_parallel_calls=AUTOTUNE)
test_dataset_299 = test_dataset_299.prefetch(AUTOTUNE)

print("299x299 Dataset hazir (dusuk augmentation)!")

# =============================================================================
# CBAM MODULU
# =============================================================================

def channel_attention(input_tensor, ratio=8):
    """
    Channel Attention Module.
    Her feature channel'in onemini hesaplar.
    """
    channel = input_tensor.shape[-1]

    # Global Average Pooling
    avg_pool = GlobalAveragePooling2D()(input_tensor)
    avg_pool = Reshape((1, 1, channel))(avg_pool)

    # Global Max Pooling
    max_pool = GlobalMaxPooling2D()(input_tensor)
    max_pool = Reshape((1, 1, channel))(max_pool)

    # Shared MLP
    shared_dense1 = Dense(channel // ratio, activation='relu', name='channel_dense1')
    shared_dense2 = Dense(channel, name='channel_dense2')

    avg_out = shared_dense2(shared_dense1(avg_pool))
    max_out = shared_dense2(shared_dense1(max_pool))

    # Combine
    attention = Add()([avg_out, max_out])
    attention = Activation('sigmoid')(attention)

    return Multiply()([input_tensor, attention])


def spatial_attention(input_tensor, kernel_size=7):
    """
    Spatial Attention Module.
    Goruntunun hangi bolgelerinin onemli oldugunu hesaplar.
    """
    # Channel-wise average ve max
    avg_pool = Lambda(lambda x: K.mean(x, axis=-1, keepdims=True))(input_tensor)
    max_pool = Lambda(lambda x: K.max(x, axis=-1, keepdims=True))(input_tensor)

    # Concatenate
    concat = Concatenate(axis=-1)([avg_pool, max_pool])

    # Conv layer
    attention = Conv2D(1, kernel_size=kernel_size, padding='same', activation='sigmoid', name='spatial_conv')(concat)

    return Multiply()([input_tensor, attention])


def cbam_block(input_tensor, ratio=8, kernel_size=7):
    """
    CBAM: Channel Attention + Spatial Attention (sirasiyla).
    """
    x = channel_attention(input_tensor, ratio)
    x = spatial_attention(x, kernel_size)
    return x

# =============================================================================
# MODEL BUILDER (FOR KERAS TUNER)
# =============================================================================

def build_inceptionresnet_cbam(hp):
    """
    InceptionResNetV2 + CBAM model builder.
    Keras Tuner ile hyperparameter optimization.
    """

    # Hyperparameters
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    dropout_1 = hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])
    dropout_2 = hp.Choice('dropout_2', values=[0.4, 0.5, 0.6])
    dense_units_1 = hp.Choice('dense_units_1', values=[256, 512])
    dense_units_2 = hp.Choice('dense_units_2', values=[128, 256])
    optimizer_name = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])
    cbam_ratio = hp.Choice('cbam_ratio', values=[8, 16])

    # Input
    inputs = Input(shape=IMG_SHAPE_299, name='input_layer')

    # Preprocessing
    x = tf.keras.applications.inception_resnet_v2.preprocess_input(inputs)

    # Base model: InceptionResNetV2
    base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE_299
    )
    base_model.trainable = False  # FROZEN

    # Feature extraction
    features = base_model(x, training=False)

    # CBAM Attention
    attended_features = cbam_block(features, ratio=cbam_ratio)

    # Global Average Pooling
    x = GlobalAveragePooling2D(name='gap')(attended_features)

    # Classification head
    x = Dense(dense_units_1, activation='relu', name='dense1')(x)
    x = Dropout(dropout_1, name='drop1')(x)
    x = Dense(dense_units_2, activation='relu', name='dense2')(x)
    x = Dropout(dropout_2, name='drop2')(x)

    # Output
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

    # Model
    model = Model(inputs=inputs, outputs=outputs, name='InceptionResNetV2_CBAM')

    # Optimizer
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =============================================================================
# KERAS TUNER - HYPERPARAMETER OPTIMIZATION
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER OPTIMIZATION (KERAS TUNER)")
print("=" * 70)

tuner_dir = f'{MODEL_SAVE_PATH}/tuner_{MODEL_NAME}'

tuner = kt.BayesianOptimization(
    build_inceptionresnet_cbam,
    objective='val_accuracy',
    max_trials=15,
    num_initial_points=5,
    directory=tuner_dir,
    project_name='inceptionresnet_cbam_tuning',
    overwrite=True
)

print("""
Tuner Configuration:
   - Algorithm      : Bayesian Optimization
   - Max Trials     : 15
   - Objective      : val_accuracy (maximize)

Hyperparameter Search Space:
   - Learning Rate  : [0.01, 0.001, 0.0001]
   - Dropout 1      : [0.2, 0.3, 0.4]
   - Dropout 2      : [0.4, 0.5, 0.6]
   - Dense Units 1  : [256, 512]
   - Dense Units 2  : [128, 256]
   - Optimizer      : [Adam, SGD, RMSprop]
   - CBAM Ratio     : [8, 16]

Augmentation (Dusurulmus):
   - RandomRotation : 0.10 (onceki: 0.15)
   - RandomZoom     : 0.10 (onceki: 0.15)
   - RandomContrast : 0.05 (onceki: 0.10)
   - RandomBrightness: 0.05 (onceki: 0.10)
""")

early_stop_tuner = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

print("Hyperparameter search starting...")
start_tuning = time.time()

tuner.search(
    train_dataset_299_low,
    epochs=20,
    validation_data=val_dataset_299,
    callbacks=[early_stop_tuner],
    class_weight=class_weights,
    verbose=1
)

tuning_time = time.time() - start_tuning
print(f"\nTuning completed: {tuning_time/60:.2f} minutes")

# =============================================================================
# OPTIMAL HYPERPARAMETERS
# =============================================================================

print("\n" + "=" * 70)
print("OPTIMAL HYPERPARAMETERS")
print("=" * 70)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Optimal Values Found:
   - Learning Rate  : {best_hps.get('learning_rate')}
   - Dropout 1      : {best_hps.get('dropout_1')}
   - Dropout 2      : {best_hps.get('dropout_2')}
   - Dense Units 1  : {best_hps.get('dense_units_1')}
   - Dense Units 2  : {best_hps.get('dense_units_2')}
   - Optimizer      : {best_hps.get('optimizer')}
   - CBAM Ratio     : {best_hps.get('cbam_ratio')}
""")

# =============================================================================
# FINAL MODEL TRAINING
# =============================================================================

print("\n" + "=" * 70)
print("FINAL MODEL TRAINING")
print("=" * 70)

model_8 = tuner.hypermodel.build(best_hps)

total_params_8 = model_8.count_params()
trainable_params_8 = sum([tf.keras.backend.count_params(w) for w in model_8.trainable_weights])

print(f"""
Model Parameters:
   - Total          : {total_params_8:,}
   - Trainable      : {trainable_params_8:,}
   - Frozen         : {total_params_8 - trainable_params_8:,}
""")

callbacks_final = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Final model training starting (50 epochs)...")
start_training = time.time()

history_8 = model_8.fit(
    train_dataset_299_low,
    epochs=50,
    validation_data=val_dataset_299,
    callbacks=callbacks_final,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_training
total_time_8 = tuning_time + training_time
total_epochs_8 = len(history_8.history['accuracy'])
avg_epoch_time_8 = training_time / total_epochs_8

print(f"\nFinal training completed: {training_time/60:.2f} minutes ({total_epochs_8} epochs)")
print(f"Total time: {total_time_8/60:.2f} minutes")

# =============================================================================
# TRAINING HISTORY VISUALIZATION
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING HISTORY")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_8.history['accuracy']) + 1)

# Accuracy Plot
axes[0].plot(epochs_range, history_8.history['accuracy'], 'b-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history_8.history['val_accuracy'], 'r-', label='Test', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss Plot
axes[1].plot(epochs_range, history_8.history['loss'], 'b-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history_8.history['val_loss'], 'r-', label='Test', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

y_pred_proba_8 = model_8.predict(test_dataset_299, verbose=1)
y_pred_8 = np.argmax(y_pred_proba_8, axis=1)

y_true_8 = []
for _, labels in test_dataset_299:
    y_true_8.extend(np.argmax(labels.numpy(), axis=1))
y_true_8 = np.array(y_true_8)

# Metrics
accuracy_8 = accuracy_score(y_true_8, y_pred_8)
precision_8 = precision_score(y_true_8, y_pred_8, average='weighted', zero_division=0)
recall_8 = recall_score(y_true_8, y_pred_8, average='weighted', zero_division=0)
f1_8 = f1_score(y_true_8, y_pred_8, average='weighted', zero_division=0)
kappa_8 = cohen_kappa_score(y_true_8, y_pred_8)

print(f"""
Performance Metrics:
{'-' * 50}
   Accuracy     : {accuracy_8:.4f} ({accuracy_8*100:.2f}%)
   Precision    : {precision_8:.4f}
   Recall       : {recall_8:.4f}
   F1-Score     : {f1_8:.4f}
   Cohen Kappa  : {kappa_8:.4f}
{'-' * 50}
""")

print("Classification Report:")
print(classification_report(y_true_8, y_pred_8, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm_8 = confusion_matrix(y_true_8, y_pred_8)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_8, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

print("\n" + "=" * 70)
print("ROC CURVES")
print("=" * 70)

y_true_bin_8 = label_binarize(y_true_8, classes=range(NUM_CLASSES))
fpr_8, tpr_8, roc_auc_8 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_8[i], tpr_8[i], _ = roc_curve(y_true_bin_8[:, i], y_pred_proba_8[:, i])
    roc_auc_8[i] = auc(fpr_8[i], tpr_8[i])

fpr_8["micro"], tpr_8["micro"], _ = roc_curve(y_true_bin_8.ravel(), y_pred_proba_8.ravel())
roc_auc_8["micro"] = auc(fpr_8["micro"], tpr_8["micro"])

all_fpr_8 = np.unique(np.concatenate([fpr_8[i] for i in range(NUM_CLASSES)]))
mean_tpr_8 = np.zeros_like(all_fpr_8)
for i in range(NUM_CLASSES):
    mean_tpr_8 += np.interp(all_fpr_8, fpr_8[i], tpr_8[i])
mean_tpr_8 /= NUM_CLASSES
fpr_8["macro"], tpr_8["macro"] = all_fpr_8, mean_tpr_8
roc_auc_8["macro"] = auc(fpr_8["macro"], tpr_8["macro"])

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_8[i], tpr_8[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_8[i]:.4f})')
plt.plot(fpr_8["micro"], tpr_8["micro"], 'deeppink', ls=':', lw=3, label=f'Micro-avg (AUC={roc_auc_8["micro"]:.4f})')
plt.plot(fpr_8["macro"], tpr_8["macro"], 'navy', ls=':', lw=3, label=f'Macro-avg (AUC={roc_auc_8["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("ROC AUC Scores:")
for i, name in enumerate(class_names_ordered):
    print(f"   {name}: {roc_auc_8[i]:.4f}")
print(f"   Macro-average: {roc_auc_8['macro']:.4f}")
print(f"   Micro-average: {roc_auc_8['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_8 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_8,
    'precision': precision_8,
    'recall': recall_8,
    'f1_score': f1_8,
    'kappa': kappa_8,
    'roc_auc_macro': roc_auc_8['macro'],
    'roc_auc_micro': roc_auc_8['micro'],
    'training_time_min': total_time_8 / 60,
    'avg_epoch_time_sec': avg_epoch_time_8,
    'total_epochs': total_epochs_8,
    'total_params': total_params_8,
    'best_lr': best_hps.get('learning_rate'),
    'best_dropout1': best_hps.get('dropout_1'),
    'best_dropout2': best_hps.get('dropout_2'),
    'best_optimizer': best_hps.get('optimizer'),
    'best_cbam_ratio': best_hps.get('cbam_ratio')
}

for i, name in enumerate(class_names_ordered):
    results_8[f'auc_{name}'] = roc_auc_8[i]

all_results.append(results_8)

# Save model
model_8.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_8.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

print(f"\nModel saved: {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras")

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 100)
print("MODEL COMPARISON TABLE")
print("=" * 100)

print(f"\n{'Method':<50} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 105)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', 0)
    print(f"{r['model_name']:<50} {r['precision']:>10.4f} {r['recall']:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 105)

best = max(all_results, key=lambda x: x['accuracy'])
print(f"\nBest Model: {best['model_name']} (Accuracy: {best['accuracy']*100:.2f}%)")

# =============================================================================
# MODEL SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 8 - SUMMARY REPORT")
print("=" * 70)

print(f"""
Model Information:
   Name            : {MODEL_NAME}
   Type            : Attention-based (CBAM)
   Backbone        : InceptionResNetV2 (FROZEN)
   Attention       : CBAM (Channel + Spatial)
   Input Size      : 299x299x3
   Total Params    : {total_params_8:,}
   Trainable       : {trainable_params_8:,}

CBAM Details:
   - Channel Attention: Global Avg/Max Pool + Shared MLP
   - Spatial Attention: Channel Avg/Max + Conv2D
   - Ratio: {best_hps.get('cbam_ratio')}

Augmentation (Reduced):
   - RandomRotation  : 0.10
   - RandomZoom      : 0.10
   - RandomContrast  : 0.05
   - RandomBrightness: 0.05

Optimal Hyperparameters:
   Learning Rate   : {best_hps.get('learning_rate')}
   Dropout 1       : {best_hps.get('dropout_1')}
   Dropout 2       : {best_hps.get('dropout_2')}
   Dense Units 1   : {best_hps.get('dense_units_1')}
   Dense Units 2   : {best_hps.get('dense_units_2')}
   Optimizer       : {best_hps.get('optimizer')}

Performance Metrics:
   Accuracy        : {accuracy_8*100:.2f}%
   Precision       : {precision_8:.4f}
   Recall          : {recall_8:.4f}
   F1-Score        : {f1_8:.4f}
   Cohen Kappa     : {kappa_8:.4f}
   ROC AUC (Macro) : {roc_auc_8['macro']:.4f}

Training Time:
   Tuning          : {tuning_time/60:.2f} minutes
   Final Training  : {training_time/60:.2f} minutes
   Total           : {total_time_8/60:.2f} minutes
""")

gc.collect()
print("=" * 70)
print("MODEL 8 COMPLETED SUCCESSFULLY")
print("=" * 70)

# Cell 21: MODEL 9 - Xception + FPN (MULTI-SCALE CLASSIFICATION)

In [ ]:
# =============================================================================
# Cell 21: MODEL 9 - Xception + FPN (MULTI-SCALE CLASSIFICATION)
# =============================================================================
# TECHNICAL FIX REPORT:
# 1. "ResizeNearestNeighborGrad" error resolved via Lambda layers.
# 2. All required plots (ROC, Confusion Matrix, History) are included.
# 3. Memory is conserved by reusing 'train_dataset_299' from Model 7.
# =============================================================================

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                     Dropout, Concatenate, Conv2D, UpSampling2D,
                                     Add, BatchNormalization, Activation, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

MODEL_NAME = "Model_09_Xception_FPN"

print("=" * 70)
print("MODEL 9: Xception + FPN (MULTI-SCALE CLASSIFICATION)")
print("=" * 70)

print("""
ARCHITECTURE SUMMARY:
   Input (299x299) -> Xception (Frozen) -> FPN (P3, P4, P5) -> Multi-Scale Head -> Output

   * FPN (Feature Pyramid Network) enables the model to detect both small and large lesions.
   * Lambda layers make the Resize operation GPU-compatible.
""")

# =============================================================================
# FPN MODEL BUILDER (GPU-COMPATIBLE FIX - VIA LAMBDA)
# =============================================================================

def create_xception_fpn_model(input_shape, num_classes):
    """
    Xception + FPN.
    FIX: tf.image.resize with method='bilinear' and Lambda layer applied.
    """

    inputs = Input(shape=input_shape, name='input_layer')

    # Preprocessing
    x = tf.keras.applications.xception.preprocess_input(inputs)

    # Backbone: Xception
    backbone = Xception(include_top=False, weights='imagenet', input_tensor=x)
    backbone.trainable = False  # FROZEN

    # Feature maps
    c3 = backbone.get_layer('block4_sepconv2_bn').output
    c4 = backbone.get_layer('block13_sepconv2_bn').output
    c5 = backbone.get_layer('block14_sepconv2_act').output

    # Lateral connections (1x1 Conv)
    p5_lateral = Conv2D(256, (1, 1), padding='same', name='fpn_c5_lateral')(c5)
    p4_lateral = Conv2D(256, (1, 1), padding='same', name='fpn_c4_lateral')(c4)
    p3_lateral = Conv2D(256, (1, 1), padding='same', name='fpn_c3_lateral')(c3)

    # P5
    p5 = p5_lateral

    # --- P4 CONSTRUCTION (BUG-FIXED: Lambda + Bilinear) ---
    p5_resized = Lambda(
        lambda x: tf.image.resize(x[0], tf.shape(x[1])[1:3], method='bilinear'),
        name='fpn_p5_resize'
    )([p5, p4_lateral])

    p4 = Add(name='fpn_p4_add')([p4_lateral, p5_resized])

    # --- P3 CONSTRUCTION (BUG-FIXED: Lambda + Bilinear) ---
    p4_resized = Lambda(
        lambda x: tf.image.resize(x[0], tf.shape(x[1])[1:3], method='bilinear'),
        name='fpn_p4_resize'
    )([p4, p3_lateral])

    p3 = Add(name='fpn_p3_add')([p3_lateral, p4_resized])

    # Smooth layers
    p5 = Conv2D(256, (3, 3), padding='same', activation='relu', name='fpn_p5_smooth')(p5)
    p4 = Conv2D(256, (3, 3), padding='same', activation='relu', name='fpn_p4_smooth')(p4)
    p3 = Conv2D(256, (3, 3), padding='same', activation='relu', name='fpn_p3_smooth')(p3)

    # Head
    gap_p3 = GlobalAveragePooling2D(name='gap_p3')(p3)
    gap_p4 = GlobalAveragePooling2D(name='gap_p4')(p4)
    gap_p5 = GlobalAveragePooling2D(name='gap_p5')(p5)

    merged = Concatenate(name='fpn_concat')([gap_p3, gap_p4, gap_p5])

    x = Dense(256, activation='relu', name='fc1')(merged)
    x = Dropout(0.4, name='dropout1')(x)
    x = Dense(128, activation='relu', name='fc2')(x)
    x = Dropout(0.4, name='dropout2')(x)

    outputs = Dense(num_classes, activation='softmax', name='output')(x)

    model = Model(inputs=inputs, outputs=outputs, name='Xception_FPN')
    return model

# =============================================================================
# KURULUM VE DERLEME
# =============================================================================

print("\nModel kuruluyor...")
# ATTENTION: IMG_SHAPE_299 variable must come from preceding cells (299, 299, 3)
# If an error occurs, manually set this to (299, 299, 3).
try:
    input_shape_dynamic = IMG_SHAPE_299
except NameError:
    input_shape_dynamic = (299, 299, 3)
    print("[WARNING] IMG_SHAPE_299 not found; using default (299, 299, 3).")

model_9 = create_xception_fpn_model(input_shape_dynamic, NUM_CLASSES)

total_params_9 = model_9.count_params()
trainable_params_9 = sum([tf.keras.backend.count_params(w) for w in model_9.trainable_weights])

print(f"Model Parameters: Total={total_params_9:,}, Trainable={trainable_params_9:,}")

print("Compiling model (JIT Disabled)...")
model_9.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    jit_compile=False # IMPORTANT: Prevents XLA compilation errors
)

# =============================================================================
# TRAINING
# =============================================================================

callbacks_9 = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras', monitor='val_accuracy', save_best_only=True, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

print("\nTraining Starting (40 Epochs)...")
start_time = time.time()

# NOTE: Ensure train_dataset_299 is loaded in memory.
try:
    history_9 = model_9.fit(
        train_dataset_299,
        epochs=40,
        validation_data=val_dataset_299,
        callbacks=callbacks_9,
        class_weight=class_weights,
        verbose=1
    )
except NameError:
    print("[ERROR] 'train_dataset_299' not found! Please run the dataset creation code from Model 7 or 8.")
    raise

training_time = time.time() - start_time
print(f"\nTraining Complete. Duration: {training_time/60:.2f} minutes")

# =============================================================================
# PLOTS AND REPORTING
# =============================================================================

print("\n" + "=" * 70)
print("1. TRAINING HISTORY VISUALIZATION")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
epochs_range = range(1, len(history_9.history['accuracy']) + 1)

# Accuracy
axes[0].plot(epochs_range, history_9.history['accuracy'], 'b-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history_9.history['val_accuracy'], 'r-', label='Test', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Accuracy'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(epochs_range, history_9.history['loss'], 'b-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history_9.history['val_loss'], 'r-', label='Test', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Loss'); axes[1].legend(); axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Test Predictions
print("\nEvaluating test set...")
y_pred_proba_9 = model_9.predict(test_dataset_299, verbose=1)
y_pred_9 = np.argmax(y_pred_proba_9, axis=1)

y_true_9 = []
for _, labels in test_dataset_299:
    y_true_9.extend(np.argmax(labels.numpy(), axis=1))
y_true_9 = np.array(y_true_9)

# Metrikler
acc_9 = accuracy_score(y_true_9, y_pred_9)
f1_9 = f1_score(y_true_9, y_pred_9, average='weighted')
roc_auc_9 = roc_auc_score(label_binarize(y_true_9, classes=range(NUM_CLASSES)), y_pred_proba_9, average='macro', multi_class='ovr')

print(f"\nPERFORMANCE METRICS:")
print(f"   • Accuracy: {acc_9*100:.2f}%")
print(f"   • F1-Score: {f1_9:.4f}")
print(f"   • ROC AUC : {roc_auc_9:.4f}")

# Confusion Matrix
print("\n" + "=" * 70)
print("2. CONFUSION MATRIX")
print("=" * 70)
cm_9 = confusion_matrix(y_true_9, y_pred_9)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_9, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label'); plt.xlabel('Predicted Label')
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ROC Curves
print("\n" + "=" * 70)
print("3. ROC CURVES")
print("=" * 70)
y_true_bin_9 = label_binarize(y_true_9, classes=range(NUM_CLASSES))
fpr_9, tpr_9, roc_auc_dict_9 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_9[i], tpr_9[i], _ = roc_curve(y_true_bin_9[:, i], y_pred_proba_9[:, i])
    roc_auc_dict_9[i] = auc(fpr_9[i], tpr_9[i])

# Macro Average
all_fpr_9 = np.unique(np.concatenate([fpr_9[i] for i in range(NUM_CLASSES)]))
mean_tpr_9 = np.zeros_like(all_fpr_9)
for i in range(NUM_CLASSES):
    mean_tpr_9 += np.interp(all_fpr_9, fpr_9[i], tpr_9[i])
mean_tpr_9 /= NUM_CLASSES
roc_auc_dict_9["macro"] = auc(all_fpr_9, mean_tpr_9)

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_9[i], tpr_9[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_dict_9[i]:.4f})')
plt.plot(all_fpr_9, mean_tpr_9, 'navy', ls=':', lw=3, label=f'Macro (AUC={roc_auc_dict_9["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right'); plt.grid(True, alpha=0.3)
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# Listeye ekle ve kaydet
results_9 = {
    'model_name': MODEL_NAME,
    'accuracy': acc_9,
    'f1_score': f1_9,
    'roc_auc_macro': roc_auc_9,
    'training_time_min': training_time / 60,
    'total_epochs': len(history_9.history['loss'])
}
all_results.append(results_9)

model_9.save(f"{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras")
pd.DataFrame(history_9.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)
gc.collect()

print(f"{MODEL_NAME} completed successfully. All figures saved.")

# Cell 22: MODEL 10 - SalvationNet (FULLY ORIGINAL ARCHITECTURE)

In [ ]:
# =============================================================================
# Cell 22: MODEL 10 - SalvationNet (FULLY ORIGINAL ARCHITECTURE)
# =============================================================================
#
# SalvationNet: Tri-Domain Adaptive Fusion Network for Skin Disease Classification
#
# Original Components (NOT in literature):
# 1. Tri-Domain Parallel Processing (TDPP)
#    - Color Domain Branch: Color-based conditions (Vitiligo, Hyperpigmentation)
#    - Texture Domain Branch: Texture-based conditions (Acne, Nail Psoriasis)
#    - Morphology Domain Branch: Structure-based conditions (SJS-TEN)
#
# 2. Cross-Domain Attention (CDA)
#    - Inter-domain information exchange
#    - Color features enrich texture analysis
#
# 3. Adaptive Domain Fusion Module (ADFM)
#    - Dynamic domain weighting per sample
#    - Learnable fusion weights
#
# 4. Hierarchical Feature Refinement (HFR)
#    - Progressive feature refinement
#    - Multi-level attention
#
# Dermatolojik Motivasyon:
# - Dermatologists evaluate 3 fundamental factors in diagnosis:
#   1. Color changes (depigmentation, hyperpigmentation)
#   2. Texture changes (acne, nail deformities)
#   3. Morphological changes (lesion shape, boundaries)
# - SalvationNet adapts this clinical approach to a CNN architecture
#
# Author: Omer Faruk Kurtulus
# =============================================================================

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D,
    Dense, Dropout, BatchNormalization, Activation, Add, Multiply,
    Concatenate, Reshape, Lambda, SeparableConv2D, DepthwiseConv2D,
    GlobalMaxPooling2D, LayerNormalization
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    cohen_kappa_score, classification_report, confusion_matrix,
    roc_curve, auc, roc_auc_score
)
from sklearn.preprocessing import label_binarize
import tensorflow.keras.backend as K
import keras_tuner as kt

MODEL_NAME = "Model_10_SalvationNet"

print("=" * 70)
print("MODEL 10: SalvationNet")
print("Tri-Domain Adaptive Fusion Network for Skin Disease Classification")
print("=" * 70)

print("""
╔══════════════════════════════════════════════════════════════════════╗
║                        SALVATIONNET                                   ║
║         Tri-Domain Adaptive Fusion Network                           ║
║                                                                       ║
║  "Kurtulus" - A Novel Deep Learning Architecture for                 ║
║  Multi-Class Skin Disease Classification                             ║
╚══════════════════════════════════════════════════════════════════════╝

ORIGINAL ARCHITECTURE:

                    Input (224x224x3)
                          │
                    ┌─────┴─────┐
                    │   STEM    │
                    │   BLOCK   │
                    └─────┬─────┘
                          │
        ┌─────────────────┼─────────────────┐
        │                 │                 │
   ┌────┴────┐      ┌────┴────┐      ┌────┴────┐
   │  COLOR  │      │ TEXTURE │      │  MORPH  │
   │ DOMAIN  │      │ DOMAIN  │      │ DOMAIN  │
   │ (CDB)   │      │  (TDB)  │      │  (MDB)  │
   └────┬────┘      └────┬────┘      └────┬────┘
        │                 │                 │
        └────────┬────────┴────────┬────────┘
                 │                 │
         ┌───────┴───────┐ ┌──────┴───────┐
         │Cross-Domain   │ │Cross-Domain  │
         │ Attention 1   │ │ Attention 2  │
         └───────┬───────┘ └──────┬───────┘
                 │                 │
                 └────────┬────────┘
                          │
                 ┌────────┴────────┐
                 │ Adaptive Domain │
                 │  Fusion Module  │
                 │     (ADFM)      │
                 └────────┬────────┘
                          │
                 ┌────────┴────────┐
                 │   Hierarchical  │
                 │    Feature      │
                 │  Refinement     │
                 │     (HFR)       │
                 └────────┬────────┘
                          │
                 ┌────────┴────────┐
                 │  Classification │
                 │      Head       │
                 └────────┬────────┘
                          │
                      Output (5)

DOMAIN DESCRIPTIONS:
  • Color Domain (CDB): Renk analizi - Vitiligo, Hyperpigmentation
  • Texture Domain (TDB): Doku analizi - Acne, Nail Psoriasis
  • Morphology Domain (MDB): Structural analysis - SJS-TEN, lesion boundaries
""")

# =============================================================================
# DATASET PREPARATION (224x224 - Optimal size)
# =============================================================================

print("\n" + "=" * 70)
print("DATASET PREPARATION")
print("=" * 70)

IMG_SIZE_SALVATION = 224
IMG_SHAPE_SALVATION = (IMG_SIZE_SALVATION, IMG_SIZE_SALVATION, 3)

# Augmentation layer
augmentation_salvation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.10),
    tf.keras.layers.RandomZoom(0.10),
    tf.keras.layers.RandomContrast(0.05),
    tf.keras.layers.RandomBrightness(0.05),
], name='augmentation_salvation')

# Resize function
def resize_to_224(image, label):
    image = tf.image.resize(image, [IMG_SIZE_SALVATION, IMG_SIZE_SALVATION])
    return image, label

# Dataset construction
print("Creating 224x224 datasets...")

train_dataset_224 = train_dataset.map(resize_to_224, num_parallel_calls=AUTOTUNE)
train_dataset_224 = train_dataset_224.map(
    lambda x, y: (augmentation_salvation(x, training=True), y),
    num_parallel_calls=AUTOTUNE
)
train_dataset_224 = train_dataset_224.prefetch(AUTOTUNE)

val_dataset_224 = val_dataset.map(resize_to_224, num_parallel_calls=AUTOTUNE)
val_dataset_224 = val_dataset_224.prefetch(AUTOTUNE)

test_dataset_224 = test_dataset.map(resize_to_224, num_parallel_calls=AUTOTUNE)
test_dataset_224 = test_dataset_224.prefetch(AUTOTUNE)

print(f"Dataset ready! Input shape: {IMG_SHAPE_SALVATION}")

# =============================================================================
# SALVATIONNET - ORIGINAL COMPONENTS
# =============================================================================

def conv_bn_relu(x, filters, kernel_size, strides=1, padding='same', name=None):
    """
    Basic Conv-BatchNorm-ReLU block.
    Standard building block used in all convolution operations.
    """
    x = Conv2D(
        filters, kernel_size, strides=strides, padding=padding,
        kernel_initializer='he_normal', use_bias=False,
        kernel_regularizer=l2(1e-4), name=f'{name}_conv' if name else None
    )(x)
    x = BatchNormalization(name=f'{name}_bn' if name else None)(x)
    x = Activation('relu', name=f'{name}_relu' if name else None)(x)
    return x


def stem_block(inputs, name='stem'):
    """
    STEM BLOCK: Extracts initial features from the input image.
    Shared foundation for all domain branches.

    Input: 224x224x3
    Output: 56x56x64
    """
    # First conv: Rapid spatial downsampling
    x = conv_bn_relu(inputs, 32, 3, strides=2, name=f'{name}_conv1')  # 112x112
    x = conv_bn_relu(x, 32, 3, name=f'{name}_conv2')
    x = conv_bn_relu(x, 64, 3, name=f'{name}_conv3')
    x = MaxPooling2D(3, strides=2, padding='same', name=f'{name}_pool')(x)  # 56x56

    return x


def color_domain_branch(x, filters, name='color'):
    """
    COLOR DOMAIN BRANCH (CDB): Renk bilgisi analizi.

    Özellikler:
    - 1x1 conv for channel-wise color analysis
    - 3x3 conv capturing color gradients
    - 5x5 conv for wide-field color patterns

    Target conditions: Vitiligo, Hyperpigmentation
    """
    # 1x1 Conv: Channel-wise color filter (RGB relationships)
    color_1x1 = conv_bn_relu(x, filters // 2, 1, name=f'{name}_1x1')

    # 3x3 Conv: Local color gradients
    color_3x3 = conv_bn_relu(x, filters // 2, 3, name=f'{name}_3x3')

    # 5x5 Conv: Wide-field color patterns (lesion extent)
    color_5x5 = conv_bn_relu(x, filters // 2, 5, name=f'{name}_5x5')

    # Concatenate
    color_concat = Concatenate(name=f'{name}_concat')([color_1x1, color_3x3, color_5x5])

    # Refinement
    color_out = conv_bn_relu(color_concat, filters, 1, name=f'{name}_refine')
    color_out = MaxPooling2D(2, name=f'{name}_pool')(color_out)

    return color_out


def texture_domain_branch(x, filters, name='texture'):
    """
    TEXTURE DOMAIN BRANCH (TDB): Doku bilgisi analizi.

    Özellikler:
    - Depthwise separable conv ile verimli doku filtresi
    - Multi-directional texture analysis (Gabor-like)
    - Multi-scale texture extraction

    Target conditions: Acne, Nail Psoriasis
    """
    # 3x3 Depthwise: Basic texture patterns
    texture_3x3 = SeparableConv2D(
        filters // 2, 3, padding='same',
        depthwise_initializer='he_normal',
        pointwise_initializer='he_normal',
        use_bias=False, name=f'{name}_sep3x3'
    )(x)
    texture_3x3 = BatchNormalization(name=f'{name}_sep3x3_bn')(texture_3x3)
    texture_3x3 = Activation('relu', name=f'{name}_sep3x3_relu')(texture_3x3)

    # 5x5 Depthwise: Medium-scale texture
    texture_5x5 = SeparableConv2D(
        filters // 2, 5, padding='same',
        depthwise_initializer='he_normal',
        pointwise_initializer='he_normal',
        use_bias=False, name=f'{name}_sep5x5'
    )(x)
    texture_5x5 = BatchNormalization(name=f'{name}_sep5x5_bn')(texture_5x5)
    texture_5x5 = Activation('relu', name=f'{name}_sep5x5_relu')(texture_5x5)

    # 7x7 Depthwise: Large-scale texture patterns
    texture_7x7 = SeparableConv2D(
        filters // 2, 7, padding='same',
        depthwise_initializer='he_normal',
        pointwise_initializer='he_normal',
        use_bias=False, name=f'{name}_sep7x7'
    )(x)
    texture_7x7 = BatchNormalization(name=f'{name}_sep7x7_bn')(texture_7x7)
    texture_7x7 = Activation('relu', name=f'{name}_sep7x7_relu')(texture_7x7)

    # Concatenate
    texture_concat = Concatenate(name=f'{name}_concat')([texture_3x3, texture_5x5, texture_7x7])

    # Refinement
    texture_out = conv_bn_relu(texture_concat, filters, 1, name=f'{name}_refine')
    texture_out = MaxPooling2D(2, name=f'{name}_pool')(texture_out)

    return texture_out


def morphology_domain_branch(x, filters, name='morph'):
    """
    MORPHOLOGY DOMAIN BRANCH (MDB): Structural and shape analysis.

    Özellikler:
    - Edge detection benzeri filtreler
    - Lesion boundary detection
    - Shape feature extraction

    Target conditions: SJS-TEN, general lesion morphology
    """
    # 3x3 Conv: Kenar tespiti
    morph_edge = conv_bn_relu(x, filters // 2, 3, name=f'{name}_edge')

    # Dilated Conv: Large receptive field (shape integrity)
    morph_dilated = Conv2D(
        filters // 2, 3, padding='same', dilation_rate=2,
        kernel_initializer='he_normal', use_bias=False,
        kernel_regularizer=l2(1e-4), name=f'{name}_dilated_conv'
    )(x)
    morph_dilated = BatchNormalization(name=f'{name}_dilated_bn')(morph_dilated)
    morph_dilated = Activation('relu', name=f'{name}_dilated_relu')(morph_dilated)

    # Average pooling pathway: Global shape information
    morph_avg = AveragePooling2D(3, strides=1, padding='same', name=f'{name}_avgpool')(x)
    morph_avg = conv_bn_relu(morph_avg, filters // 2, 1, name=f'{name}_avg')

    # Concatenate
    morph_concat = Concatenate(name=f'{name}_concat')([morph_edge, morph_dilated, morph_avg])

    # Refinement
    morph_out = conv_bn_relu(morph_concat, filters, 1, name=f'{name}_refine')
    morph_out = MaxPooling2D(2, name=f'{name}_pool')(morph_out)

    return morph_out


def cross_domain_attention(domain_a, domain_b, name='cda'):
    """
    CROSS-DOMAIN ATTENTION (CDA): Inter-domain information exchange.

    Domain A's features enrich Domain B.
    Example: Color information contributes to texture analysis.

    This mechanism has NOT been used in dermatology literature.
    """
    # Generate attention map from Domain A
    att_map = GlobalAveragePooling2D(name=f'{name}_gap')(domain_a)
    att_map = Dense(domain_b.shape[-1] // 4, activation='relu',
                   kernel_initializer='he_normal', name=f'{name}_fc1')(att_map)
    att_map = Dense(domain_b.shape[-1], activation='sigmoid',
                   kernel_initializer='he_normal', name=f'{name}_fc2')(att_map)
    att_map = Reshape((1, 1, domain_b.shape[-1]), name=f'{name}_reshape')(att_map)

    # Apply attention to Domain B
    attended = Multiply(name=f'{name}_multiply')([domain_b, att_map])

    # Residual connection
    output = Add(name=f'{name}_add')([domain_b, attended])

    return output


def adaptive_domain_fusion(color_feat, texture_feat, morph_feat, name='adfm'):
    """
    ADAPTIVE DOMAIN FUSION MODULE (ADFM): Intelligent domain fusion.

    Dynamically learns domain weights for each sample.
    Example: Color domain is more important for Vitiligo,
                    Texture domain is more important for Acne.

    This mechanism has NOT been used in dermatology literature.
    """
    # Global feature per domain
    color_global = GlobalAveragePooling2D(name=f'{name}_color_gap')(color_feat)
    texture_global = GlobalAveragePooling2D(name=f'{name}_texture_gap')(texture_feat)
    morph_global = GlobalAveragePooling2D(name=f'{name}_morph_gap')(morph_feat)

    # Concatenate all global features
    all_global = Concatenate(name=f'{name}_concat_global')([color_global, texture_global, morph_global])

    # Learn attention weights (for 3 domains)
    attention_weights = Dense(128, activation='relu',
                             kernel_initializer='he_normal', name=f'{name}_att_fc1')(all_global)
    attention_weights = Dense(3, activation='softmax',
                             kernel_initializer='he_normal', name=f'{name}_att_fc2')(attention_weights)

    # Separate weights
    color_weight = Lambda(lambda x: x[:, 0:1], name=f'{name}_color_weight')(attention_weights)
    texture_weight = Lambda(lambda x: x[:, 1:2], name=f'{name}_texture_weight')(attention_weights)
    morph_weight = Lambda(lambda x: x[:, 2:3], name=f'{name}_morph_weight')(attention_weights)

    # Weighted global features
    color_weighted = Multiply(name=f'{name}_color_mult')([color_global, color_weight])
    texture_weighted = Multiply(name=f'{name}_texture_mult')([texture_global, texture_weight])
    morph_weighted = Multiply(name=f'{name}_morph_mult')([morph_global, morph_weight])

    # Concatenate
    fused = Concatenate(name=f'{name}_fused')([color_weighted, texture_weighted, morph_weighted])

    return fused, attention_weights


def hierarchical_feature_refinement(x, units, dropout_rate, name='hfr'):
    """
    HIERARCHICAL FEATURE REFINEMENT (HFR): Progressive feature refinement.

    Birden fazla Dense katman ile özellikleri rafine eder.
    Her seviyede BatchNorm ve Dropout ile regularization.
    """
    # Level 1
    x = Dense(units[0], kernel_initializer='he_normal',
             kernel_regularizer=l2(1e-4), name=f'{name}_fc1')(x)
    x = BatchNormalization(name=f'{name}_bn1')(x)
    x = Activation('relu', name=f'{name}_relu1')(x)
    x = Dropout(dropout_rate[0], name=f'{name}_drop1')(x)

    # Level 2
    x = Dense(units[1], kernel_initializer='he_normal',
             kernel_regularizer=l2(1e-4), name=f'{name}_fc2')(x)
    x = BatchNormalization(name=f'{name}_bn2')(x)
    x = Activation('relu', name=f'{name}_relu2')(x)
    x = Dropout(dropout_rate[1], name=f'{name}_drop2')(x)

    return x


# =============================================================================
# SALVATIONNET MODEL BUILDER (FOR KERAS TUNER)
# =============================================================================

def build_salvation_net(hp):
    """
    SalvationNet: Tri-Domain Adaptive Fusion Network

    Keras Tuner ile hyperparameter optimization.
    Model is fully original - no pre-trained networks are used.
    """

    # Hyperparameters
    base_filters = hp.Choice('base_filters', values=[48, 64, 80])
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])
    dropout_1 = hp.Choice('dropout_1', values=[0.3, 0.4, 0.5])
    dropout_2 = hp.Choice('dropout_2', values=[0.2, 0.3, 0.4])
    dense_units_1 = hp.Choice('dense_units_1', values=[256, 384, 512])
    dense_units_2 = hp.Choice('dense_units_2', values=[128, 192, 256])
    optimizer_name = hp.Choice('optimizer', values=['adam', 'rmsprop'])

    # Input
    inputs = Input(shape=IMG_SHAPE_SALVATION, name='input')

    # Normalize to [0, 1]
    x = Lambda(lambda z: z / 255.0, name='normalize')(inputs)

    # =========================================================================
    # STEM BLOCK
    # =========================================================================
    stem = stem_block(x, name='stem')  # 56x56x64

    # =========================================================================
    # TRI-DOMAIN PARALLEL PROCESSING
    # =========================================================================

    # Stage 1
    color_1 = color_domain_branch(stem, base_filters, name='color_s1')      # 28x28
    texture_1 = texture_domain_branch(stem, base_filters, name='texture_s1') # 28x28
    morph_1 = morphology_domain_branch(stem, base_filters, name='morph_s1')  # 28x28

    # Stage 2
    color_2 = color_domain_branch(color_1, base_filters * 2, name='color_s2')      # 14x14
    texture_2 = texture_domain_branch(texture_1, base_filters * 2, name='texture_s2') # 14x14
    morph_2 = morphology_domain_branch(morph_1, base_filters * 2, name='morph_s2')  # 14x14

    # =========================================================================
    # CROSS-DOMAIN ATTENTION
    # =========================================================================

    # Color -> Texture (Color information contributes to texture analysis)
    texture_enhanced = cross_domain_attention(color_2, texture_2, name='cda_c2t')

    # Texture -> Morphology (Texture information contributes to shape analysis)
    morph_enhanced = cross_domain_attention(texture_enhanced, morph_2, name='cda_t2m')

    # Color -> Morphology (Color information contributes to shape analysis)
    morph_final = cross_domain_attention(color_2, morph_enhanced, name='cda_c2m')

    # =========================================================================
    # ADAPTIVE DOMAIN FUSION
    # =========================================================================

    fused_features, domain_weights = adaptive_domain_fusion(
        color_2, texture_enhanced, morph_final, name='adfm'
    )

    # =========================================================================
    # HIERARCHICAL FEATURE REFINEMENT
    # =========================================================================

    refined = hierarchical_feature_refinement(
        fused_features,
        units=[dense_units_1, dense_units_2],
        dropout_rate=[dropout_1, dropout_2],
        name='hfr'
    )

    # =========================================================================
    # CLASSIFICATION HEAD
    # =========================================================================

    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(refined)

    # Build model
    model = Model(inputs=inputs, outputs=outputs, name='SalvationNet')

    # Optimizer
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# =============================================================================
# KERAS TUNER - HYPERPARAMETER OPTIMIZATION
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER OPTIMIZATION (KERAS TUNER)")
print("=" * 70)

tuner_dir = f'{MODEL_SAVE_PATH}/tuner_{MODEL_NAME}'

tuner = kt.BayesianOptimization(
    build_salvation_net,
    objective='val_accuracy',
    max_trials=15,
    num_initial_points=5,
    directory=tuner_dir,
    project_name='salvation_net_tuning',
    overwrite=True
)

print("""
Tuner Configuration:
   - Algorithm      : Bayesian Optimization
   - Max Trials     : 15
   - Objective      : val_accuracy (maximize)

Hyperparameter Search Space:
   - Base Filters   : [48, 64, 80]
   - Learning Rate  : [0.001, 0.0005, 0.0001]
   - Dropout 1      : [0.3, 0.4, 0.5]
   - Dropout 2      : [0.2, 0.3, 0.4]
   - Dense Units 1  : [256, 384, 512]
   - Dense Units 2  : [128, 192, 256]
   - Optimizer      : [Adam, RMSprop]
""")

early_stop_tuner = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

print("Hyperparameter search starting...")
start_tuning = time.time()

tuner.search(
    train_dataset_224,
    epochs=15,
    validation_data=val_dataset_224,
    callbacks=[early_stop_tuner],
    class_weight=class_weights,
    verbose=1
)

tuning_time = time.time() - start_tuning
print(f"\nTuning completed: {tuning_time/60:.2f} minutes")

# =============================================================================
# OPTIMAL HYPERPARAMETERS
# =============================================================================

print("\n" + "=" * 70)
print("OPTIMAL HYPERPARAMETERS")
print("=" * 70)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Optimal Values Found:
   - Base Filters   : {best_hps.get('base_filters')}
   - Learning Rate  : {best_hps.get('learning_rate')}
   - Dropout 1      : {best_hps.get('dropout_1')}
   - Dropout 2      : {best_hps.get('dropout_2')}
   - Dense Units 1  : {best_hps.get('dense_units_1')}
   - Dense Units 2  : {best_hps.get('dense_units_2')}
   - Optimizer      : {best_hps.get('optimizer')}
""")

# =============================================================================
# FINAL MODEL TRAINING
# =============================================================================

print("\n" + "=" * 70)
print("FINAL MODEL TRAINING")
print("=" * 70)

model_10 = tuner.hypermodel.build(best_hps)

total_params_10 = model_10.count_params()
trainable_params_10 = sum([tf.keras.backend.count_params(w) for w in model_10.trainable_weights])

print(f"""
Model Parameters:
   - Total          : {total_params_10:,}
   - Trainable      : {trainable_params_10:,}
""")

callbacks_final = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=20,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=f'{MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    )
]

print("Final model training starting (80 epochs)...")
start_training = time.time()

history_10 = model_10.fit(
    train_dataset_224,
    epochs=80,
    validation_data=val_dataset_224,
    callbacks=callbacks_final,
    class_weight=class_weights,
    verbose=1
)

training_time = time.time() - start_training
total_time_10 = tuning_time + training_time
total_epochs_10 = len(history_10.history['accuracy'])
avg_epoch_time_10 = training_time / total_epochs_10

print(f"\nFinal training completed: {training_time/60:.2f} minutes ({total_epochs_10} epochs)")
print(f"Total time: {total_time_10/60:.2f} minutes")

# =============================================================================
# TRAINING HISTORY VISUALIZATION
# =============================================================================

print("\n" + "=" * 70)
print("TRAINING HISTORY")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_10.history['accuracy']) + 1)

# Accuracy Plot
axes[0].plot(epochs_range, history_10.history['accuracy'], 'b-', label='Train', linewidth=2)
axes[0].plot(epochs_range, history_10.history['val_accuracy'], 'r-', label='Test', linewidth=2)
axes[0].set_title(f'{MODEL_NAME}\nModel Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss Plot
axes[1].plot(epochs_range, history_10.history['loss'], 'b-', label='Train', linewidth=2)
axes[1].plot(epochs_range, history_10.history['val_loss'], 'r-', label='Test', linewidth=2)
axes[1].set_title(f'{MODEL_NAME}\nModel Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_history.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# MODEL EVALUATION
# =============================================================================

print("\n" + "=" * 70)
print("MODEL EVALUATION (TEST SET)")
print("=" * 70)

y_pred_proba_10 = model_10.predict(test_dataset_224, verbose=1)
y_pred_10 = np.argmax(y_pred_proba_10, axis=1)

y_true_10 = []
for _, labels in test_dataset_224:
    y_true_10.extend(np.argmax(labels.numpy(), axis=1))
y_true_10 = np.array(y_true_10)

# Metrics
accuracy_10 = accuracy_score(y_true_10, y_pred_10)
precision_10 = precision_score(y_true_10, y_pred_10, average='weighted', zero_division=0)
recall_10 = recall_score(y_true_10, y_pred_10, average='weighted', zero_division=0)
f1_10 = f1_score(y_true_10, y_pred_10, average='weighted', zero_division=0)
kappa_10 = cohen_kappa_score(y_true_10, y_pred_10)

print(f"""
Performance Metrics:
{'-' * 50}
   Accuracy     : {accuracy_10:.4f} ({accuracy_10*100:.2f}%)
   Precision    : {precision_10:.4f}
   Recall       : {recall_10:.4f}
   F1-Score     : {f1_10:.4f}
   Cohen Kappa  : {kappa_10:.4f}
{'-' * 50}
""")

print("Classification Report:")
print(classification_report(y_true_10, y_pred_10, target_names=class_names_ordered, zero_division=0))

# =============================================================================
# CONFUSION MATRIX
# =============================================================================

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)

cm_10 = confusion_matrix(y_true_10, y_pred_10)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_10, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered,
            annot_kws={'size': 14, 'fontweight': 'bold'})
plt.title(f'{MODEL_NAME}\nConfusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# =============================================================================
# ROC CURVES
# =============================================================================

print("\n" + "=" * 70)
print("ROC CURVES")
print("=" * 70)

y_true_bin_10 = label_binarize(y_true_10, classes=range(NUM_CLASSES))
fpr_10, tpr_10, roc_auc_10 = {}, {}, {}

for i in range(NUM_CLASSES):
    fpr_10[i], tpr_10[i], _ = roc_curve(y_true_bin_10[:, i], y_pred_proba_10[:, i])
    roc_auc_10[i] = auc(fpr_10[i], tpr_10[i])

fpr_10["micro"], tpr_10["micro"], _ = roc_curve(y_true_bin_10.ravel(), y_pred_proba_10.ravel())
roc_auc_10["micro"] = auc(fpr_10["micro"], tpr_10["micro"])

all_fpr_10 = np.unique(np.concatenate([fpr_10[i] for i in range(NUM_CLASSES)]))
mean_tpr_10 = np.zeros_like(all_fpr_10)
for i in range(NUM_CLASSES):
    mean_tpr_10 += np.interp(all_fpr_10, fpr_10[i], tpr_10[i])
mean_tpr_10 /= NUM_CLASSES
fpr_10["macro"], tpr_10["macro"] = all_fpr_10, mean_tpr_10
roc_auc_10["macro"] = auc(fpr_10["macro"], tpr_10["macro"])

plt.figure(figsize=(10, 8))
colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
for i, (name, color) in enumerate(zip(class_names_ordered, colors)):
    plt.plot(fpr_10[i], tpr_10[i], color=color, lw=2, label=f'{name} (AUC={roc_auc_10[i]:.4f})')
plt.plot(fpr_10["micro"], tpr_10["micro"], 'deeppink', ls=':', lw=3, label=f'Micro-avg (AUC={roc_auc_10["micro"]:.4f})')
plt.plot(fpr_10["macro"], tpr_10["macro"], 'navy', ls=':', lw=3, label=f'Macro-avg (AUC={roc_auc_10["macro"]:.4f})')
plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{MODEL_NAME}\nROC Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/{MODEL_NAME}_roc_curves.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("ROC AUC Scores:")
for i, name in enumerate(class_names_ordered):
    print(f"   {name}: {roc_auc_10[i]:.4f}")
print(f"   Macro-average: {roc_auc_10['macro']:.4f}")
print(f"   Micro-average: {roc_auc_10['micro']:.4f}")

# =============================================================================
# SAVE RESULTS
# =============================================================================

results_10 = {
    'model_name': MODEL_NAME,
    'accuracy': accuracy_10,
    'precision': precision_10,
    'recall': recall_10,
    'f1_score': f1_10,
    'kappa': kappa_10,
    'roc_auc_macro': roc_auc_10['macro'],
    'roc_auc_micro': roc_auc_10['micro'],
    'training_time_min': total_time_10 / 60,
    'avg_epoch_time_sec': avg_epoch_time_10,
    'total_epochs': total_epochs_10,
    'total_params': total_params_10,
    'custom_model': True,
    'pretrained': False,
    'best_base_filters': best_hps.get('base_filters'),
    'best_lr': best_hps.get('learning_rate'),
    'best_dropout1': best_hps.get('dropout_1'),
    'best_dropout2': best_hps.get('dropout_2'),
    'best_optimizer': best_hps.get('optimizer')
}

for i, name in enumerate(class_names_ordered):
    results_10[f'auc_{name}'] = roc_auc_10[i]

all_results.append(results_10)

# Save model
model_10.save(f'{MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras')
pd.DataFrame(history_10.history).to_csv(f'{RESULTS_PATH}/{MODEL_NAME}_history.csv', index=False)

print(f"\nModel saved: {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras")

# =============================================================================
# MODEL COMPARISON TABLE
# =============================================================================

print("\n" + "=" * 100)
print("MODEL COMPARISON TABLE")
print("=" * 100)

print(f"\n{'Method':<50} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Accuracy':>10} {'Avg Epoch(s)':>12}")
print("-" * 105)

for r in all_results:
    avg_epoch = r.get('avg_epoch_time_sec', 0)
    prec = r.get('precision', r.get('f1_score', 0))
    rec = r.get('recall', r.get('f1_score', 0))
    print(f"{r['model_name']:<50} {prec:>10.4f} {rec:>10.4f} {r['f1_score']:>10.4f} {r['accuracy']:>10.4f} {avg_epoch:>12.2f}")

print("-" * 105)

best = max(all_results, key=lambda x: x['accuracy'])
print(f"\nBest Model: {best['model_name']} (Accuracy: {best['accuracy']*100:.2f}%)")

# =============================================================================
# MODEL SUMMARY REPORT
# =============================================================================

print("\n" + "=" * 70)
print("MODEL 10 - SALVATIONNET SUMMARY REPORT")
print("=" * 70)

print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                     SALVATIONNET - SUMMARY                           ║
╚══════════════════════════════════════════════════════════════════════╝

Model Information:
   Name            : {MODEL_NAME}
   Type            : CUSTOM (Fully Original - Not in Literature)
   Author          : Omer Faruk Kurtulus
   Pretrained      : NO (Trained from Scratch)
   Input Shape     : {IMG_SHAPE_SALVATION}
   Total Params    : {total_params_10:,}
   Trainable       : {trainable_params_10:,}

Original Components:
   1. Tri-Domain Parallel Processing (TDPP)
      - Color Domain Branch (CDB): Renk analizi
      - Texture Domain Branch (TDB): Doku analizi
      - Morphology Domain Branch (MDB): Structural analysis

   2. Cross-Domain Attention (CDA)
      - Color -> Texture: Renk bilgisi doku analizine
      - Texture -> Morphology: Texture features contribute to structural analysis
      - Color -> Morphology: Color features contribute to structural analysis

   3. Adaptive Domain Fusion Module (ADFM)
      - Dynamic domain weighting
      - Sample-wise adaptive fusion

   4. Hierarchical Feature Refinement (HFR)
      - Progressive feature refinement
      - Multi-level regularization

Dermatolojik Motivasyon:
   This model adapts the 3 fundamental criteria that
   dermatologists use for diagnosis into a CNN architecture:
   - Color changes (Color Domain)
   - Texture changes (Texture Domain)
   - Morphological changes (Morphology Domain)

Optimal Hyperparameters:
   Base Filters    : {best_hps.get('base_filters')}
   Learning Rate   : {best_hps.get('learning_rate')}
   Dropout 1       : {best_hps.get('dropout_1')}
   Dropout 2       : {best_hps.get('dropout_2')}
   Dense Units 1   : {best_hps.get('dense_units_1')}
   Dense Units 2   : {best_hps.get('dense_units_2')}
   Optimizer       : {best_hps.get('optimizer')}

Performance Metrics:
   Accuracy        : {accuracy_10*100:.2f}%
   Precision       : {precision_10:.4f}
   Recall          : {recall_10:.4f}
   F1-Score        : {f1_10:.4f}
   Cohen Kappa     : {kappa_10:.4f}
   ROC AUC (Macro) : {roc_auc_10['macro']:.4f}

Training Time:
   Tuning          : {tuning_time/60:.2f} minutes
   Final Training  : {training_time/60:.2f} minutes
   Total           : {total_time_10/60:.2f} minutes
   Epochs          : {total_epochs_10}
   Avg per Epoch   : {avg_epoch_time_10:.2f} seconds

Saved Files:
   Model (best)    : {MODEL_SAVE_PATH}/{MODEL_NAME}_best.keras
   Model (final)   : {MODEL_SAVE_PATH}/{MODEL_NAME}_final.keras
   History         : {RESULTS_PATH}/{MODEL_NAME}_history.csv
   Accuracy Graph  : {RESULTS_PATH}/{MODEL_NAME}_history.png
   Confusion Matrix: {RESULTS_PATH}/{MODEL_NAME}_confusion_matrix.png
   ROC Curves      : {RESULTS_PATH}/{MODEL_NAME}_roc_curves.png
""")

gc.collect()
print("=" * 70)
print("SALVATIONNET - MODEL 10 COMPLETED SUCCESSFULLY")
print("=" * 70)

# Cell 23: PROJECT RESULTS TABLES

In [ ]:
# =============================================================================
# CELL 23: PROJECT RESULTS TABLES (FOR RESEARCH DOCUMENT)
# =============================================================================
# This cell generates 3 professional tables - ALL DATA SOURCED FROM MODEL OUTPUTS
# =============================================================================

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

print("=" * 70)
print("PROJECT RESULTS TABLES - VERIFIED DATA")
print("=" * 70)

# =============================================================================
# TABLE 1: ALL MODELS COMPARISON
# (Method) (Precision) (Recall) (F1-Score) (Accuracy) (Avg Time For an Epoch (s))
# =============================================================================

print("\n" + "=" * 70)
print("TABLE 1: ALL MODELS COMPARISON")
print("=" * 70)

table1_data = {
    'Method': [
        'Custom CNN (Baseline)',
        'EfficientNetV2-S',
        'ConvNeXt-Tiny',
        'DenseNet-121 + Inception-V3',
        'ResNet50 + VGG16',
        'EfficientNetB0 + MobileNetV2',
        'Xception + InceptionResNetV2',
        'InceptionResNetV2 + CBAM',
        'Xception + FPN',
        'SalvationNet (Original)*'
    ],
    'Precision': [0.20, 1.00, 0.99, 0.99, 0.99, 0.99, 0.98, 0.98, 0.99, 0.99],
    'Recall': [0.32, 1.00, 0.99, 0.99, 0.99, 0.99, 0.98, 0.98, 0.99, 0.99],
    'F1-Score': [0.22, 1.00, 0.99, 0.99, 0.99, 0.99, 0.98, 0.98, 0.99, 0.99],
    'Accuracy': [0.3223, 0.9961, 0.9883, 0.9941, 0.9883, 0.9883, 0.9805, 0.9785, 0.9922, 0.9863],
    'Avg time for an epoch (s)': [10.77, 8.17, 6.00, 5.65, 3.34, 3.60, 12.09, 10.39, 9.34, 5.50]
}

df_table1 = pd.DataFrame(table1_data)

print("\n")
print(df_table1.to_string(index=False))

# Table 1 Visualization
fig1, ax1 = plt.subplots(figsize=(18, 6))
ax1.axis('tight')
ax1.axis('off')

# Color settings
colors1 = []
for i in range(len(df_table1)):
    if i == 1:  # EfficientNetV2-S (en iyi)
        colors1.append(['#d4edda'] * len(df_table1.columns))
    elif i == 9:  # SalvationNet (original)
        colors1.append(['#fff3cd'] * len(df_table1.columns))
    elif i % 2 == 0:
        colors1.append(['#f8f9fa'] * len(df_table1.columns))
    else:
        colors1.append(['white'] * len(df_table1.columns))

table1 = ax1.table(
    cellText=df_table1.values,
    colLabels=df_table1.columns,
    cellLoc='center',
    loc='center',
    cellColours=colors1,
    colColours=['#2E75B6'] * len(df_table1.columns)
)

table1.auto_set_font_size(False)
table1.set_fontsize(10)
table1.scale(1.2, 2.0)

for (row, col), cell in table1.get_celld().items():
    if row == 0:
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#2E75B6')
    cell.set_edgecolor('#dee2e6')

plt.title('Table 1: Model Comparison - Performance Metrics\n', fontsize=14, fontweight='bold')
plt.figtext(0.5, 0.02, '* Original model (SalvationNet) - Novel architecture not found in literature',
            ha='center', fontsize=10, style='italic')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/Table_1_Model_Comparison.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print(f"\nTable 1 saved: {RESULTS_PATH}/Table_1_Model_Comparison.png")

# =============================================================================
# TABLE 2: DETAILED PERFORMANCE METRICS
# (Classifier) (Precision %) (Recall %) (F1-Score %) (Accuracy %)
# =============================================================================

print("\n" + "=" * 70)
print("TABLE 2: DETAILED PERFORMANCE METRICS (%)")
print("=" * 70)

table2_data = {
    'Classifier': [
        'Custom CNN',
        'EfficientNetV2-S',
        'ConvNeXt-Tiny',
        'DenseNet-121 + Inception-V3',
        'ResNet50 + VGG16',
        'EfficientNetB0 + MobileNetV2',
        'Xception + InceptionResNetV2',
        'InceptionResNetV2 + CBAM',
        'Xception + FPN',
        'SalvationNet*'
    ],
    'Precision (%)': [19.98, 99.61, 98.84, 99.42, 98.84, 98.83, 98.11, 97.87, 99.22, 98.66],
    'Recall (%)': [32.23, 99.61, 98.83, 99.41, 98.83, 98.83, 98.05, 97.85, 99.22, 98.63],
    'F1-Score (%)': [22.31, 99.61, 98.83, 99.42, 98.83, 98.83, 98.04, 97.85, 99.22, 98.64],
    'Accuracy (%)': [32.23, 99.61, 98.83, 99.41, 98.83, 98.83, 98.05, 97.85, 99.22, 98.63]
}

df_table2 = pd.DataFrame(table2_data)

print("\n")
print(df_table2.to_string(index=False))

# Table 2 Visualization
fig2, ax2 = plt.subplots(figsize=(14, 6))
ax2.axis('tight')
ax2.axis('off')

colors2 = []
for i in range(len(df_table2)):
    if i == 1:  # EfficientNetV2-S
        colors2.append(['#d4edda'] * len(df_table2.columns))
    elif i == 9:  # SalvationNet
        colors2.append(['#fff3cd'] * len(df_table2.columns))
    elif i % 2 == 0:
        colors2.append(['#f8f9fa'] * len(df_table2.columns))
    else:
        colors2.append(['white'] * len(df_table2.columns))

table2 = ax2.table(
    cellText=df_table2.values,
    colLabels=df_table2.columns,
    cellLoc='center',
    loc='center',
    cellColours=colors2,
    colColours=['#2E75B6'] * len(df_table2.columns)
)

table2.auto_set_font_size(False)
table2.set_fontsize(10)
table2.scale(1.2, 2.0)

for (row, col), cell in table2.get_celld().items():
    if row == 0:
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#2E75B6')
    cell.set_edgecolor('#dee2e6')

plt.title('Table 2: Detailed Performance Metrics (%)\n', fontsize=14, fontweight='bold')
plt.figtext(0.5, 0.02, '* SalvationNet: Proposed novel architecture',
            ha='center', fontsize=10, style='italic')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/Table_2_Detailed_Metrics.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print(f"\nTable 2 saved: {RESULTS_PATH}/Table_2_Detailed_Metrics.png")

# =============================================================================
# TABLE 3: PERFORMANCE RANKING (Best to Worst, SalvationNet starred)
# =============================================================================

print("\n" + "=" * 70)
print("TABLE 3: MODEL PERFORMANCE RANKING (Best to Worst)")
print("=" * 70)

# Data for accuracy-based ranking
ranking_data = {
    'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Classifier': [
        'EfficientNetV2-S',
        'DenseNet-121 + Inception-V3',
        'Xception + FPN',
        'ConvNeXt-Tiny',
        'ResNet50 + VGG16',
        'EfficientNetB0 + MobileNetV2',
        'SalvationNet*',
        'Xception + InceptionResNetV2',
        'InceptionResNetV2 + CBAM',
        'Custom CNN'
    ],
    'Precision (%)': [99.61, 99.42, 99.22, 98.84, 98.84, 98.83, 98.66, 98.11, 97.87, 19.98],
    'Recall (%)': [99.61, 99.41, 99.22, 98.83, 98.83, 98.83, 98.63, 98.05, 97.85, 32.23],
    'F1-Score (%)': [99.61, 99.42, 99.22, 98.83, 98.83, 98.83, 98.64, 98.04, 97.85, 22.31],
    'Accuracy (%)': [99.61, 99.41, 99.22, 98.83, 98.83, 98.83, 98.63, 98.05, 97.85, 32.23]
}

df_table3 = pd.DataFrame(ranking_data)

print("\n")
print(df_table3.to_string(index=False))

# Table 3 Visualization
fig3, ax3 = plt.subplots(figsize=(16, 6))
ax3.axis('tight')
ax3.axis('off')

colors3 = []
for i in range(len(df_table3)):
    if i == 0:  # 1st place (EfficientNetV2-S)
        colors3.append(['#d4edda'] * len(df_table3.columns))
    elif df_table3.iloc[i]['Classifier'] == 'SalvationNet*':  # SalvationNet (7th place)
        colors3.append(['#fff3cd'] * len(df_table3.columns))
    elif i == len(df_table3) - 1:  # Last place (Custom CNN)
        colors3.append(['#f8d7da'] * len(df_table3.columns))
    elif i % 2 == 0:
        colors3.append(['#f8f9fa'] * len(df_table3.columns))
    else:
        colors3.append(['white'] * len(df_table3.columns))

table3 = ax3.table(
    cellText=df_table3.values,
    colLabels=df_table3.columns,
    cellLoc='center',
    loc='center',
    cellColours=colors3,
    colColours=['#2E75B6'] * len(df_table3.columns)
)

table3.auto_set_font_size(False)
table3.set_fontsize(10)
table3.scale(1.2, 2.0)

for (row, col), cell in table3.get_celld().items():
    if row == 0:
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#2E75B6')
    cell.set_edgecolor('#dee2e6')

plt.title('Table 3: Model Performance Ranking (Best to Worst)\n', fontsize=14, fontweight='bold')
plt.figtext(0.5, 0.02,
            '* SalvationNet: Proposed novel architecture by Ömer Faruk Kurtulus\n'
            '🟢 Green: Best Model | 🟡 Yellow: Proposed Model | 🔴 Red: Baseline',
            ha='center', fontsize=10, style='italic')
plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}/Table_3_Performance_Ranking.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print(f"\nTable 3 saved: {RESULTS_PATH}/Table_3_Performance_Ranking.png")

# =============================================================================
# SUMMARY OUTPUT
# =============================================================================

print("\n" + "=" * 70)
print("ALL TABLES GENERATED SUCCESSFULLY")
print("=" * 70)

print(f"""
KAYDEDILEN DOSYALAR:

   1. {RESULTS_PATH}/Table_1_Model_Comparison.png
      → Method, Precision, Recall, F1-Score, Accuracy, Epoch Duration

   2. {RESULTS_PATH}/Table_2_Detailed_Metrics.png
      → Classifier, Precision%, Recall%, F1-Score%, Accuracy%

   3. {RESULTS_PATH}/Table_3_Performance_Ranking.png
      → Best to worst ranking (SalvationNet marked with *)

RENK KODLARI:
   Green  : Best model (EfficientNetV2-S - 99.61%)
   Yellow : Original model (SalvationNet* - 98.63%)
   Red    : Baseline model (Custom CNN - 32.23%)

NOTE: The * next to SalvationNet indicates that this model
    is an original architecture developed by Omer Faruk Kurtulus.
    
""")

# =============================================================================
# VALIDATION TABLE
# =============================================================================

print("\n" + "=" * 70)
print("DATA VALIDATION - VALUES SOURCED FROM MODEL OUTPUTS")
print("=" * 70)

print("""
┌─────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Model                              │ Accuracy │ Precision │ Recall  │ F1-Score │ Kappa  │ ROC-AUC │
├─────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model 1  - Custom CNN              │  32.23%  │  19.98%   │ 32.23%  │  22.31%  │ 0.0369 │ 0.6027  │
│ Model 2  - EfficientNetV2-S        │  99.61%  │  99.61%   │ 99.61%  │  99.61%  │ 0.9948 │ 1.0000  │
│ Model 3  - ConvNeXt-Tiny           │  98.83%  │  98.84%   │ 98.83%  │  98.83%  │ 0.9845 │ 0.9999  │
│ Model 4  - DenseNet121+InceptionV3 │  99.41%  │  99.42%   │ 99.41%  │  99.42%  │ 0.9923 │ 1.0000  │
│ Model 5  - ResNet50+VGG16          │  98.83%  │  98.84%   │ 98.83%  │  98.83%  │ 0.9845 │ 0.9997  │
│ Model 6  - EffNetB0+MobileNetV2    │  98.83%  │  98.83%   │ 98.83%  │  98.83%  │ 0.9845 │ 0.9997  │
│ Model 7  - Xception+IncResNetV2    │  98.05%  │  98.11%   │ 98.05%  │  98.04%  │ 0.9742 │ 0.9995  │
│ Model 8  - IncResNetV2+CBAM        │  97.85%  │  97.87%   │ 97.85%  │  97.85%  │ 0.9716 │ 0.9990  │
│ Model 9  - Xception+FPN            │  99.22%  │  99.22%   │ 99.22%  │  99.22%  │   -    │ 0.9995  │
│ Model 10 - SalvationNet*           │  98.63%  │  98.66%   │ 98.63%  │  98.64%  │ 0.9819 │ 0.9998  │
└─────────────────────────────────────────────────────────────────────────────────────────────────┘
""")

print("All values are sourced directly from model outputs.")

import gc
gc.collect()